
# Getting Started
Here, we will go over the following with StageNet across all utility modules in PyHealth:

1. Loading the data
2. Task Processing (with padding to ensure compatibility)
3. ML Model Initialization 
4. Model training
5. Holdout Inference on Sets of Codes Not in Vocabulary
6. Interpretability Example with DeepLift

## Installation

Install the latest alpha release of StageNet modernized for PyHealth:

```bash
pip install pyhealth==2.0a10
```

## Loading Data

Load the PyHealth dataset for mortality prediction.

PyHealth datasets use a `config.yaml` file to define:
- Input tables (.csv, .tsv, etc.)
- Features to extract
- Aggregation methods

The result is a single dataframe where each row represents one patient and their features.

For more details on PyHealth datasets, see [this resource](https://colab.research.google.com/drive/1voSx7wEfzXfEf2sIfW6b-8p1KqMyuWxK#scrollTo=NSrb2PGFqUgS).
```

In [1]:
"""
Example of using StageNet for mortality prediction on MIMIC-IV.

This example demonstrates:
1. Loading MIMIC-IV data
2. Applying the MortalityPredictionStageNetMIMIC4 task
3. Creating a SampleDataset with StageNet processors
4. Training a StageNet model
"""

from pyhealth.datasets import (
    MIMIC4Dataset,
    get_dataloader,
    split_by_patient,
)
from pyhealth.models import StageNet
from pyhealth.tasks import MortalityPredictionStageNetMIMIC4
from pyhealth.trainer import Trainer

# STEP 1: Load MIMIC-IV base dataset
base_dataset = MIMIC4Dataset(
    ehr_root="/srv/local/data/physionet.org/files/mimiciv/2.2/",
    ehr_tables=[
        "patients",
        "admissions",
        "diagnoses_icd",
        "procedures_icd",
        "labevents",
    ],
    dev=True
)

/
h
o
m
e
/
R
E
D
A
C
T
E
D
_
U
S
E
R
/
m
i
n
i
c
o
n
d
a
3
/
e
n
v
s
/
m
e
d
i
c
a
l
_
c
o
d
i
n
g
_
d
e
m
o
/
l
i
b
/
p
y
t
h
o
n
3
.
1
0
/
s
i
t
e
-
p
a
c
k
a
g
e
s
/
t
q
d
m
/
a
u
t
o
.
p
y
:
2
1
:
 
T
q
d
m
W
a
r
n
i
n
g
:
 
I
P
r
o
g
r
e
s
s
 
n
o
t
 
f
o
u
n
d
.
 
P
l
e
a
s
e
 
u
p
d
a
t
e
 
j
u
p
y
t
e
r
 
a
n
d
 
i
p
y
w
i
d
g
e
t
s
.
 
S
e
e
 
h
t
t
p
s
:
/
/
i
p
y
w
i
d
g
e
t
s
.
r
e
a
d
t
h
e
d
o
c
s
.
i
o
/
e
n
/
s
t
a
b
l
e
/
u
s
e
r
_
i
n
s
t
a
l
l
.
h
t
m
l

 
 
f
r
o
m
 
.
a
u
t
o
n
o
t
e
b
o
o
k
 
i
m
p
o
r
t
 
t
q
d
m
 
a
s
 
n
o
t
e
b
o
o
k
_
t
q
d
m



M
e
m
o
r
y
 
u
s
a
g
e
 
S
t
a
r
t
i
n
g
 
M
I
M
I
C
4
D
a
t
a
s
e
t
 
i
n
i
t
:
 
7
9
7
.
9
 
M
B

I
n
i
t
i
a
l
i
z
i
n
g
 
M
I
M
I
C
4
E
H
R
D
a
t
a
s
e
t
 
w
i
t
h
 
t
a
b
l
e
s
:
 
[
'
p
a
t
i
e
n
t
s
'
,
 
'
a
d
m
i
s
s
i
o
n
s
'
,
 
'
d
i
a
g
n
o
s
e
s
_
i
c
d
'
,
 
'
p
r
o
c
e
d
u
r
e
s
_
i
c
d
'
,
 
'
l
a
b
e
v
e
n
t
s
'
]
 
(
d
e
v
 
m
o
d
e
:
 
T
r
u
e
)

U
s
i
n
g
 
d
e
f
a
u
l
t
 
E
H
R
 
c
o
n
f
i
g
:
 
/
h
o
m
e
/
R
E
D
A
C
T
E
D
_
U
S
E
R
/
p
r
o
j
e
c
t
s
/
P
y
H
e
a
l
t
h
_
B
r
a
n
c
h
_
T
e
s
t
i
n
g
/
P
y
H
e
a
l
t
h
/
p
y
h
e
a
l
t
h
/
d
a
t
a
s
e
t
s
/
c
o
n
f
i
g
s
/
m
i
m
i
c
4
_
e
h
r
.
y
a
m
l

M
e
m
o
r
y
 
u
s
a
g
e
 
B
e
f
o
r
e
 
i
n
i
t
i
a
l
i
z
i
n
g
 
m
i
m
i
c
4
_
e
h
r
:
 
7
9
7
.
9
 
M
B

D
u
p
l
i
c
a
t
e
 
t
a
b
l
e
 
n
a
m
e
s
 
i
n
 
t
a
b
l
e
s
 
l
i
s
t
.
 
R
e
m
o
v
i
n
g
 
d
u
p
l
i
c
a
t
e
s
.

I
n
i
t
i
a
l
i
z
i
n
g
 
m
i
m
i
c
4
_
e
h
r
 
d
a
t
a
s
e
t
 
f
r
o
m
 
/
s
r
v
/
l
o
c
a
l
/
d
a
t
a
/
p
h
y
s
i
o
n
e
t
.
o
r
g
/
f
i
l
e
s
/
m

#
#
 
I
n
p
u
t
 
a
n
d
 
O
u
t
p
u
t
 
S
c
h
e
m
a
s

I
n
p
u
t
 
a
n
d
 
o
u
t
p
u
t
 
s
c
h
e
m
a
s
 
m
a
p
 
f
e
a
t
u
r
e
 
k
e
y
s
 
(
e
.
g
.
,
 
"
l
a
b
s
"
,
 
"
i
c
d
_
c
o
d
e
s
"
)
 
t
o
 
S
t
a
g
e
N
e
t
 
p
r
o
c
e
s
s
o
r
s
.
 
E
a
c
h
 
p
r
o
c
e
s
s
o
r
 
c
o
n
v
e
r
t
s
 
f
e
a
t
u
r
e
s
 
i
n
t
o
 
t
u
p
l
e
 
o
b
j
e
c
t
s
 
u
s
e
d
 
f
o
r
 
t
r
a
i
n
i
n
g
 
a
n
d
 
i
n
f
e
r
e
n
c
e
.


*
*
R
e
q
u
i
r
e
d
 
f
o
r
m
a
t
:
*
*
 
E
a
c
h
 
f
e
a
t
u
r
e
 
p
r
o
c
e
s
s
e
d
 
i
n
 
o
u
r
 
t
a
s
k
 
c
a
l
l
 
m
u
s
t
 
f
o
l
l
o
w
 
t
h
i
s
 
s
t
r
u
c
t
u
r
e
:

`
`
`
p
y
t
h
o
n

"
f
e
a
t
u
r
e
"
:
 
(
m
y
_
t
i
m
e
s
_
l
i
s
t
,
 
m
y
_
v
a
l
u
e
s
_
l
i
s
t
)

`
`
`

W
e
 
o
f
f
e
r
 
t
w
o
 
t
y
p
e
s
 
o
f
 
S
t
a
g
e
N
e
t
 
p
r
o
c
e
s
s
o
r
s
,
 
o
n
e
 
f
o
r
 
c
a
t
e
g
o
r
i
c
a
l
 
v
a
r
i
a
b
l
e
s
,
 
a
n
d
 
t
h
e
 
o
t
h
e
r
 
f
o
r
 
n
u
m
e
r
i
c
a
l
 
f
e
a
t
u
r
e
 
v
a
r
i
a
b
l
e
s
.
 
O
u
r
 
g
o
a
l
 
h
e
r
e
 
i
s
 
t
o
 
r
e
p
r
e
s
e
n
t
 
e
a
c
h
 
f
e
a
t
u
r
e
 
a
s
 
a
 
p
r
e
-
d
e
f
i
n
e
d
 
t
u
p
l
e
 
(
t
i
m
e
,
 
v
a
l
u
e
)
 
t
h
a
t
 
w
e
 
c
a
n
 
l
a
t
e
r
 
p
a
s
s
 
t
o
 
S
t
a
g
e
N
e
t
 
f
o
r
 
p
r
o
c
e
s
s
i
n
g
.



#
#
 
W
h
a
t
 
a
r
e
 
t
h
e
s
e
 
p
r
o
c
e
s
s
o
r
s
?


E
f
f
e
c
t
i
v
e
l
y
 
p
r
o
c
e
s
s
o
r
s
 
t
a
k
e
 
e
x
i
s
t
i
n
g
 
d
a
t
a
 
v
a
r
i
a
b
l
e
s
 
a
n
d
 
t
u
r
n
s
 
t
h
e
m
 
i
n
t
o
 
a
 
t
e
n
s
o
r
 
f
o
r
m
a
t
.
 
H
e
r
e
,
 
w
e
 
d
e
f
i
n
e
 
a
 
s
e
t
 
o
f
 
c
u
s
t
o
m
 
p
r
o
c
e
s
s
o
r
s
 
s
o
 
w
e
 
c
a
n
 
l
e
v
e
r
a
g
e
 
S
t
a
g
e
N
e
t
'
s
 
a
b
i
l
i
t
y
 
t
o
 
t
a
k
e
 
i
n
 
a
 
t
i
m
e
-
s
e
r
i
e
s
 
s
e
t
 
o
f
 
t
i
m
e
 
i
n
t
e
r
v
a
l
s
 
a
n
d
 
f
e
a
t
u
r
e
 
s
e
t
s
.


#
#
 
S
t
a
g
e
N
e
t
P
r
o
c
e
s
s
o
r
 
-
 
F
o
r
 
C
a
t
e
g
o
r
i
e
s
 
(
L
a
b
e
l
s
)


*
*
W
h
a
t
 
i
t
 
h
a
n
d
l
e
s
:
*
*
 
T
e
x
t
 
l
a
b
e
l
s
 
l
i
k
e
 
d
i
a
g
n
o
s
i
s
 
c
o
d
e
s
,
 
m
e
d
i
c
a
t
i
o
n
 
n
a
m
e
s
,
 
o
r
 
l
a
b
 
t
e
s
t
 
t
y
p
e
s
.


*
*
W
h
a
t
 
i
t
 
d
o
e
s
:
*
*

-
 
T
a
k
e
s
 
l
i
s
t
s
 
o
f
 
c
o
d
e
s
 
(
l
i
k
e
 
`
[
"
d
i
a
b
e
t
e
s
"
,
 
"
h
y
p
e
r
t
e
n
s
i
o
n
"
]
`
)

-
 
C
o
n
v
e
r
t
s
 
e
a
c
h
 
w
o
r
d
 
i
n
t
o
 
a
 
u
n
i
q
u
e
 
n
u
m
b
e
r
 
(
l
i
k
e
 
`
[
"
d
i
a
b
e
t
e
s
"
=
1
,
 
"
h
y
p
e
r
t
e
n
s
i
o
n
"
=
2
]
`
)

-
 
K
e
e
p
s
 
t
r
a
c
k
 
o
f
 
w
h
e
n
 
t
h
i
n
g
s
 
h
a
p
p
e
n
e
d
 
(
t
i
m
e
s
t
a
m
p
s
)

-
 
C
a
n
 
h
a
n
d
l
e
 
n
e
s
t
e
d
 
l
i
s
t
s
 
(
l
i
k
e
 
m
u
l
t
i
p
l
e
 
c
o
d
e
s
 
p
e
r
 
v
i
s
i
t
)


*
*
E
x
a
m
p
l
e
:
*
*
 
I
f
 
a
 
p
a
t
i
e
n
t
 
h
a
d
 
3
 
d
o
c
t
o
r
 
v
i
s
i
t
s
 
w
i
t
h
 
d
i
f
f
e
r
e
n
t
 
d
i
a
g
n
o
s
e
s
,
 
t
h
i
s
 
p
r
o
c
e
s
s
o
r
 
r
e
m
e
m
b
e
r
s
 
w
h
a
t
 
d
i
a
g
n
o
s
i
s
 
h
a
p
p
e
n
e
d
 
a
t
 
e
a
c
h
 
v
i
s
i
t
 
a
n
d
 
w
h
e
n
.


#
#
 
S
t
a
g
e
N
e
t
T
e
n
s
o
r
P
r
o
c
e
s
s
o
r
 
-
 
F
o
r
 
N
u
m
b
e
r
s
 
(
M
e
a
s
u
r
e
m
e
n
t
s
)


*
*
W
h
a
t
 
i
t
 
h
a
n
d
l
e
s
:
*
*
 
A
c
t
u
a
l
 
m
e
a
s
u
r
e
m
e
n
t
s
 
l
i
k
e
 
b
l
o
o
d
 
p
r
e
s
s
u
r
e
,
 
t
e
m
p
e
r
a
t
u
r
e
,
 
o
r
 
l
a
b
 
v
a
l
u
e
s
.


*
*
W
h
a
t
 
i
t
 
d
o
e
s
:
*
*

-
 
T
a
k
e
s
 
l
i
s
t
s
 
o
f
 
n
u
m
b
e
r
s
 
(
l
i
k
e
 
`
[
9
8
.
6
,
 
9
9
.
1
,
 
9
8
.
8
]
`
 
f
o
r
 
t
e
m
p
e
r
a
t
u
r
e
s
)

-
 
F
i
l
l
s
 
i
n
 
m
i
s
s
i
n
g
 
m
e
a
s
u
r
e
m
e
n
t
s
 
u
s
i
n
g
 
t
h
e
 
l
a
s
t
 
k
n
o
w
n
 
v
a
l
u
e
 
(
f
o
r
w
a
r
d
-
f
i
l
l
)

-
 
K
e
e
p
s
 
t
r
a
c
k
 
o
f
 
w
h
e
n
 
m
e
a
s
u
r
e
m
e
n
t
s
 
w
e
r
e
 
t
a
k
e
n

-
 
C
a
n
 
h
a
n
d
l
e
 
m
u
l
t
i
p
l
e
 
m
e
a
s
u
r
e
m
e
n
t
s
 
a
t
 
o
n
c
e
 
(
l
i
k
e
 
b
l
o
o
d
 
p
r
e
s
s
u
r
e
 
A
N
D
 
h
e
a
r
t
 
r
a
t
e
)


*
*
E
x
a
m
p
l
e
:
*
*
 
I
f
 
a
 
p
a
t
i
e
n
t
'
s
 
h
e
a
r
t
 
r
a
t
e
 
w
a
s
 
m
e
a
s
u
r
e
d
 
a
s
 
`
[
7
2
,
 
N
o
n
e
,
 
6
8
]
`
,
 
i
t
 
f
i
l
l
s
 
i
n
 
t
h
e
 
m
i
s
s
i
n
g
 
v
a
l
u
e
 
a
s
 
`
[
7
2
,
 
7
2
,
 
6
8
]
`
 
(
c
o
p
y
i
n
g
 
t
h
e
 
l
a
s
t
 
k
n
o
w
n
 
v
a
l
u
e
)
.


#
#
 
H
o
w
 
T
i
m
e
 
P
r
o
c
e
s
s
i
n
g
 
W
o
r
k
s


B
o
t
h
 
p
r
o
c
e
s
s
o
r
s
 
h
a
n
d
l
e
 
t
i
m
e
 
i
n
f
o
r
m
a
t
i
o
n
 
i
n
 
a
 
f
l
e
x
i
b
l
e
 
w
a
y
:


*
*
I
n
p
u
t
 
f
o
r
m
a
t
s
 
a
c
c
e
p
t
e
d
:
*
*

-
 
S
i
m
p
l
e
 
l
i
s
t
:
 
`
[
0
.
0
,
 
1
.
5
,
 
3
.
0
]
`
 
-
 
t
i
m
e
 
i
n
t
e
r
v
a
l
s
 
i
n
 
h
o
u
r
s
/
d
a
y
s

-
 
N
e
s
t
e
d
 
l
i
s
t
:
 
`
[
[
0
.
0
]
,
 
[
1
.
5
]
,
 
[
3
.
0
]
]
`
 
-
 
a
u
t
o
m
a
t
i
c
a
l
l
y
 
f
l
a
t
t
e
n
e
d

-
 
N
o
 
t
i
m
e
:
 
`
N
o
n
e
`
 
-
 
w
h
e
n
 
t
i
m
i
n
g
 
d
o
e
s
n
'
t
 
m
a
t
t
e
r


*
*
W
h
a
t
 
t
h
e
 
t
i
m
e
 
m
e
a
n
s
:
*
*

-
 
T
i
m
e
s
 
r
e
p
r
e
s
e
n
t
 
i
n
t
e
r
v
a
l
s
 
o
r
 
d
e
l
a
y
s
 
b
e
t
w
e
e
n
 
e
v
e
n
t
s

-
 
F
o
r
 
e
x
a
m
p
l
e
:
 
`
[
0
.
0
,
 
2
.
5
,
 
1
.
0
]
`
 
c
o
u
l
d
 
m
e
a
n
 
"
f
i
r
s
t
 
e
v
e
n
t
 
a
t
 
s
t
a
r
t
,
 
s
e
c
o
n
d
 
e
v
e
n
t
 
2
.
5
 
h
o
u
r
s
 
l
a
t
e
r
,
 
t
h
i
r
d
 
e
v
e
n
t
 
1
 
h
o
u
r
 
a
f
t
e
r
 
t
h
a
t
"

-
 
T
i
m
e
s
 
a
r
e
 
c
o
n
v
e
r
t
e
d
 
t
o
 
f
l
o
a
t
 
t
e
n
s
o
r
s
 
s
o
 
t
h
e
 
m
o
d
e
l
 
c
a
n
 
l
e
a
r
n
 
t
e
m
p
o
r
a
l
 
p
a
t
t
e
r
n
s


*
*
E
x
a
m
p
l
e
:
*
*

`
`
`
p
y
t
h
o
n

#
 
P
a
t
i
e
n
t
 
t
e
m
p
e
r
a
t
u
r
e
 
r
e
a
d
i
n
g
s

d
a
t
a
 
=
 
{

 
 
 
 
"
v
a
l
u
e
"
:
 
[
9
8
.
6
,
 
9
9
.
1
,
 
9
8
.
8
]
,
 
 
#
 
t
e
m
p
e
r
a
t
u
r
e
s
 
i
n
 
°
F

 
 
 
 
"
t
i
m
e
"
:
 
[
0
.
0
,
 
2
.
0
,
 
1
.
0
]
 
 
 
 
 
 
 
 
#
 
h
o
u
r
s
 
s
i
n
c
e
 
p
r
e
v
i
o
u
s
 
a
d
m
i
s
s
i
o
n
s

}

`
`
`


T
h
e
 
p
r
o
c
e
s
s
o
r
 
k
e
e
p
s
 
t
h
e
 
t
i
m
e
 
a
n
d
 
v
a
l
u
e
s
 
p
a
i
r
e
d
 
t
o
g
e
t
h
e
r
,
 
s
o
 
t
h
e
 
m
o
d
e
l
 
k
n
o
w
s
 
t
h
a
t
 
9
9
.
1
°
F
 
w
a
s
 
r
e
c
o
r
d
e
d
 
a
t
 
2
 
h
o
u
r
s
 
a
f
t
e
r
 
a
d
m
i
s
s
i
o
n
.


F
o
r
 
s
y
n
t
a
c
t
i
c
 
r
e
a
s
o
n
s
,
 
w
e
 
a
d
d
 
t
h
e
 
s
u
f
f
i
x
 
"
E
x
"
 
a
s
 
t
h
e
y
'
r
e
 
a
l
r
e
a
d
y
 
i
m
p
l
e
m
e
n
t
e
d
 
i
n
 
P
y
H
e
a
l
t
h
.
 
T
h
i
s
 
i
s
 
m
o
r
e
 
t
o
 
s
h
o
w
c
a
s
e
 
w
h
a
t
'
s
 
h
a
p
p
e
n
i
n
g
 
u
n
d
e
r
n
e
a
t
h
 
t
h
e
 
h
o
o
d
.

In [2]:
f
r
o
m
 
t
y
p
i
n
g
 
i
m
p
o
r
t
 
A
n
y
,
 
D
i
c
t
,
 
L
i
s
t
,
 
O
p
t
i
o
n
a
l
,
 
T
u
p
l
e


i
m
p
o
r
t
 
t
o
r
c
h


f
r
o
m
 
p
y
h
e
a
l
t
h
.
p
r
o
c
e
s
s
o
r
s
 
i
m
p
o
r
t
 
r
e
g
i
s
t
e
r
_
p
r
o
c
e
s
s
o
r

f
r
o
m
 
p
y
h
e
a
l
t
h
.
p
r
o
c
e
s
s
o
r
s
.
b
a
s
e
_
p
r
o
c
e
s
s
o
r
 
i
m
p
o
r
t
 
F
e
a
t
u
r
e
P
r
o
c
e
s
s
o
r


@
r
e
g
i
s
t
e
r
_
p
r
o
c
e
s
s
o
r
(
"
s
t
a
g
e
n
e
t
_
e
x
"
)

c
l
a
s
s
 
S
t
a
g
e
N
e
t
P
r
o
c
e
s
s
o
r
(
F
e
a
t
u
r
e
P
r
o
c
e
s
s
o
r
)
:

 
 
 
 
"
"
"

 
 
 
 
F
e
a
t
u
r
e
 
p
r
o
c
e
s
s
o
r
 
f
o
r
 
S
t
a
g
e
N
e
t
 
C
O
D
E
 
i
n
p
u
t
s
 
w
i
t
h
 
c
o
u
p
l
e
d
 
v
a
l
u
e
/
t
i
m
e
 
d
a
t
a
.


 
 
 
 
T
h
i
s
 
p
r
o
c
e
s
s
o
r
 
h
a
n
d
l
e
s
 
c
a
t
e
g
o
r
i
c
a
l
 
c
o
d
e
 
s
e
q
u
e
n
c
e
s
 
(
f
l
a
t
 
o
r
 
n
e
s
t
e
d
)
.

 
 
 
 
F
o
r
 
n
u
m
e
r
i
c
 
f
e
a
t
u
r
e
s
,
 
u
s
e
 
S
t
a
g
e
N
e
t
T
e
n
s
o
r
P
r
o
c
e
s
s
o
r
 
i
n
s
t
e
a
d
.


 
 
 
 
I
n
p
u
t
 
F
o
r
m
a
t
 
(
t
u
p
l
e
)
:

 
 
 
 
 
 
 
 
(
t
i
m
e
,
 
v
a
l
u
e
s
)
 
w
h
e
r
e
:

 
 
 
 
 
 
 
 
-
 
t
i
m
e
:
 
L
i
s
t
 
o
f
 
s
c
a
l
a
r
s
 
[
0
.
0
,
 
2
.
0
,
 
1
.
3
]
 
o
r
 
N
o
n
e

 
 
 
 
 
 
 
 
-
 
v
a
l
u
e
s
:
 
[
"
c
o
d
e
1
"
,
 
"
c
o
d
e
2
"
]
 
o
r
 
[
[
"
A
"
,
 
"
B
"
]
,
 
[
"
C
"
]
]


 
 
 
 
T
h
e
 
p
r
o
c
e
s
s
o
r
 
a
u
t
o
m
a
t
i
c
a
l
l
y
 
d
e
t
e
c
t
s
:

 
 
 
 
-
 
L
i
s
t
 
o
f
 
s
t
r
i
n
g
s
 
-
>
 
f
l
a
t
 
c
o
d
e
 
s
e
q
u
e
n
c
e
s

 
 
 
 
-
 
L
i
s
t
 
o
f
 
l
i
s
t
s
 
o
f
 
s
t
r
i
n
g
s
 
-
>
 
n
e
s
t
e
d
 
c
o
d
e
 
s
e
q
u
e
n
c
e
s


 
 
 
 
A
r
g
s
:

 
 
 
 
 
 
 
 
p
a
d
d
i
n
g
:
 
A
d
d
i
t
i
o
n
a
l
 
p
a
d
d
i
n
g
 
t
o
 
a
d
d
 
o
n
 
t
o
p
 
o
f
 
t
h
e
 
o
b
s
e
r
v
e
d
 
m
a
x
i
m
u
m
 
n
e
s
t
e
d

 
 
 
 
 
 
 
 
 
 
 
 
s
e
q
u
e
n
c
e
 
l
e
n
g
t
h
.
 
T
h
e
 
a
c
t
u
a
l
 
p
a
d
d
i
n
g
 
l
e
n
g
t
h
 
w
i
l
l
 
b
e
 
o
b
s
e
r
v
e
d
_
m
a
x
 
+
 
p
a
d
d
i
n
g
.

 
 
 
 
 
 
 
 
 
 
 
 
T
h
i
s
 
e
n
s
u
r
e
s
 
t
h
e
 
p
r
o
c
e
s
s
o
r
 
c
a
n
 
h
a
n
d
l
e
 
s
e
q
u
e
n
c
e
s
 
l
o
n
g
e
r
 
t
h
a
n
 
t
h
o
s
e
 
i
n
 
t
h
e

 
 
 
 
 
 
 
 
 
 
 
 
t
r
a
i
n
i
n
g
 
d
a
t
a
.
 
D
e
f
a
u
l
t
:
 
0
 
(
n
o
 
e
x
t
r
a
 
p
a
d
d
i
n
g
)
.
 
O
n
l
y
 
a
p
p
l
i
e
s
 
t
o
 
n
e
s
t
e
d
 
s
e
q
u
e
n
c
e
s
.


 
 
 
 
R
e
t
u
r
n
s
:

 
 
 
 
 
 
 
 
T
u
p
l
e
 
o
f
 
(
t
i
m
e
_
t
e
n
s
o
r
,
 
v
a
l
u
e
_
t
e
n
s
o
r
)
 
w
h
e
r
e
 
t
i
m
e
_
t
e
n
s
o
r
 
c
a
n
 
b
e
 
N
o
n
e


 
 
 
 
E
x
a
m
p
l
e
s
:

 
 
 
 
 
 
 
 
>
>
>
 
#
 
C
a
s
e
 
1
:
 
C
o
d
e
 
s
e
q
u
e
n
c
e
 
w
i
t
h
 
t
i
m
e

 
 
 
 
 
 
 
 
>
>
>
 
p
r
o
c
e
s
s
o
r
 
=
 
S
t
a
g
e
N
e
t
P
r
o
c
e
s
s
o
r
(
)

 
 
 
 
 
 
 
 
>
>
>
 
d
a
t
a
 
=
 
(
[
0
.
0
,
 
1
.
5
,
 
2
.
3
]
,
 
[
"
c
o
d
e
1
"
,
 
"
c
o
d
e
2
"
,
 
"
c
o
d
e
3
"
]
)

 
 
 
 
 
 
 
 
>
>
>
 
t
i
m
e
,
 
v
a
l
u
e
s
 
=
 
p
r
o
c
e
s
s
o
r
.
p
r
o
c
e
s
s
(
d
a
t
a
)

 
 
 
 
 
 
 
 
>
>
>
 
v
a
l
u
e
s
.
s
h
a
p
e
 
 
#
 
(
3
,
)
 
-
 
s
e
q
u
e
n
c
e
 
o
f
 
c
o
d
e
 
i
n
d
i
c
e
s

 
 
 
 
 
 
 
 
>
>
>
 
t
i
m
e
.
s
h
a
p
e
 
 
 
 
#
 
(
3
,
)
 
-
 
t
i
m
e
 
i
n
t
e
r
v
a
l
s


 
 
 
 
 
 
 
 
>
>
>
 
#
 
C
a
s
e
 
2
:
 
N
e
s
t
e
d
 
c
o
d
e
s
 
w
i
t
h
 
t
i
m
e
 
(
w
i
t
h
 
c
u
s
t
o
m
 
p
a
d
d
i
n
g
 
f
o
r
 
e
x
t
r
a
 
c
a
p
a
c
i
t
y
)

 
 
 
 
 
 
 
 
>
>
>
 
p
r
o
c
e
s
s
o
r
 
=
 
S
t
a
g
e
N
e
t
P
r
o
c
e
s
s
o
r
(
p
a
d
d
i
n
g
=
2
0
)

 
 
 
 
 
 
 
 
>
>
>
 
d
a
t
a
 
=
 
(
[
0
.
0
,
 
1
.
5
]
,
 
[
[
"
A
"
,
 
"
B
"
]
,
 
[
"
C
"
]
]
)

 
 
 
 
 
 
 
 
>
>
>
 
t
i
m
e
,
 
v
a
l
u
e
s
 
=
 
p
r
o
c
e
s
s
o
r
.
p
r
o
c
e
s
s
(
d
a
t
a
)

 
 
 
 
 
 
 
 
>
>
>
 
v
a
l
u
e
s
.
s
h
a
p
e
 
 
#
 
(
2
,
 
o
b
s
e
r
v
e
d
_
m
a
x
 
+
 
2
0
)
 
-
 
p
a
d
d
e
d
 
n
e
s
t
e
d
 
s
e
q
u
e
n
c
e
s

 
 
 
 
 
 
 
 
>
>
>
 
t
i
m
e
.
s
h
a
p
e
 
 
 
 
#
 
(
2
,
)


 
 
 
 
 
 
 
 
>
>
>
 
#
 
C
a
s
e
 
3
:
 
C
o
d
e
s
 
w
i
t
h
o
u
t
 
t
i
m
e

 
 
 
 
 
 
 
 
>
>
>
 
d
a
t
a
 
=
 
(
N
o
n
e
,
 
[
"
c
o
d
e
1
"
,
 
"
c
o
d
e
2
"
]
)

 
 
 
 
 
 
 
 
>
>
>
 
t
i
m
e
,
 
v
a
l
u
e
s
 
=
 
p
r
o
c
e
s
s
o
r
.
p
r
o
c
e
s
s
(
d
a
t
a
)

 
 
 
 
 
 
 
 
>
>
>
 
v
a
l
u
e
s
.
s
h
a
p
e
 
 
#
 
(
2
,
)

 
 
 
 
 
 
 
 
>
>
>
 
t
i
m
e
 
 
 
 
 
 
 
 
 
 
#
 
N
o
n
e

 
 
 
 
"
"
"


 
 
 
 
d
e
f
 
_
_
i
n
i
t
_
_
(
s
e
l
f
,
 
p
a
d
d
i
n
g
:
 
i
n
t
 
=
 
0
)
:

 
 
 
 
 
 
 
 
#
 
<
u
n
k
>
 
w
i
l
l
 
b
e
 
s
e
t
 
t
o
 
l
e
n
(
v
o
c
a
b
)
 
a
f
t
e
r
 
f
i
t

 
 
 
 
 
 
 
 
s
e
l
f
.
c
o
d
e
_
v
o
c
a
b
:
 
D
i
c
t
[
A
n
y
,
 
i
n
t
]
 
=
 
{
"
<
u
n
k
>
"
:
 
N
o
n
e
,
 
"
<
p
a
d
>
"
:
 
0
}

 
 
 
 
 
 
 
 
s
e
l
f
.
_
n
e
x
t
_
i
n
d
e
x
 
=
 
1

 
 
 
 
 
 
 
 
s
e
l
f
.
_
i
s
_
n
e
s
t
e
d
 
=
 
N
o
n
e
 
 
#
 
W
i
l
l
 
b
e
 
d
e
t
e
r
m
i
n
e
d
 
d
u
r
i
n
g
 
f
i
t

 
 
 
 
 
 
 
 
#
 
M
a
x
 
i
n
n
e
r
 
s
e
q
u
e
n
c
e
 
l
e
n
g
t
h
 
f
o
r
 
n
e
s
t
e
d
 
c
o
d
e
s

 
 
 
 
 
 
 
 
s
e
l
f
.
_
m
a
x
_
n
e
s
t
e
d
_
l
e
n
 
=
 
N
o
n
e

 
 
 
 
 
 
 
 
s
e
l
f
.
_
p
a
d
d
i
n
g
 
=
 
p
a
d
d
i
n
g
 
 
#
 
A
d
d
i
t
i
o
n
a
l
 
p
a
d
d
i
n
g
 
b
e
y
o
n
d
 
o
b
s
e
r
v
e
d
 
m
a
x


 
 
 
 
d
e
f
 
f
i
t
(
s
e
l
f
,
 
s
a
m
p
l
e
s
:
 
L
i
s
t
[
D
i
c
t
]
,
 
k
e
y
:
 
s
t
r
)
 
-
>
 
N
o
n
e
:

 
 
 
 
 
 
 
 
"
"
"
B
u
i
l
d
 
v
o
c
a
b
u
l
a
r
y
 
a
n
d
 
d
e
t
e
r
m
i
n
e
 
i
n
p
u
t
 
s
t
r
u
c
t
u
r
e
.


 
 
 
 
 
 
 
 
A
r
g
s
:

 
 
 
 
 
 
 
 
 
 
 
 
s
a
m
p
l
e
s
:
 
L
i
s
t
 
o
f
 
s
a
m
p
l
e
 
d
i
c
t
i
o
n
a
r
i
e
s

 
 
 
 
 
 
 
 
 
 
 
 
k
e
y
:
 
T
h
e
 
k
e
y
 
i
n
 
s
a
m
p
l
e
s
 
t
h
a
t
 
c
o
n
t
a
i
n
s
 
t
u
p
l
e
 
(
t
i
m
e
,
 
v
a
l
u
e
s
)

 
 
 
 
 
 
 
 
"
"
"

 
 
 
 
 
 
 
 
#
 
E
x
a
m
i
n
e
 
f
i
r
s
t
 
n
o
n
-
N
o
n
e
 
s
a
m
p
l
e
 
t
o
 
d
e
t
e
r
m
i
n
e
 
s
t
r
u
c
t
u
r
e

 
 
 
 
 
 
 
 
f
o
r
 
s
a
m
p
l
e
 
i
n
 
s
a
m
p
l
e
s
:

 
 
 
 
 
 
 
 
 
 
 
 
i
f
 
k
e
y
 
i
n
 
s
a
m
p
l
e
 
a
n
d
 
s
a
m
p
l
e
[
k
e
y
]
 
i
s
 
n
o
t
 
N
o
n
e
:

 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
#
 
U
n
p
a
c
k
 
t
u
p
l
e
:
 
(
t
i
m
e
,
 
v
a
l
u
e
s
)

 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
t
i
m
e
_
d
a
t
a
,
 
v
a
l
u
e
_
d
a
t
a
 
=
 
s
a
m
p
l
e
[
k
e
y
]


 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
#
 
D
e
t
e
r
m
i
n
e
 
n
e
s
t
i
n
g
 
l
e
v
e
l
 
f
o
r
 
c
o
d
e
s

 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
i
f
 
i
s
i
n
s
t
a
n
c
e
(
v
a
l
u
e
_
d
a
t
a
,
 
l
i
s
t
)
 
a
n
d
 
l
e
n
(
v
a
l
u
e
_
d
a
t
a
)
 
>
 
0
:

 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
f
i
r
s
t
_
e
l
e
m
 
=
 
v
a
l
u
e
_
d
a
t
a
[
0
]


 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
i
f
 
i
s
i
n
s
t
a
n
c
e
(
f
i
r
s
t
_
e
l
e
m
,
 
s
t
r
)
:

 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
#
 
C
a
s
e
 
1
:
 
[
"
c
o
d
e
1
"
,
 
"
c
o
d
e
2
"
,
 
.
.
.
]

 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
s
e
l
f
.
_
i
s
_
n
e
s
t
e
d
 
=
 
F
a
l
s
e

 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
e
l
i
f
 
i
s
i
n
s
t
a
n
c
e
(
f
i
r
s
t
_
e
l
e
m
,
 
l
i
s
t
)
:

 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
i
f
 
l
e
n
(
f
i
r
s
t
_
e
l
e
m
)
 
>
 
0
 
a
n
d
 
i
s
i
n
s
t
a
n
c
e
(
f
i
r
s
t
_
e
l
e
m
[
0
]
,
 
s
t
r
)
:

 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
#
 
C
a
s
e
 
2
:
 
[
[
"
A
"
,
 
"
B
"
]
,
 
[
"
C
"
]
,
 
.
.
.
]

 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
s
e
l
f
.
_
i
s
_
n
e
s
t
e
d
 
=
 
T
r
u
e

 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
b
r
e
a
k


 
 
 
 
 
 
 
 
#
 
B
u
i
l
d
 
v
o
c
a
b
u
l
a
r
y
 
f
o
r
 
c
o
d
e
s
 
a
n
d
 
f
i
n
d
 
m
a
x
 
n
e
s
t
e
d
 
l
e
n
g
t
h

 
 
 
 
 
 
 
 
m
a
x
_
i
n
n
e
r
_
l
e
n
 
=
 
0

 
 
 
 
 
 
 
 
f
o
r
 
s
a
m
p
l
e
 
i
n
 
s
a
m
p
l
e
s
:

 
 
 
 
 
 
 
 
 
 
 
 
i
f
 
k
e
y
 
i
n
 
s
a
m
p
l
e
 
a
n
d
 
s
a
m
p
l
e
[
k
e
y
]
 
i
s
 
n
o
t
 
N
o
n
e
:

 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
#
 
U
n
p
a
c
k
 
t
u
p
l
e
:
 
(
t
i
m
e
,
 
v
a
l
u
e
s
)

 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
t
i
m
e
_
d
a
t
a
,
 
v
a
l
u
e
_
d
a
t
a
 
=
 
s
a
m
p
l
e
[
k
e
y
]


 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
i
f
 
s
e
l
f
.
_
i
s
_
n
e
s
t
e
d
:

 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
#
 
N
e
s
t
e
d
 
c
o
d
e
s

 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
f
o
r
 
i
n
n
e
r
_
l
i
s
t
 
i
n
 
v
a
l
u
e
_
d
a
t
a
:

 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
#
 
T
r
a
c
k
 
m
a
x
 
i
n
n
e
r
 
l
e
n
g
t
h

 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
m
a
x
_
i
n
n
e
r
_
l
e
n
 
=
 
m
a
x
(
m
a
x
_
i
n
n
e
r
_
l
e
n
,
 
l
e
n
(
i
n
n
e
r
_
l
i
s
t
)
)

 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
f
o
r
 
c
o
d
e
 
i
n
 
i
n
n
e
r
_
l
i
s
t
:

 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
i
f
 
c
o
d
e
 
i
s
 
n
o
t
 
N
o
n
e
 
a
n
d
 
c
o
d
e
 
n
o
t
 
i
n
 
s
e
l
f
.
c
o
d
e
_
v
o
c
a
b
:

 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
s
e
l
f
.
c
o
d
e
_
v
o
c
a
b
[
c
o
d
e
]
 
=
 
s
e
l
f
.
_
n
e
x
t
_
i
n
d
e
x

 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
s
e
l
f
.
_
n
e
x
t
_
i
n
d
e
x
 
+
=
 
1

 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
e
l
s
e
:

 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
#
 
F
l
a
t
 
c
o
d
e
s

 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
f
o
r
 
c
o
d
e
 
i
n
 
v
a
l
u
e
_
d
a
t
a
:

 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
i
f
 
c
o
d
e
 
i
s
 
n
o
t
 
N
o
n
e
 
a
n
d
 
c
o
d
e
 
n
o
t
 
i
n
 
s
e
l
f
.
c
o
d
e
_
v
o
c
a
b
:

 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
s
e
l
f
.
c
o
d
e
_
v
o
c
a
b
[
c
o
d
e
]
 
=
 
s
e
l
f
.
_
n
e
x
t
_
i
n
d
e
x

 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
s
e
l
f
.
_
n
e
x
t
_
i
n
d
e
x
 
+
=
 
1


 
 
 
 
 
 
 
 
#
 
S
t
o
r
e
 
m
a
x
 
n
e
s
t
e
d
 
l
e
n
g
t
h
:
 
a
d
d
 
u
s
e
r
-
s
p
e
c
i
f
i
e
d
 
p
a
d
d
i
n
g
 
t
o
 
o
b
s
e
r
v
e
d
 
m
a
x
i
m
u
m

 
 
 
 
 
 
 
 
#
 
T
h
i
s
 
e
n
s
u
r
e
s
 
t
h
e
 
p
r
o
c
e
s
s
o
r
 
c
a
n
 
h
a
n
d
l
e
 
s
e
q
u
e
n
c
e
s
 
l
o
n
g
e
r
 
t
h
a
n
 
t
h
o
s
e
 
i
n
 
t
r
a
i
n
i
n
g
 
d
a
t
a

 
 
 
 
 
 
 
 
i
f
 
s
e
l
f
.
_
i
s
_
n
e
s
t
e
d
:

 
 
 
 
 
 
 
 
 
 
 
 
o
b
s
e
r
v
e
d
_
m
a
x
 
=
 
m
a
x
(
1
,
 
m
a
x
_
i
n
n
e
r
_
l
e
n
)

 
 
 
 
 
 
 
 
 
 
 
 
s
e
l
f
.
_
m
a
x
_
n
e
s
t
e
d
_
l
e
n
 
=
 
o
b
s
e
r
v
e
d
_
m
a
x
 
+
 
s
e
l
f
.
_
p
a
d
d
i
n
g


 
 
 
 
 
 
 
 
#
 
S
e
t
 
<
u
n
k
>
 
t
o
k
e
n
 
t
o
 
t
h
e
 
n
e
x
t
 
a
v
a
i
l
a
b
l
e
 
i
n
d
e
x

 
 
 
 
 
 
 
 
#
 
S
i
n
c
e
 
<
u
n
k
>
 
i
s
 
a
l
r
e
a
d
y
 
i
n
 
t
h
e
 
v
o
c
a
b
 
d
i
c
t
,
 
w
e
 
u
s
e
 
_
n
e
x
t
_
i
n
d
e
x

 
 
 
 
 
 
 
 
s
e
l
f
.
c
o
d
e
_
v
o
c
a
b
[
"
<
u
n
k
>
"
]
 
=
 
s
e
l
f
.
_
n
e
x
t
_
i
n
d
e
x


 
 
 
 
d
e
f
 
p
r
o
c
e
s
s
(

 
 
 
 
 
 
 
 
s
e
l
f
,
 
v
a
l
u
e
:
 
T
u
p
l
e
[
O
p
t
i
o
n
a
l
[
L
i
s
t
]
,
 
L
i
s
t
]

 
 
 
 
)
 
-
>
 
T
u
p
l
e
[
O
p
t
i
o
n
a
l
[
t
o
r
c
h
.
T
e
n
s
o
r
]
,
 
t
o
r
c
h
.
T
e
n
s
o
r
]
:

 
 
 
 
 
 
 
 
"
"
"
P
r
o
c
e
s
s
 
t
u
p
l
e
 
f
o
r
m
a
t
 
d
a
t
a
 
i
n
t
o
 
t
e
n
s
o
r
s
.


 
 
 
 
 
 
 
 
A
r
g
s
:

 
 
 
 
 
 
 
 
 
 
 
 
v
a
l
u
e
:
 
T
u
p
l
e
 
o
f
 
(
t
i
m
e
,
 
v
a
l
u
e
s
)
 
w
h
e
r
e
 
v
a
l
u
e
s
 
a
r
e
 
c
o
d
e
s


 
 
 
 
 
 
 
 
R
e
t
u
r
n
s
:

 
 
 
 
 
 
 
 
 
 
 
 
T
u
p
l
e
 
o
f
 
(
t
i
m
e
_
t
e
n
s
o
r
,
 
v
a
l
u
e
_
t
e
n
s
o
r
)
,
 
t
i
m
e
 
c
a
n
 
b
e
 
N
o
n
e

 
 
 
 
 
 
 
 
"
"
"

 
 
 
 
 
 
 
 
#
 
U
n
p
a
c
k
 
t
u
p
l
e
:
 
(
t
i
m
e
,
 
v
a
l
u
e
s
)

 
 
 
 
 
 
 
 
t
i
m
e
_
d
a
t
a
,
 
v
a
l
u
e
_
d
a
t
a
 
=
 
v
a
l
u
e


 
 
 
 
 
 
 
 
#
 
E
n
c
o
d
e
 
c
o
d
e
s
 
t
o
 
i
n
d
i
c
e
s

 
 
 
 
 
 
 
 
i
f
 
s
e
l
f
.
_
i
s
_
n
e
s
t
e
d
:

 
 
 
 
 
 
 
 
 
 
 
 
#
 
N
e
s
t
e
d
 
c
o
d
e
s
:
 
[
[
"
A
"
,
 
"
B
"
]
,
 
[
"
C
"
]
]

 
 
 
 
 
 
 
 
 
 
 
 
v
a
l
u
e
_
t
e
n
s
o
r
 
=
 
s
e
l
f
.
_
e
n
c
o
d
e
_
n
e
s
t
e
d
_
c
o
d
e
s
(
v
a
l
u
e
_
d
a
t
a
)

 
 
 
 
 
 
 
 
e
l
s
e
:

 
 
 
 
 
 
 
 
 
 
 
 
#
 
F
l
a
t
 
c
o
d
e
s
:
 
[
"
c
o
d
e
1
"
,
 
"
c
o
d
e
2
"
]

 
 
 
 
 
 
 
 
 
 
 
 
v
a
l
u
e
_
t
e
n
s
o
r
 
=
 
s
e
l
f
.
_
e
n
c
o
d
e
_
c
o
d
e
s
(
v
a
l
u
e
_
d
a
t
a
)


 
 
 
 
 
 
 
 
#
 
P
r
o
c
e
s
s
 
t
i
m
e
 
i
f
 
p
r
e
s
e
n
t

 
 
 
 
 
 
 
 
t
i
m
e
_
t
e
n
s
o
r
 
=
 
N
o
n
e

 
 
 
 
 
 
 
 
i
f
 
t
i
m
e
_
d
a
t
a
 
i
s
 
n
o
t
 
N
o
n
e
 
a
n
d
 
l
e
n
(
t
i
m
e
_
d
a
t
a
)
 
>
 
0
:

 
 
 
 
 
 
 
 
 
 
 
 
#
 
H
a
n
d
l
e
 
b
o
t
h
 
[
0
.
0
,
 
1
.
5
]
 
a
n
d
 
[
[
0
.
0
]
,
 
[
1
.
5
]
]
 
f
o
r
m
a
t
s

 
 
 
 
 
 
 
 
 
 
 
 
i
f
 
i
s
i
n
s
t
a
n
c
e
(
t
i
m
e
_
d
a
t
a
[
0
]
,
 
l
i
s
t
)
:

 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
#
 
F
l
a
t
t
e
n
 
[
[
0
.
0
]
,
 
[
1
.
5
]
]
 
-
>
 
[
0
.
0
,
 
1
.
5
]

 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
t
i
m
e
_
d
a
t
a
 
=
 
[
t
[
0
]
 
i
f
 
i
s
i
n
s
t
a
n
c
e
(
t
,
 
l
i
s
t
)
 
e
l
s
e
 
t
 
f
o
r
 
t
 
i
n
 
t
i
m
e
_
d
a
t
a
]

 
 
 
 
 
 
 
 
 
 
 
 
t
i
m
e
_
t
e
n
s
o
r
 
=
 
t
o
r
c
h
.
t
e
n
s
o
r
(
t
i
m
e
_
d
a
t
a
,
 
d
t
y
p
e
=
t
o
r
c
h
.
f
l
o
a
t
)


 
 
 
 
 
 
 
 
r
e
t
u
r
n
 
(
t
i
m
e
_
t
e
n
s
o
r
,
 
v
a
l
u
e
_
t
e
n
s
o
r
)


 
 
 
 
d
e
f
 
_
e
n
c
o
d
e
_
c
o
d
e
s
(
s
e
l
f
,
 
c
o
d
e
s
:
 
L
i
s
t
[
s
t
r
]
)
 
-
>
 
t
o
r
c
h
.
T
e
n
s
o
r
:

 
 
 
 
 
 
 
 
"
"
"
E
n
c
o
d
e
 
f
l
a
t
 
c
o
d
e
 
l
i
s
t
 
t
o
 
i
n
d
i
c
e
s
.
"
"
"

 
 
 
 
 
 
 
 
#
 
H
a
n
d
l
e
 
e
m
p
t
y
 
c
o
d
e
 
l
i
s
t
 
-
 
r
e
t
u
r
n
 
s
i
n
g
l
e
 
p
a
d
d
i
n
g
 
t
o
k
e
n

 
 
 
 
 
 
 
 
i
f
 
l
e
n
(
c
o
d
e
s
)
 
=
=
 
0
:

 
 
 
 
 
 
 
 
 
 
 
 
r
e
t
u
r
n
 
t
o
r
c
h
.
t
e
n
s
o
r
(
[
s
e
l
f
.
c
o
d
e
_
v
o
c
a
b
[
"
<
p
a
d
>
"
]
]
,
 
d
t
y
p
e
=
t
o
r
c
h
.
l
o
n
g
)


 
 
 
 
 
 
 
 
i
n
d
i
c
e
s
 
=
 
[
]

 
 
 
 
 
 
 
 
f
o
r
 
c
o
d
e
 
i
n
 
c
o
d
e
s
:

 
 
 
 
 
 
 
 
 
 
 
 
i
f
 
c
o
d
e
 
i
s
 
N
o
n
e
 
o
r
 
c
o
d
e
 
n
o
t
 
i
n
 
s
e
l
f
.
c
o
d
e
_
v
o
c
a
b
:

 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
i
n
d
i
c
e
s
.
a
p
p
e
n
d
(
s
e
l
f
.
c
o
d
e
_
v
o
c
a
b
[
"
<
u
n
k
>
"
]
)

 
 
 
 
 
 
 
 
 
 
 
 
e
l
s
e
:

 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
i
n
d
i
c
e
s
.
a
p
p
e
n
d
(
s
e
l
f
.
c
o
d
e
_
v
o
c
a
b
[
c
o
d
e
]
)

 
 
 
 
 
 
 
 
r
e
t
u
r
n
 
t
o
r
c
h
.
t
e
n
s
o
r
(
i
n
d
i
c
e
s
,
 
d
t
y
p
e
=
t
o
r
c
h
.
l
o
n
g
)


 
 
 
 
d
e
f
 
_
e
n
c
o
d
e
_
n
e
s
t
e
d
_
c
o
d
e
s
(
s
e
l
f
,
 
n
e
s
t
e
d
_
c
o
d
e
s
:
 
L
i
s
t
[
L
i
s
t
[
s
t
r
]
]
)
 
-
>
 
t
o
r
c
h
.
T
e
n
s
o
r
:

 
 
 
 
 
 
 
 
"
"
"
E
n
c
o
d
e
 
n
e
s
t
e
d
 
c
o
d
e
 
l
i
s
t
s
 
t
o
 
p
a
d
d
e
d
 
2
D
 
t
e
n
s
o
r
.


 
 
 
 
 
 
 
 
P
a
d
s
 
a
l
l
 
i
n
n
e
r
 
s
e
q
u
e
n
c
e
s
 
t
o
 
s
e
l
f
.
_
m
a
x
_
n
e
s
t
e
d
_
l
e
n
 
(
g
l
o
b
a
l
 
m
a
x
)
.

 
 
 
 
 
 
 
 
"
"
"

 
 
 
 
 
 
 
 
#
 
H
a
n
d
l
e
 
e
m
p
t
y
 
n
e
s
t
e
d
 
c
o
d
e
s
 
(
n
o
 
v
i
s
i
t
s
/
e
v
e
n
t
s
)

 
 
 
 
 
 
 
 
#
 
R
e
t
u
r
n
 
s
i
n
g
l
e
 
p
a
d
d
i
n
g
 
t
o
k
e
n
 
w
i
t
h
 
s
h
a
p
e
 
(
1
,
 
m
a
x
_
l
e
n
)

 
 
 
 
 
 
 
 
i
f
 
l
e
n
(
n
e
s
t
e
d
_
c
o
d
e
s
)
 
=
=
 
0
:

 
 
 
 
 
 
 
 
 
 
 
 
p
a
d
_
t
o
k
e
n
 
=
 
s
e
l
f
.
c
o
d
e
_
v
o
c
a
b
[
"
<
p
a
d
>
"
]

 
 
 
 
 
 
 
 
 
 
 
 
r
e
t
u
r
n
 
t
o
r
c
h
.
t
e
n
s
o
r
(
[
[
p
a
d
_
t
o
k
e
n
]
 
*
 
s
e
l
f
.
_
m
a
x
_
n
e
s
t
e
d
_
l
e
n
]
,
 
d
t
y
p
e
=
t
o
r
c
h
.
l
o
n
g
)


 
 
 
 
 
 
 
 
e
n
c
o
d
e
d
_
s
e
q
u
e
n
c
e
s
 
=
 
[
]

 
 
 
 
 
 
 
 
#
 
U
s
e
 
g
l
o
b
a
l
 
m
a
x
 
l
e
n
g
t
h
 
d
e
t
e
r
m
i
n
e
d
 
d
u
r
i
n
g
 
f
i
t

 
 
 
 
 
 
 
 
m
a
x
_
l
e
n
 
=
 
s
e
l
f
.
_
m
a
x
_
n
e
s
t
e
d
_
l
e
n


 
 
 
 
 
 
 
 
f
o
r
 
i
n
n
e
r
_
c
o
d
e
s
 
i
n
 
n
e
s
t
e
d
_
c
o
d
e
s
:

 
 
 
 
 
 
 
 
 
 
 
 
i
n
d
i
c
e
s
 
=
 
[
]

 
 
 
 
 
 
 
 
 
 
 
 
f
o
r
 
c
o
d
e
 
i
n
 
i
n
n
e
r
_
c
o
d
e
s
:

 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
i
f
 
c
o
d
e
 
i
s
 
N
o
n
e
 
o
r
 
c
o
d
e
 
n
o
t
 
i
n
 
s
e
l
f
.
c
o
d
e
_
v
o
c
a
b
:

 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
i
n
d
i
c
e
s
.
a
p
p
e
n
d
(
s
e
l
f
.
c
o
d
e
_
v
o
c
a
b
[
"
<
u
n
k
>
"
]
)

 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
e
l
s
e
:

 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
i
n
d
i
c
e
s
.
a
p
p
e
n
d
(
s
e
l
f
.
c
o
d
e
_
v
o
c
a
b
[
c
o
d
e
]
)

 
 
 
 
 
 
 
 
 
 
 
 
#
 
P
a
d
 
t
o
 
G
L
O
B
A
L
 
m
a
x
_
l
e
n

 
 
 
 
 
 
 
 
 
 
 
 
w
h
i
l
e
 
l
e
n
(
i
n
d
i
c
e
s
)
 
<
 
m
a
x
_
l
e
n
:

 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
i
n
d
i
c
e
s
.
a
p
p
e
n
d
(
s
e
l
f
.
c
o
d
e
_
v
o
c
a
b
[
"
<
p
a
d
>
"
]
)

 
 
 
 
 
 
 
 
 
 
 
 
e
n
c
o
d
e
d
_
s
e
q
u
e
n
c
e
s
.
a
p
p
e
n
d
(
i
n
d
i
c
e
s
)


 
 
 
 
 
 
 
 
r
e
t
u
r
n
 
t
o
r
c
h
.
t
e
n
s
o
r
(
e
n
c
o
d
e
d
_
s
e
q
u
e
n
c
e
s
,
 
d
t
y
p
e
=
t
o
r
c
h
.
l
o
n
g
)


 
 
 
 
d
e
f
 
s
i
z
e
(
s
e
l
f
)
 
-
>
 
i
n
t
:

 
 
 
 
 
 
 
 
"
"
"
R
e
t
u
r
n
 
v
o
c
a
b
u
l
a
r
y
 
s
i
z
e
.
"
"
"

 
 
 
 
 
 
 
 
r
e
t
u
r
n
 
l
e
n
(
s
e
l
f
.
c
o
d
e
_
v
o
c
a
b
)


 
 
 
 
d
e
f
 
_
_
r
e
p
r
_
_
(
s
e
l
f
)
:

 
 
 
 
 
 
 
 
i
f
 
s
e
l
f
.
_
i
s
_
n
e
s
t
e
d
:

 
 
 
 
 
 
 
 
 
 
 
 
r
e
t
u
r
n
 
(

 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
f
"
S
t
a
g
e
N
e
t
P
r
o
c
e
s
s
o
r
(
i
s
_
n
e
s
t
e
d
=
{
s
e
l
f
.
_
i
s
_
n
e
s
t
e
d
}
,
 
"

 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
f
"
v
o
c
a
b
_
s
i
z
e
=
{
l
e
n
(
s
e
l
f
.
c
o
d
e
_
v
o
c
a
b
)
}
,
 
"

 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
f
"
m
a
x
_
n
e
s
t
e
d
_
l
e
n
=
{
s
e
l
f
.
_
m
a
x
_
n
e
s
t
e
d
_
l
e
n
}
,
 
"

 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
f
"
p
a
d
d
i
n
g
=
{
s
e
l
f
.
_
p
a
d
d
i
n
g
}
)
"

 
 
 
 
 
 
 
 
 
 
 
 
)

 
 
 
 
 
 
 
 
e
l
s
e
:

 
 
 
 
 
 
 
 
 
 
 
 
r
e
t
u
r
n
 
(

 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
f
"
S
t
a
g
e
N
e
t
P
r
o
c
e
s
s
o
r
(
i
s
_
n
e
s
t
e
d
=
{
s
e
l
f
.
_
i
s
_
n
e
s
t
e
d
}
,
 
"

 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
f
"
v
o
c
a
b
_
s
i
z
e
=
{
l
e
n
(
s
e
l
f
.
c
o
d
e
_
v
o
c
a
b
)
}
,
 
"

 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
f
"
p
a
d
d
i
n
g
=
{
s
e
l
f
.
_
p
a
d
d
i
n
g
}
)
"

 
 
 
 
 
 
 
 
 
 
 
 
)



@
r
e
g
i
s
t
e
r
_
p
r
o
c
e
s
s
o
r
(
"
s
t
a
g
e
n
e
t
_
t
e
n
s
o
r
_
e
x
"
)

c
l
a
s
s
 
S
t
a
g
e
N
e
t
T
e
n
s
o
r
P
r
o
c
e
s
s
o
r
(
F
e
a
t
u
r
e
P
r
o
c
e
s
s
o
r
)
:

 
 
 
 
"
"
"

 
 
 
 
F
e
a
t
u
r
e
 
p
r
o
c
e
s
s
o
r
 
f
o
r
 
S
t
a
g
e
N
e
t
 
N
U
M
E
R
I
C
 
i
n
p
u
t
s
 
w
i
t
h
 
c
o
u
p
l
e
d
 
v
a
l
u
e
/
t
i
m
e
 
d
a
t
a
.


 
 
 
 
T
h
i
s
 
p
r
o
c
e
s
s
o
r
 
h
a
n
d
l
e
s
 
n
u
m
e
r
i
c
 
f
e
a
t
u
r
e
 
s
e
q
u
e
n
c
e
s
 
(
f
l
a
t
 
o
r
 
n
e
s
t
e
d
)
 
a
n
d
 
a
p
p
l
i
e
s

 
 
 
 
f
o
r
w
a
r
d
-
f
i
l
l
 
i
m
p
u
t
a
t
i
o
n
 
t
o
 
h
a
n
d
l
e
 
m
i
s
s
i
n
g
 
v
a
l
u
e
s
 
(
N
a
N
/
N
o
n
e
)
.

 
 
 
 
F
o
r
 
c
a
t
e
g
o
r
i
c
a
l
 
c
o
d
e
s
,
 
u
s
e
 
S
t
a
g
e
N
e
t
P
r
o
c
e
s
s
o
r
 
i
n
s
t
e
a
d
.


 
 
 
 
F
o
r
m
a
t
:

 
 
 
 
{

 
 
 
 
 
 
 
 
"
v
a
l
u
e
"
:
 
[
[
1
.
0
,
 
2
.
0
,
 
3
.
0
]
,
 
[
4
.
0
,
 
5
.
0
,
 
6
.
0
]
]
,
 
 
#
 
n
e
s
t
e
d
 
n
u
m
e
r
i
c
s

 
 
 
 
 
 
 
 
"
t
i
m
e
"
:
 
[
0
.
0
,
 
1
.
5
]
 
o
r
 
N
o
n
e

 
 
 
 
}


 
 
 
 
T
h
e
 
p
r
o
c
e
s
s
o
r
 
a
u
t
o
m
a
t
i
c
a
l
l
y
 
d
e
t
e
c
t
s
:

 
 
 
 
-
 
L
i
s
t
 
o
f
 
n
u
m
b
e
r
s
 
-
>
 
f
l
a
t
 
n
u
m
e
r
i
c
 
s
e
q
u
e
n
c
e
s

 
 
 
 
-
 
L
i
s
t
 
o
f
 
l
i
s
t
s
 
o
f
 
n
u
m
b
e
r
s
 
-
>
 
n
e
s
t
e
d
 
n
u
m
e
r
i
c
 
s
e
q
u
e
n
c
e
s
 
(
f
e
a
t
u
r
e
 
v
e
c
t
o
r
s
)


 
 
 
 
I
m
p
u
t
a
t
i
o
n
 
S
t
r
a
t
e
g
y
:

 
 
 
 
-
 
F
o
r
w
a
r
d
-
f
i
l
l
:
 
M
i
s
s
i
n
g
 
v
a
l
u
e
s
 
(
N
a
N
/
N
o
n
e
)
 
a
r
e
 
f
i
l
l
e
d
 
w
i
t
h
 
t
h
e
 
l
a
s
t
 
o
b
s
e
r
v
e
d

 
 
 
 
 
 
v
a
l
u
e
 
f
o
r
 
t
h
a
t
 
f
e
a
t
u
r
e
 
d
i
m
e
n
s
i
o
n
.
 
I
f
 
n
o
 
p
r
i
o
r
 
v
a
l
u
e
 
e
x
i
s
t
s
,
 
0
.
0
 
i
s
 
u
s
e
d
.

 
 
 
 
-
 
A
p
p
l
i
e
d
 
p
e
r
 
f
e
a
t
u
r
e
 
d
i
m
e
n
s
i
o
n
 
i
n
d
e
p
e
n
d
e
n
t
l
y


 
 
 
 
R
e
t
u
r
n
s
:

 
 
 
 
 
 
 
 
T
u
p
l
e
 
o
f
 
(
t
i
m
e
_
t
e
n
s
o
r
,
 
v
a
l
u
e
_
t
e
n
s
o
r
)
 
w
h
e
r
e
 
t
i
m
e
_
t
e
n
s
o
r
 
c
a
n
 
b
e
 
N
o
n
e


 
 
 
 
E
x
a
m
p
l
e
s
:

 
 
 
 
 
 
 
 
>
>
>
 
#
 
C
a
s
e
 
1
:
 
F
e
a
t
u
r
e
 
v
e
c
t
o
r
s
 
w
i
t
h
 
m
i
s
s
i
n
g
 
v
a
l
u
e
s

 
 
 
 
 
 
 
 
>
>
>
 
p
r
o
c
e
s
s
o
r
 
=
 
S
t
a
g
e
N
e
t
T
e
n
s
o
r
P
r
o
c
e
s
s
o
r
(
)

 
 
 
 
 
 
 
 
>
>
>
 
d
a
t
a
 
=
 
{

 
 
 
 
 
 
 
 
.
.
.
 
 
 
 
 
"
v
a
l
u
e
"
:
 
[
[
1
.
0
,
 
N
o
n
e
,
 
3
.
0
]
,
 
[
N
o
n
e
,
 
5
.
0
,
 
6
.
0
]
,
 
[
7
.
0
,
 
8
.
0
,
 
N
o
n
e
]
]
,

 
 
 
 
 
 
 
 
.
.
.
 
 
 
 
 
"
t
i
m
e
"
:
 
[
0
.
0
,
 
1
.
5
,
 
3
.
0
]

 
 
 
 
 
 
 
 
.
.
.
 
}

 
 
 
 
 
 
 
 
>
>
>
 
t
i
m
e
,
 
v
a
l
u
e
s
 
=
 
p
r
o
c
e
s
s
o
r
.
p
r
o
c
e
s
s
(
d
a
t
a
)

 
 
 
 
 
 
 
 
>
>
>
 
v
a
l
u
e
s
 
 
#
 
[
[
1
.
0
,
 
0
.
0
,
 
3
.
0
]
,
 
[
1
.
0
,
 
5
.
0
,
 
6
.
0
]
,
 
[
7
.
0
,
 
8
.
0
,
 
6
.
0
]
]

 
 
 
 
 
 
 
 
>
>
>
 
v
a
l
u
e
s
.
d
t
y
p
e
 
 
#
 
t
o
r
c
h
.
f
l
o
a
t
3
2

 
 
 
 
 
 
 
 
>
>
>
 
t
i
m
e
.
s
h
a
p
e
 
 
 
 
#
 
(
3
,
)

 
 
 
 
"
"
"


 
 
 
 
d
e
f
 
_
_
i
n
i
t
_
_
(
s
e
l
f
)
:

 
 
 
 
 
 
 
 
s
e
l
f
.
_
s
i
z
e
 
=
 
N
o
n
e
 
 
#
 
F
e
a
t
u
r
e
 
d
i
m
e
n
s
i
o
n
 
(
s
e
t
 
d
u
r
i
n
g
 
f
i
t
)

 
 
 
 
 
 
 
 
s
e
l
f
.
_
i
s
_
n
e
s
t
e
d
 
=
 
N
o
n
e


 
 
 
 
d
e
f
 
f
i
t
(
s
e
l
f
,
 
s
a
m
p
l
e
s
:
 
L
i
s
t
[
D
i
c
t
]
,
 
k
e
y
:
 
s
t
r
)
 
-
>
 
N
o
n
e
:

 
 
 
 
 
 
 
 
"
"
"
D
e
t
e
r
m
i
n
e
 
i
n
p
u
t
 
s
t
r
u
c
t
u
r
e
.


 
 
 
 
 
 
 
 
A
r
g
s
:

 
 
 
 
 
 
 
 
 
 
 
 
s
a
m
p
l
e
s
:
 
L
i
s
t
 
o
f
 
s
a
m
p
l
e
 
d
i
c
t
i
o
n
a
r
i
e
s

 
 
 
 
 
 
 
 
 
 
 
 
k
e
y
:
 
T
h
e
 
k
e
y
 
i
n
 
s
a
m
p
l
e
s
 
t
h
a
t
 
c
o
n
t
a
i
n
s
 
t
u
p
l
e
 
(
t
i
m
e
,
 
v
a
l
u
e
s
)

 
 
 
 
 
 
 
 
"
"
"

 
 
 
 
 
 
 
 
#
 
E
x
a
m
i
n
e
 
f
i
r
s
t
 
n
o
n
-
N
o
n
e
 
s
a
m
p
l
e
 
t
o
 
d
e
t
e
r
m
i
n
e
 
s
t
r
u
c
t
u
r
e

 
 
 
 
 
 
 
 
f
o
r
 
s
a
m
p
l
e
 
i
n
 
s
a
m
p
l
e
s
:

 
 
 
 
 
 
 
 
 
 
 
 
i
f
 
k
e
y
 
i
n
 
s
a
m
p
l
e
 
a
n
d
 
s
a
m
p
l
e
[
k
e
y
]
 
i
s
 
n
o
t
 
N
o
n
e
:

 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
#
 
U
n
p
a
c
k
 
t
u
p
l
e
:
 
(
t
i
m
e
,
 
v
a
l
u
e
s
)

 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
t
i
m
e
_
d
a
t
a
,
 
v
a
l
u
e
_
d
a
t
a
 
=
 
s
a
m
p
l
e
[
k
e
y
]


 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
#
 
D
e
t
e
r
m
i
n
e
 
n
e
s
t
i
n
g
 
l
e
v
e
l
 
f
o
r
 
n
u
m
e
r
i
c
s

 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
i
f
 
i
s
i
n
s
t
a
n
c
e
(
v
a
l
u
e
_
d
a
t
a
,
 
l
i
s
t
)
 
a
n
d
 
l
e
n
(
v
a
l
u
e
_
d
a
t
a
)
 
>
 
0
:

 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
f
i
r
s
t
_
e
l
e
m
 
=
 
v
a
l
u
e
_
d
a
t
a
[
0
]


 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
i
f
 
i
s
i
n
s
t
a
n
c
e
(
f
i
r
s
t
_
e
l
e
m
,
 
(
i
n
t
,
 
f
l
o
a
t
)
)
:

 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
#
 
F
l
a
t
 
n
u
m
e
r
i
c
:
 
[
1
.
5
,
 
2
.
0
,
 
.
.
.
]

 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
s
e
l
f
.
_
i
s
_
n
e
s
t
e
d
 
=
 
F
a
l
s
e

 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
s
e
l
f
.
_
s
i
z
e
 
=
 
1

 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
e
l
i
f
 
i
s
i
n
s
t
a
n
c
e
(
f
i
r
s
t
_
e
l
e
m
,
 
l
i
s
t
)
:

 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
i
f
 
l
e
n
(
f
i
r
s
t
_
e
l
e
m
)
 
>
 
0
:

 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
i
f
 
i
s
i
n
s
t
a
n
c
e
(
f
i
r
s
t
_
e
l
e
m
[
0
]
,
 
(
i
n
t
,
 
f
l
o
a
t
)
)
:

 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
#
 
N
e
s
t
e
d
 
n
u
m
e
r
i
c
s
:
 
[
[
1
.
0
,
 
2
.
0
]
,
 
[
3
.
0
,
 
4
.
0
]
]

 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
s
e
l
f
.
_
i
s
_
n
e
s
t
e
d
 
=
 
T
r
u
e

 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
s
e
l
f
.
_
s
i
z
e
 
=
 
l
e
n
(
f
i
r
s
t
_
e
l
e
m
)

 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
b
r
e
a
k


 
 
 
 
d
e
f
 
p
r
o
c
e
s
s
(

 
 
 
 
 
 
 
 
s
e
l
f
,
 
v
a
l
u
e
:
 
T
u
p
l
e
[
O
p
t
i
o
n
a
l
[
L
i
s
t
]
,
 
L
i
s
t
]

 
 
 
 
)
 
-
>
 
T
u
p
l
e
[
O
p
t
i
o
n
a
l
[
t
o
r
c
h
.
T
e
n
s
o
r
]
,
 
t
o
r
c
h
.
T
e
n
s
o
r
]
:

 
 
 
 
 
 
 
 
"
"
"
P
r
o
c
e
s
s
 
t
u
p
l
e
 
f
o
r
m
a
t
 
n
u
m
e
r
i
c
 
d
a
t
a
 
i
n
t
o
 
t
e
n
s
o
r
s
.


 
 
 
 
 
 
 
 
A
p
p
l
i
e
s
 
f
o
r
w
a
r
d
-
f
i
l
l
 
i
m
p
u
t
a
t
i
o
n
 
t
o
 
h
a
n
d
l
e
 
N
a
N
/
N
o
n
e
 
v
a
l
u
e
s
.

 
 
 
 
 
 
 
 
F
o
r
 
e
a
c
h
 
f
e
a
t
u
r
e
 
d
i
m
e
n
s
i
o
n
,
 
m
i
s
s
i
n
g
 
v
a
l
u
e
s
 
a
r
e
 
f
i
l
l
e
d
 
w
i
t
h
 
t
h
e

 
 
 
 
 
 
 
 
l
a
s
t
 
o
b
s
e
r
v
e
d
 
v
a
l
u
e
 
(
o
r
 
0
.
0
 
i
f
 
n
o
 
p
r
i
o
r
 
v
a
l
u
e
 
e
x
i
s
t
s
)
.


 
 
 
 
 
 
 
 
A
r
g
s
:

 
 
 
 
 
 
 
 
 
 
 
 
v
a
l
u
e
:
 
T
u
p
l
e
 
o
f
 
(
t
i
m
e
,
 
v
a
l
u
e
s
)
 
w
h
e
r
e
 
v
a
l
u
e
s
 
a
r
e
 
n
u
m
e
r
i
c
s


 
 
 
 
 
 
 
 
R
e
t
u
r
n
s
:

 
 
 
 
 
 
 
 
 
 
 
 
T
u
p
l
e
 
o
f
 
(
t
i
m
e
_
t
e
n
s
o
r
,
 
v
a
l
u
e
_
t
e
n
s
o
r
)
,
 
t
i
m
e
 
c
a
n
 
b
e
 
N
o
n
e

 
 
 
 
 
 
 
 
"
"
"

 
 
 
 
 
 
 
 
#
 
U
n
p
a
c
k
 
t
u
p
l
e
:
 
(
t
i
m
e
,
 
v
a
l
u
e
s
)

 
 
 
 
 
 
 
 
t
i
m
e
_
d
a
t
a
,
 
v
a
l
u
e
_
d
a
t
a
 
=
 
v
a
l
u
e


 
 
 
 
 
 
 
 
#
 
C
o
n
v
e
r
t
 
t
o
 
n
u
m
p
y
 
f
o
r
 
e
a
s
i
e
r
 
i
m
p
u
t
a
t
i
o
n
 
h
a
n
d
l
i
n
g

 
 
 
 
 
 
 
 
i
m
p
o
r
t
 
n
u
m
p
y
 
a
s
 
n
p


 
 
 
 
 
 
 
 
v
a
l
u
e
_
a
r
r
a
y
 
=
 
n
p
.
a
r
r
a
y
(
v
a
l
u
e
_
d
a
t
a
,
 
d
t
y
p
e
=
f
l
o
a
t
)


 
 
 
 
 
 
 
 
#
 
A
p
p
l
y
 
f
o
r
w
a
r
d
-
f
i
l
l
 
i
m
p
u
t
a
t
i
o
n

 
 
 
 
 
 
 
 
i
f
 
v
a
l
u
e
_
a
r
r
a
y
.
n
d
i
m
 
=
=
 
1
:

 
 
 
 
 
 
 
 
 
 
 
 
#
 
F
l
a
t
 
n
u
m
e
r
i
c
:
 
[
1
.
5
,
 
2
.
0
,
 
n
a
n
,
 
3
.
0
,
 
.
.
.
]

 
 
 
 
 
 
 
 
 
 
 
 
l
a
s
t
_
v
a
l
u
e
 
=
 
0
.
0

 
 
 
 
 
 
 
 
 
 
 
 
f
o
r
 
i
 
i
n
 
r
a
n
g
e
(
l
e
n
(
v
a
l
u
e
_
a
r
r
a
y
)
)
:

 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
i
f
 
n
o
t
 
n
p
.
i
s
n
a
n
(
v
a
l
u
e
_
a
r
r
a
y
[
i
]
)
:

 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
l
a
s
t
_
v
a
l
u
e
 
=
 
v
a
l
u
e
_
a
r
r
a
y
[
i
]

 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
e
l
s
e
:

 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
v
a
l
u
e
_
a
r
r
a
y
[
i
]
 
=
 
l
a
s
t
_
v
a
l
u
e

 
 
 
 
 
 
 
 
e
l
i
f
 
v
a
l
u
e
_
a
r
r
a
y
.
n
d
i
m
 
=
=
 
2
:

 
 
 
 
 
 
 
 
 
 
 
 
#
 
F
e
a
t
u
r
e
 
v
e
c
t
o
r
s
:
 
[
[
1
.
0
,
 
n
a
n
,
 
3
.
0
]
,
 
[
n
a
n
,
 
5
.
0
,
 
6
.
0
]
]

 
 
 
 
 
 
 
 
 
 
 
 
n
u
m
_
f
e
a
t
u
r
e
s
 
=
 
v
a
l
u
e
_
a
r
r
a
y
.
s
h
a
p
e
[
1
]

 
 
 
 
 
 
 
 
 
 
 
 
f
o
r
 
f
 
i
n
 
r
a
n
g
e
(
n
u
m
_
f
e
a
t
u
r
e
s
)
:

 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
l
a
s
t
_
v
a
l
u
e
 
=
 
0
.
0

 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
f
o
r
 
t
 
i
n
 
r
a
n
g
e
(
v
a
l
u
e
_
a
r
r
a
y
.
s
h
a
p
e
[
0
]
)
:

 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
i
f
 
n
o
t
 
n
p
.
i
s
n
a
n
(
v
a
l
u
e
_
a
r
r
a
y
[
t
,
 
f
]
)
:

 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
l
a
s
t
_
v
a
l
u
e
 
=
 
v
a
l
u
e
_
a
r
r
a
y
[
t
,
 
f
]

 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
e
l
s
e
:

 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
v
a
l
u
e
_
a
r
r
a
y
[
t
,
 
f
]
 
=
 
l
a
s
t
_
v
a
l
u
e


 
 
 
 
 
 
 
 
#
 
C
o
n
v
e
r
t
 
t
o
 
f
l
o
a
t
 
t
e
n
s
o
r

 
 
 
 
 
 
 
 
v
a
l
u
e
_
t
e
n
s
o
r
 
=
 
t
o
r
c
h
.
t
e
n
s
o
r
(
v
a
l
u
e
_
a
r
r
a
y
,
 
d
t
y
p
e
=
t
o
r
c
h
.
f
l
o
a
t
)


 
 
 
 
 
 
 
 
#
 
P
r
o
c
e
s
s
 
t
i
m
e
 
i
f
 
p
r
e
s
e
n
t

 
 
 
 
 
 
 
 
t
i
m
e
_
t
e
n
s
o
r
 
=
 
N
o
n
e

 
 
 
 
 
 
 
 
i
f
 
t
i
m
e
_
d
a
t
a
 
i
s
 
n
o
t
 
N
o
n
e
 
a
n
d
 
l
e
n
(
t
i
m
e
_
d
a
t
a
)
 
>
 
0
:

 
 
 
 
 
 
 
 
 
 
 
 
#
 
H
a
n
d
l
e
 
b
o
t
h
 
[
0
.
0
,
 
1
.
5
]
 
a
n
d
 
[
[
0
.
0
]
,
 
[
1
.
5
]
]
 
f
o
r
m
a
t
s

 
 
 
 
 
 
 
 
 
 
 
 
i
f
 
i
s
i
n
s
t
a
n
c
e
(
t
i
m
e
_
d
a
t
a
[
0
]
,
 
l
i
s
t
)
:

 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
#
 
F
l
a
t
t
e
n
 
[
[
0
.
0
]
,
 
[
1
.
5
]
]
 
-
>
 
[
0
.
0
,
 
1
.
5
]

 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
t
i
m
e
_
d
a
t
a
 
=
 
[
t
[
0
]
 
i
f
 
i
s
i
n
s
t
a
n
c
e
(
t
,
 
l
i
s
t
)
 
e
l
s
e
 
t
 
f
o
r
 
t
 
i
n
 
t
i
m
e
_
d
a
t
a
]

 
 
 
 
 
 
 
 
 
 
 
 
t
i
m
e
_
t
e
n
s
o
r
 
=
 
t
o
r
c
h
.
t
e
n
s
o
r
(
t
i
m
e
_
d
a
t
a
,
 
d
t
y
p
e
=
t
o
r
c
h
.
f
l
o
a
t
)


 
 
 
 
 
 
 
 
r
e
t
u
r
n
 
(
t
i
m
e
_
t
e
n
s
o
r
,
 
v
a
l
u
e
_
t
e
n
s
o
r
)


 
 
 
 
@
p
r
o
p
e
r
t
y

 
 
 
 
d
e
f
 
s
i
z
e
(
s
e
l
f
)
:

 
 
 
 
 
 
 
 
"
"
"
R
e
t
u
r
n
 
f
e
a
t
u
r
e
 
d
i
m
e
n
s
i
o
n
.
"
"
"

 
 
 
 
 
 
 
 
r
e
t
u
r
n
 
s
e
l
f
.
_
s
i
z
e


 
 
 
 
d
e
f
 
_
_
r
e
p
r
_
_
(
s
e
l
f
)
:

 
 
 
 
 
 
 
 
r
e
t
u
r
n
 
(

 
 
 
 
 
 
 
 
 
 
 
 
f
"
S
t
a
g
e
N
e
t
T
e
n
s
o
r
P
r
o
c
e
s
s
o
r
(
i
s
_
n
e
s
t
e
d
=
{
s
e
l
f
.
_
i
s
_
n
e
s
t
e
d
}
,
 
"

 
 
 
 
 
 
 
 
 
 
 
 
f
"
f
e
a
t
u
r
e
_
d
i
m
=
{
s
e
l
f
.
_
s
i
z
e
}
)
"

 
 
 
 
 
 
 
 
)



#
#
 
D
e
f
i
n
i
n
g
 
a
 
O
u
r
 
S
t
a
g
e
N
e
t
-
s
p
e
c
i
f
i
c
 
T
a
s
k


W
e
'
l
l
 
p
r
e
d
i
c
t
 
p
a
t
i
e
n
t
 
m
o
r
t
a
l
i
t
y
 
u
s
i
n
g
 
S
t
a
g
e
N
e
t
 
a
c
r
o
s
s
 
t
i
m
e
-
s
e
r
i
e
s
 
d
a
t
a
 
f
r
o
m
 
m
u
l
t
i
p
l
e
 
v
i
s
i
t
s
.
 
E
a
c
h
 
v
i
s
i
t
 
i
n
c
l
u
d
e
s
:


-
 
D
i
a
g
n
o
s
i
s
 
c
o
d
e
s

-
 
P
r
o
c
e
d
u
r
e
 
c
o
d
e
s

-
 
L
a
b
 
e
v
e
n
t
s


H
e
r
e
,
 
e
a
c
h
 
f
e
a
t
u
r
e
 
w
i
l
l
 
a
l
s
o
 
n
e
e
d
 
h
a
v
e
 
i
t
s
 
o
w
n
 
c
o
r
r
e
s
p
o
n
d
i
n
g
 
t
i
m
e
 
i
n
t
e
r
v
a
l
s
.
 
A
s
 
d
e
f
i
n
e
d
 
b
y
 
t
h
e
 
S
t
a
g
e
N
e
t
 
p
a
p
e
r
,
 
e
a
c
h
 
t
i
m
e
 
i
n
t
e
r
v
a
l
 
i
s
 
d
e
f
i
n
e
d
 
a
s
 
t
h
e
 
d
i
f
f
e
r
e
n
c
e
 
i
n
 
t
i
m
e
 
b
e
t
w
e
e
n
 
t
h
e
 
c
u
r
r
e
n
t
 
v
i
s
i
t
 
a
n
d
 
t
h
e
 
p
r
e
v
i
o
u
s
 
v
i
s
i
t
.
 


T
o
 
d
e
f
i
n
e
 
a
 
t
a
s
k
,
 
s
p
e
c
i
f
y
 
t
h
e
 
`
_
_
c
a
l
l
_
_
`
 
m
e
t
h
o
d
,
 
i
n
p
u
t
 
s
c
h
e
m
a
,
 
a
n
d
 
o
u
t
p
u
t
 
s
c
h
e
m
a
.
 
F
o
r
 
a
 
d
e
t
a
i
l
e
d
 
e
x
p
l
a
n
a
t
i
o
n
,
 
s
e
e
 
[
t
h
i
s
 
t
u
t
o
r
i
a
l
]
(
h
t
t
p
s
:
/
/
c
o
l
a
b
.
r
e
s
e
a
r
c
h
.
g
o
o
g
l
e
.
c
o
m
/
d
r
i
v
e
/
1
k
K
K
B
V
S
_
G
c
l
H
o
Y
T
b
n
O
t
j
y
Y
n
S
e
e
7
9
h
s
y
T
?
u
s
p
=
s
h
a
r
i
n
g
)
.


#
#
#
 
H
e
l
p
e
r
 
F
u
n
c
t
i
o
n
s


U
s
e
 
`
p
a
t
i
e
n
t
.
g
e
t
_
e
v
e
n
t
s
(
)
`
 
t
o
 
r
e
t
r
i
e
v
e
 
a
l
l
 
e
v
e
n
t
s
 
f
r
o
m
 
a
 
s
p
e
c
i
f
i
c
 
t
a
b
l
e
,
 
w
i
t
h
 
o
p
t
i
o
n
a
l
 
f
i
l
t
e
r
i
n
g
.
 
S
e
e
 
t
h
e
 
[
M
I
M
I
C
-
I
V
 
Y
A
M
L
 
f
i
l
e
]
(
h
t
t
p
s
:
/
/
g
i
t
h
u
b
.
c
o
m
/
R
E
D
A
C
T
E
D
_
O
R
G
/
P
y
H
e
a
l
t
h
/
b
l
o
b
/
m
a
s
t
e
r
/
p
y
h
e
a
l
t
h
/
d
a
t
a
s
e
t
s
/
c
o
n
f
i
g
s
/
m
i
m
i
c
4
_
e
h
r
.
y
a
m
l
)
 
f
o
r
 
a
v
a
i
l
a
b
l
e
 
t
a
b
l
e
s
.

In [3]:
f
r
o
m
 
d
a
t
e
t
i
m
e
 
i
m
p
o
r
t
 
d
a
t
e
t
i
m
e

f
r
o
m
 
t
y
p
i
n
g
 
i
m
p
o
r
t
 
A
n
y
,
 
C
l
a
s
s
V
a
r
,
 
D
i
c
t
,
 
L
i
s
t
,
 
T
u
p
l
e


i
m
p
o
r
t
 
p
o
l
a
r
s
 
a
s
 
p
l


f
r
o
m
 
p
y
h
e
a
l
t
h
.
t
a
s
k
s
.
b
a
s
e
_
t
a
s
k
 
i
m
p
o
r
t
 
B
a
s
e
T
a
s
k



c
l
a
s
s
 
M
o
r
t
a
l
i
t
y
P
r
e
d
i
c
t
i
o
n
S
t
a
g
e
N
e
t
M
I
M
I
C
4
(
B
a
s
e
T
a
s
k
)
:

 
 
 
 
"
"
"
T
a
s
k
 
f
o
r
 
p
r
e
d
i
c
t
i
n
g
 
m
o
r
t
a
l
i
t
y
 
u
s
i
n
g
 
M
I
M
I
C
-
I
V
 
w
i
t
h
 
S
t
a
g
e
N
e
t
 
f
o
r
m
a
t
.


 
 
 
 
T
h
i
s
 
t
a
s
k
 
c
r
e
a
t
e
s
 
P
A
T
I
E
N
T
-
L
E
V
E
L
 
s
a
m
p
l
e
s
 
(
n
o
t
 
v
i
s
i
t
-
l
e
v
e
l
)
 
b
y
 
a
g
g
r
e
g
a
t
i
n
g

 
 
 
 
a
l
l
 
a
d
m
i
s
s
i
o
n
s
 
f
o
r
 
e
a
c
h
 
p
a
t
i
e
n
t
.
 
I
C
D
 
c
o
d
e
s
 
(
d
i
a
g
n
o
s
e
s
 
+
 
p
r
o
c
e
d
u
r
e
s
)
 
a
n
d

 
 
 
 
l
a
b
 
r
e
s
u
l
t
s
 
a
c
r
o
s
s
 
a
l
l
 
v
i
s
i
t
s
 
a
r
e
 
c
o
m
b
i
n
e
d
 
w
i
t
h
 
t
i
m
e
 
i
n
t
e
r
v
a
l
s
 
c
a
l
c
u
l
a
t
e
d

 
 
 
 
f
r
o
m
 
t
h
e
 
p
a
t
i
e
n
t
'
s
 
f
i
r
s
t
 
a
d
m
i
s
s
i
o
n
 
t
i
m
e
s
t
a
m
p
.


 
 
 
 
T
i
m
e
 
C
a
l
c
u
l
a
t
i
o
n
:

 
 
 
 
 
 
 
 
-
 
I
C
D
 
c
o
d
e
s
:
 
H
o
u
r
s
 
f
r
o
m
 
p
r
e
v
i
o
u
s
 
a
d
m
i
s
s
i
o
n
 
(
0
 
f
o
r
 
f
i
r
s
t
 
v
i
s
i
t
,

 
 
 
 
 
 
 
 
 
 
t
h
e
n
 
t
i
m
e
 
i
n
t
e
r
v
a
l
s
 
b
e
t
w
e
e
n
 
c
o
n
s
e
c
u
t
i
v
e
 
v
i
s
i
t
s
)

 
 
 
 
 
 
 
 
-
 
L
a
b
s
:
 
H
o
u
r
s
 
f
r
o
m
 
a
d
m
i
s
s
i
o
n
 
s
t
a
r
t
 
(
w
i
t
h
i
n
-
v
i
s
i
t
 
m
e
a
s
u
r
e
m
e
n
t
s
)


 
 
 
 
L
a
b
 
P
r
o
c
e
s
s
i
n
g
:

 
 
 
 
 
 
 
 
-
 
1
0
-
d
i
m
e
n
s
i
o
n
a
l
 
v
e
c
t
o
r
s
 
(
o
n
e
 
p
e
r
 
l
a
b
 
c
a
t
e
g
o
r
y
)

 
 
 
 
 
 
 
 
-
 
M
u
l
t
i
p
l
e
 
i
t
e
m
i
d
s
 
p
e
r
 
c
a
t
e
g
o
r
y
 
→
 
t
a
k
e
 
f
i
r
s
t
 
o
b
s
e
r
v
e
d
 
v
a
l
u
e

 
 
 
 
 
 
 
 
-
 
M
i
s
s
i
n
g
 
c
a
t
e
g
o
r
i
e
s
 
→
 
N
o
n
e
/
N
a
N
 
i
n
 
v
e
c
t
o
r


 
 
 
 
A
r
g
s
:

 
 
 
 
 
 
 
 
p
a
d
d
i
n
g
:
 
A
d
d
i
t
i
o
n
a
l
 
p
a
d
d
i
n
g
 
f
o
r
 
S
t
a
g
e
N
e
t
 
p
r
o
c
e
s
s
o
r
 
t
o
 
h
a
n
d
l
e

 
 
 
 
 
 
 
 
 
 
 
 
s
e
q
u
e
n
c
e
s
 
l
o
n
g
e
r
 
t
h
a
n
 
o
b
s
e
r
v
e
d
 
d
u
r
i
n
g
 
t
r
a
i
n
i
n
g
.
 
D
e
f
a
u
l
t
:
 
0
.


 
 
 
 
A
t
t
r
i
b
u
t
e
s
:

 
 
 
 
 
 
 
 
t
a
s
k
_
n
a
m
e
 
(
s
t
r
)
:
 
T
h
e
 
n
a
m
e
 
o
f
 
t
h
e
 
t
a
s
k
.

 
 
 
 
 
 
 
 
i
n
p
u
t
_
s
c
h
e
m
a
 
(
D
i
c
t
[
s
t
r
,
 
s
t
r
]
)
:
 
T
h
e
 
s
c
h
e
m
a
 
f
o
r
 
i
n
p
u
t
 
d
a
t
a
:

 
 
 
 
 
 
 
 
 
 
 
 
-
 
i
c
d
_
c
o
d
e
s
:
 
C
o
m
b
i
n
e
d
 
d
i
a
g
n
o
s
i
s
 
+
 
p
r
o
c
e
d
u
r
e
 
I
C
D
 
c
o
d
e
s

 
 
 
 
 
 
 
 
 
 
 
 
 
 
(
s
t
a
g
e
n
e
t
 
f
o
r
m
a
t
,
 
n
e
s
t
e
d
 
b
y
 
v
i
s
i
t
)

 
 
 
 
 
 
 
 
 
 
 
 
-
 
l
a
b
s
:
 
L
a
b
 
r
e
s
u
l
t
s
 
(
s
t
a
g
e
n
e
t
_
t
e
n
s
o
r
,
 
1
0
D
 
v
e
c
t
o
r
s
 
p
e
r
 
t
i
m
e
s
t
a
m
p
)

 
 
 
 
 
 
 
 
o
u
t
p
u
t
_
s
c
h
e
m
a
 
(
D
i
c
t
[
s
t
r
,
 
s
t
r
]
)
:
 
T
h
e
 
s
c
h
e
m
a
 
f
o
r
 
o
u
t
p
u
t
 
d
a
t
a
:

 
 
 
 
 
 
 
 
 
 
 
 
-
 
m
o
r
t
a
l
i
t
y
:
 
B
i
n
a
r
y
 
i
n
d
i
c
a
t
o
r
 
(
1
 
i
f
 
a
n
y
 
a
d
m
i
s
s
i
o
n
 
h
a
d
 
m
o
r
t
a
l
i
t
y
)

 
 
 
 
"
"
"


 
 
 
 
t
a
s
k
_
n
a
m
e
:
 
s
t
r
 
=
 
"
M
o
r
t
a
l
i
t
y
P
r
e
d
i
c
t
i
o
n
S
t
a
g
e
N
e
t
M
I
M
I
C
4
"


 
 
 
 
d
e
f
 
_
_
i
n
i
t
_
_
(
s
e
l
f
,
 
p
a
d
d
i
n
g
:
 
i
n
t
 
=
 
0
)
:

 
 
 
 
 
 
 
 
"
"
"
I
n
i
t
i
a
l
i
z
e
 
t
a
s
k
 
w
i
t
h
 
o
p
t
i
o
n
a
l
 
p
a
d
d
i
n
g
 
p
a
r
a
m
e
t
e
r
.


 
 
 
 
 
 
 
 
A
r
g
s
:

 
 
 
 
 
 
 
 
 
 
 
 
p
a
d
d
i
n
g
:
 
A
d
d
i
t
i
o
n
a
l
 
p
a
d
d
i
n
g
 
f
o
r
 
n
e
s
t
e
d
 
s
e
q
u
e
n
c
e
s
.
 
D
e
f
a
u
l
t
:
 
0
.

 
 
 
 
 
 
 
 
"
"
"

 
 
 
 
 
 
 
 
s
e
l
f
.
p
a
d
d
i
n
g
 
=
 
p
a
d
d
i
n
g

 
 
 
 
 
 
 
 
#
 
U
s
e
 
t
u
p
l
e
 
f
o
r
m
a
t
 
t
o
 
p
a
s
s
 
k
w
a
r
g
s
 
t
o
 
p
r
o
c
e
s
s
o
r

 
 
 
 
 
 
 
 
s
e
l
f
.
i
n
p
u
t
_
s
c
h
e
m
a
:
 
D
i
c
t
[
s
t
r
,
 
T
u
p
l
e
[
s
t
r
,
 
D
i
c
t
[
s
t
r
,
 
A
n
y
]
]
]
 
=
 
{

 
 
 
 
 
 
 
 
 
 
 
 
"
i
c
d
_
c
o
d
e
s
"
:
 
(
"
s
t
a
g
e
n
e
t
"
,
 
{
"
p
a
d
d
i
n
g
"
:
 
p
a
d
d
i
n
g
}
)
,

 
 
 
 
 
 
 
 
 
 
 
 
"
l
a
b
s
"
:
 
(
"
s
t
a
g
e
n
e
t
_
t
e
n
s
o
r
"
,
 
{
}
)
,

 
 
 
 
 
 
 
 
}

 
 
 
 
 
 
 
 
s
e
l
f
.
o
u
t
p
u
t
_
s
c
h
e
m
a
:
 
D
i
c
t
[
s
t
r
,
 
s
t
r
]
 
=
 
{
"
m
o
r
t
a
l
i
t
y
"
:
 
"
b
i
n
a
r
y
"
}


 
 
 
 
#
 
O
r
g
a
n
i
z
e
 
l
a
b
 
i
t
e
m
s
 
b
y
 
c
a
t
e
g
o
r
y

 
 
 
 
#
 
E
a
c
h
 
c
a
t
e
g
o
r
y
 
w
i
l
l
 
m
a
p
 
t
o
 
O
N
E
 
d
i
m
e
n
s
i
o
n
 
i
n
 
t
h
e
 
o
u
t
p
u
t
 
v
e
c
t
o
r

 
 
 
 
L
A
B
_
C
A
T
E
G
O
R
I
E
S
:
 
C
l
a
s
s
V
a
r
[
D
i
c
t
[
s
t
r
,
 
L
i
s
t
[
s
t
r
]
]
]
 
=
 
{

 
 
 
 
 
 
 
 
"
S
o
d
i
u
m
"
:
 
[
"
5
0
8
2
4
"
,
 
"
5
2
4
5
5
"
,
 
"
5
0
9
8
3
"
,
 
"
5
2
6
2
3
"
]
,

 
 
 
 
 
 
 
 
"
P
o
t
a
s
s
i
u
m
"
:
 
[
"
5
0
8
2
2
"
,
 
"
5
2
4
5
2
"
,
 
"
5
0
9
7
1
"
,
 
"
5
2
6
1
0
"
]
,

 
 
 
 
 
 
 
 
"
C
h
l
o
r
i
d
e
"
:
 
[
"
5
0
8
0
6
"
,
 
"
5
2
4
3
4
"
,
 
"
5
0
9
0
2
"
,
 
"
5
2
5
3
5
"
]
,

 
 
 
 
 
 
 
 
"
B
i
c
a
r
b
o
n
a
t
e
"
:
 
[
"
5
0
8
0
3
"
,
 
"
5
0
8
0
4
"
]
,

 
 
 
 
 
 
 
 
"
G
l
u
c
o
s
e
"
:
 
[
"
5
0
8
0
9
"
,
 
"
5
2
0
2
7
"
,
 
"
5
0
9
3
1
"
,
 
"
5
2
5
6
9
"
]
,

 
 
 
 
 
 
 
 
"
C
a
l
c
i
u
m
"
:
 
[
"
5
0
8
0
8
"
,
 
"
5
1
6
2
4
"
]
,

 
 
 
 
 
 
 
 
"
M
a
g
n
e
s
i
u
m
"
:
 
[
"
5
0
9
6
0
"
]
,

 
 
 
 
 
 
 
 
"
A
n
i
o
n
 
G
a
p
"
:
 
[
"
5
0
8
6
8
"
,
 
"
5
2
5
0
0
"
]
,

 
 
 
 
 
 
 
 
"
O
s
m
o
l
a
l
i
t
y
"
:
 
[
"
5
2
0
3
1
"
,
 
"
5
0
9
6
4
"
,
 
"
5
1
7
0
1
"
]
,

 
 
 
 
 
 
 
 
"
P
h
o
s
p
h
a
t
e
"
:
 
[
"
5
0
9
7
0
"
]
,

 
 
 
 
}


 
 
 
 
#
 
O
r
d
e
r
e
d
 
l
i
s
t
 
o
f
 
c
a
t
e
g
o
r
y
 
n
a
m
e
s
 
(
d
e
f
i
n
e
s
 
v
e
c
t
o
r
 
d
i
m
e
n
s
i
o
n
 
o
r
d
e
r
)

 
 
 
 
L
A
B
_
C
A
T
E
G
O
R
Y
_
N
A
M
E
S
:
 
C
l
a
s
s
V
a
r
[
L
i
s
t
[
s
t
r
]
]
 
=
 
[

 
 
 
 
 
 
 
 
"
S
o
d
i
u
m
"
,

 
 
 
 
 
 
 
 
"
P
o
t
a
s
s
i
u
m
"
,

 
 
 
 
 
 
 
 
"
C
h
l
o
r
i
d
e
"
,

 
 
 
 
 
 
 
 
"
B
i
c
a
r
b
o
n
a
t
e
"
,

 
 
 
 
 
 
 
 
"
G
l
u
c
o
s
e
"
,

 
 
 
 
 
 
 
 
"
C
a
l
c
i
u
m
"
,

 
 
 
 
 
 
 
 
"
M
a
g
n
e
s
i
u
m
"
,

 
 
 
 
 
 
 
 
"
A
n
i
o
n
 
G
a
p
"
,

 
 
 
 
 
 
 
 
"
O
s
m
o
l
a
l
i
t
y
"
,

 
 
 
 
 
 
 
 
"
P
h
o
s
p
h
a
t
e
"
,

 
 
 
 
]


 
 
 
 
#
 
F
l
a
t
 
l
i
s
t
 
o
f
 
a
l
l
 
l
a
b
 
i
t
e
m
 
I
D
s
 
f
o
r
 
f
i
l
t
e
r
i
n
g

 
 
 
 
L
A
B
I
T
E
M
S
:
 
C
l
a
s
s
V
a
r
[
L
i
s
t
[
s
t
r
]
]
 
=
 
[

 
 
 
 
 
 
 
 
i
t
e
m
 
f
o
r
 
i
t
e
m
i
d
s
 
i
n
 
L
A
B
_
C
A
T
E
G
O
R
I
E
S
.
v
a
l
u
e
s
(
)
 
f
o
r
 
i
t
e
m
 
i
n
 
i
t
e
m
i
d
s

 
 
 
 
]


 
 
 
 
d
e
f
 
_
_
c
a
l
l
_
_
(
s
e
l
f
,
 
p
a
t
i
e
n
t
:
 
A
n
y
)
 
-
>
 
L
i
s
t
[
D
i
c
t
[
s
t
r
,
 
A
n
y
]
]
:

 
 
 
 
 
 
 
 
"
"
"
P
r
o
c
e
s
s
 
a
 
p
a
t
i
e
n
t
 
t
o
 
c
r
e
a
t
e
 
m
o
r
t
a
l
i
t
y
 
p
r
e
d
i
c
t
i
o
n
 
s
a
m
p
l
e
s
.


 
 
 
 
 
 
 
 
C
r
e
a
t
e
s
 
O
N
E
 
s
a
m
p
l
e
 
p
e
r
 
p
a
t
i
e
n
t
 
w
i
t
h
 
a
l
l
 
a
d
m
i
s
s
i
o
n
s
 
a
g
g
r
e
g
a
t
e
d
.

 
 
 
 
 
 
 
 
T
i
m
e
 
i
n
t
e
r
v
a
l
s
 
a
r
e
 
c
a
l
c
u
l
a
t
e
d
 
b
e
t
w
e
e
n
 
c
o
n
s
e
c
u
t
i
v
e
 
a
d
m
i
s
s
i
o
n
s
.


 
 
 
 
 
 
 
 
A
r
g
s
:

 
 
 
 
 
 
 
 
 
 
 
 
p
a
t
i
e
n
t
:
 
P
a
t
i
e
n
t
 
o
b
j
e
c
t
 
w
i
t
h
 
g
e
t
_
e
v
e
n
t
s
 
m
e
t
h
o
d


 
 
 
 
 
 
 
 
R
e
t
u
r
n
s
:

 
 
 
 
 
 
 
 
 
 
 
 
L
i
s
t
 
w
i
t
h
 
s
i
n
g
l
e
 
s
a
m
p
l
e
 
c
o
n
t
a
i
n
i
n
g
 
p
a
t
i
e
n
t
_
i
d
,
 
a
l
l
 
c
o
n
d
i
t
i
o
n
s
,

 
 
 
 
 
 
 
 
 
 
 
 
p
r
o
c
e
d
u
r
e
s
,
 
l
a
b
s
 
a
c
r
o
s
s
 
v
i
s
i
t
s
,
 
a
n
d
 
f
i
n
a
l
 
m
o
r
t
a
l
i
t
y
 
l
a
b
e
l

 
 
 
 
 
 
 
 
"
"
"

 
 
 
 
 
 
 
 
#
 
F
i
l
t
e
r
 
p
a
t
i
e
n
t
s
 
b
y
 
a
g
e
 
(
>
=
 
1
8
)

 
 
 
 
 
 
 
 
d
e
m
o
g
r
a
p
h
i
c
s
 
=
 
p
a
t
i
e
n
t
.
g
e
t
_
e
v
e
n
t
s
(
e
v
e
n
t
_
t
y
p
e
=
"
p
a
t
i
e
n
t
s
"
)

 
 
 
 
 
 
 
 
i
f
 
n
o
t
 
d
e
m
o
g
r
a
p
h
i
c
s
:

 
 
 
 
 
 
 
 
 
 
 
 
r
e
t
u
r
n
 
[
]


 
 
 
 
 
 
 
 
d
e
m
o
g
r
a
p
h
i
c
s
 
=
 
d
e
m
o
g
r
a
p
h
i
c
s
[
0
]

 
 
 
 
 
 
 
 
t
r
y
:

 
 
 
 
 
 
 
 
 
 
 
 
a
n
c
h
o
r
_
a
g
e
 
=
 
i
n
t
(
d
e
m
o
g
r
a
p
h
i
c
s
.
a
n
c
h
o
r
_
a
g
e
)

 
 
 
 
 
 
 
 
 
 
 
 
i
f
 
a
n
c
h
o
r
_
a
g
e
 
<
 
1
8
:

 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
r
e
t
u
r
n
 
[
]

 
 
 
 
 
 
 
 
e
x
c
e
p
t
 
(
V
a
l
u
e
E
r
r
o
r
,
 
T
y
p
e
E
r
r
o
r
,
 
A
t
t
r
i
b
u
t
e
E
r
r
o
r
)
:

 
 
 
 
 
 
 
 
 
 
 
 
#
 
I
f
 
a
g
e
 
c
a
n
'
t
 
b
e
 
d
e
t
e
r
m
i
n
e
d
,
 
s
k
i
p
 
p
a
t
i
e
n
t

 
 
 
 
 
 
 
 
 
 
 
 
r
e
t
u
r
n
 
[
]


 
 
 
 
 
 
 
 
#
 
G
e
t
 
a
l
l
 
a
d
m
i
s
s
i
o
n
s

 
 
 
 
 
 
 
 
a
d
m
i
s
s
i
o
n
s
 
=
 
p
a
t
i
e
n
t
.
g
e
t
_
e
v
e
n
t
s
(
e
v
e
n
t
_
t
y
p
e
=
"
a
d
m
i
s
s
i
o
n
s
"
)

 
 
 
 
 
 
 
 
i
f
 
l
e
n
(
a
d
m
i
s
s
i
o
n
s
)
 
<
 
1
:

 
 
 
 
 
 
 
 
 
 
 
 
r
e
t
u
r
n
 
[
]


 
 
 
 
 
 
 
 
#
 
I
n
i
t
i
a
l
i
z
e
 
a
g
g
r
e
g
a
t
e
d
 
d
a
t
a
 
s
t
r
u
c
t
u
r
e
s

 
 
 
 
 
 
 
 
#
 
L
i
s
t
 
o
f
 
I
C
D
 
c
o
d
e
s
 
(
d
i
a
g
n
o
s
e
s
 
+
 
p
r
o
c
e
d
u
r
e
s
)
 
p
e
r
 
v
i
s
i
t

 
 
 
 
 
 
 
 
a
l
l
_
i
c
d
_
c
o
d
e
s
 
=
 
[
]

 
 
 
 
 
 
 
 
a
l
l
_
i
c
d
_
t
i
m
e
s
 
=
 
[
]
 
 
#
 
T
i
m
e
 
f
r
o
m
 
p
r
e
v
i
o
u
s
 
a
d
m
i
s
s
i
o
n
 
p
e
r
 
v
i
s
i
t

 
 
 
 
 
 
 
 
a
l
l
_
l
a
b
_
v
a
l
u
e
s
 
=
 
[
]
 
 
#
 
L
i
s
t
 
o
f
 
1
0
D
 
l
a
b
 
v
e
c
t
o
r
s

 
 
 
 
 
 
 
 
a
l
l
_
l
a
b
_
t
i
m
e
s
 
=
 
[
]
 
 
#
 
T
i
m
e
 
f
r
o
m
 
a
d
m
i
s
s
i
o
n
 
s
t
a
r
t
 
p
e
r
 
m
e
a
s
u
r
e
m
e
n
t


 
 
 
 
 
 
 
 
#
 
T
r
a
c
k
 
p
r
e
v
i
o
u
s
 
a
d
m
i
s
s
i
o
n
 
t
i
m
e
s
t
a
m
p
 
f
o
r
 
i
n
t
e
r
v
a
l
 
c
a
l
c
u
l
a
t
i
o
n

 
 
 
 
 
 
 
 
p
r
e
v
i
o
u
s
_
a
d
m
i
s
s
i
o
n
_
t
i
m
e
 
=
 
N
o
n
e


 
 
 
 
 
 
 
 
#
 
T
r
a
c
k
 
i
f
 
p
a
t
i
e
n
t
 
h
a
d
 
a
n
y
 
m
o
r
t
a
l
i
t
y
 
e
v
e
n
t

 
 
 
 
 
 
 
 
f
i
n
a
l
_
m
o
r
t
a
l
i
t
y
 
=
 
0


 
 
 
 
 
 
 
 
#
 
P
r
o
c
e
s
s
 
e
a
c
h
 
a
d
m
i
s
s
i
o
n

 
 
 
 
 
 
 
 
f
o
r
 
i
,
 
a
d
m
i
s
s
i
o
n
 
i
n
 
e
n
u
m
e
r
a
t
e
(
a
d
m
i
s
s
i
o
n
s
)
:

 
 
 
 
 
 
 
 
 
 
 
 
#
 
P
a
r
s
e
 
a
d
m
i
s
s
i
o
n
 
a
n
d
 
d
i
s
c
h
a
r
g
e
 
t
i
m
e
s

 
 
 
 
 
 
 
 
 
 
 
 
t
r
y
:

 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
a
d
m
i
s
s
i
o
n
_
t
i
m
e
 
=
 
a
d
m
i
s
s
i
o
n
.
t
i
m
e
s
t
a
m
p

 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
a
d
m
i
s
s
i
o
n
_
d
i
s
c
h
t
i
m
e
 
=
 
d
a
t
e
t
i
m
e
.
s
t
r
p
t
i
m
e
(

 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
a
d
m
i
s
s
i
o
n
.
d
i
s
c
h
t
i
m
e
,
 
"
%
Y
-
%
m
-
%
d
 
%
H
:
%
M
:
%
S
"

 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
)

 
 
 
 
 
 
 
 
 
 
 
 
e
x
c
e
p
t
 
(
V
a
l
u
e
E
r
r
o
r
,
 
A
t
t
r
i
b
u
t
e
E
r
r
o
r
)
:

 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
#
 
S
k
i
p
 
i
f
 
t
i
m
e
s
t
a
m
p
s
 
i
n
v
a
l
i
d

 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
c
o
n
t
i
n
u
e


 
 
 
 
 
 
 
 
 
 
 
 
#
 
S
k
i
p
 
i
f
 
d
i
s
c
h
a
r
g
e
 
i
s
 
b
e
f
o
r
e
 
a
d
m
i
s
s
i
o
n
 
(
d
a
t
a
 
q
u
a
l
i
t
y
 
i
s
s
u
e
)

 
 
 
 
 
 
 
 
 
 
 
 
i
f
 
a
d
m
i
s
s
i
o
n
_
d
i
s
c
h
t
i
m
e
 
<
 
a
d
m
i
s
s
i
o
n
_
t
i
m
e
:

 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
c
o
n
t
i
n
u
e


 
 
 
 
 
 
 
 
 
 
 
 
#
 
C
a
l
c
u
l
a
t
e
 
t
i
m
e
 
f
r
o
m
 
p
r
e
v
i
o
u
s
 
a
d
m
i
s
s
i
o
n
 
(
i
n
 
h
o
u
r
s
)

 
 
 
 
 
 
 
 
 
 
 
 
#
 
F
i
r
s
t
 
a
d
m
i
s
s
i
o
n
 
w
i
l
l
 
h
a
v
e
 
t
i
m
e
 
=
 
0

 
 
 
 
 
 
 
 
 
 
 
 
i
f
 
p
r
e
v
i
o
u
s
_
a
d
m
i
s
s
i
o
n
_
t
i
m
e
 
i
s
 
N
o
n
e
:

 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
t
i
m
e
_
f
r
o
m
_
p
r
e
v
i
o
u
s
 
=
 
0
.
0

 
 
 
 
 
 
 
 
 
 
 
 
e
l
s
e
:

 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
t
i
m
e
_
f
r
o
m
_
p
r
e
v
i
o
u
s
 
=
 
(

 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
a
d
m
i
s
s
i
o
n
_
t
i
m
e
 
-
 
p
r
e
v
i
o
u
s
_
a
d
m
i
s
s
i
o
n
_
t
i
m
e

 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
)
.
t
o
t
a
l
_
s
e
c
o
n
d
s
(
)
 
/
 
3
6
0
0
.
0


 
 
 
 
 
 
 
 
 
 
 
 
#
 
U
p
d
a
t
e
 
p
r
e
v
i
o
u
s
 
a
d
m
i
s
s
i
o
n
 
t
i
m
e
 
f
o
r
 
n
e
x
t
 
i
t
e
r
a
t
i
o
n

 
 
 
 
 
 
 
 
 
 
 
 
p
r
e
v
i
o
u
s
_
a
d
m
i
s
s
i
o
n
_
t
i
m
e
 
=
 
a
d
m
i
s
s
i
o
n
_
t
i
m
e


 
 
 
 
 
 
 
 
 
 
 
 
#
 
U
p
d
a
t
e
 
m
o
r
t
a
l
i
t
y
 
l
a
b
e
l
 
i
f
 
t
h
i
s
 
a
d
m
i
s
s
i
o
n
 
h
a
d
 
m
o
r
t
a
l
i
t
y

 
 
 
 
 
 
 
 
 
 
 
 
t
r
y
:

 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
i
f
 
i
n
t
(
a
d
m
i
s
s
i
o
n
.
h
o
s
p
i
t
a
l
_
e
x
p
i
r
e
_
f
l
a
g
)
 
=
=
 
1
:

 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
f
i
n
a
l
_
m
o
r
t
a
l
i
t
y
 
=
 
1

 
 
 
 
 
 
 
 
 
 
 
 
e
x
c
e
p
t
 
(
V
a
l
u
e
E
r
r
o
r
,
 
T
y
p
e
E
r
r
o
r
,
 
A
t
t
r
i
b
u
t
e
E
r
r
o
r
)
:

 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
p
a
s
s


 
 
 
 
 
 
 
 
 
 
 
 
#
 
G
e
t
 
d
i
a
g
n
o
s
i
s
 
c
o
d
e
s
 
f
o
r
 
t
h
i
s
 
a
d
m
i
s
s
i
o
n
 
u
s
i
n
g
 
h
a
d
m
_
i
d

 
 
 
 
 
 
 
 
 
 
 
 
d
i
a
g
n
o
s
e
s
_
i
c
d
 
=
 
p
a
t
i
e
n
t
.
g
e
t
_
e
v
e
n
t
s
(

 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
e
v
e
n
t
_
t
y
p
e
=
"
d
i
a
g
n
o
s
e
s
_
i
c
d
"
,

 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
f
i
l
t
e
r
s
=
[
(
"
h
a
d
m
_
i
d
"
,
 
"
=
=
"
,
 
a
d
m
i
s
s
i
o
n
.
h
a
d
m
_
i
d
)
]
,

 
 
 
 
 
 
 
 
 
 
 
 
)

 
 
 
 
 
 
 
 
 
 
 
 
v
i
s
i
t
_
d
i
a
g
n
o
s
e
s
 
=
 
[

 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
e
v
e
n
t
.
i
c
d
_
c
o
d
e

 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
f
o
r
 
e
v
e
n
t
 
i
n
 
d
i
a
g
n
o
s
e
s
_
i
c
d

 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
i
f
 
h
a
s
a
t
t
r
(
e
v
e
n
t
,
 
"
i
c
d
_
c
o
d
e
"
)
 
a
n
d
 
e
v
e
n
t
.
i
c
d
_
c
o
d
e

 
 
 
 
 
 
 
 
 
 
 
 
]


 
 
 
 
 
 
 
 
 
 
 
 
#
 
G
e
t
 
p
r
o
c
e
d
u
r
e
 
c
o
d
e
s
 
f
o
r
 
t
h
i
s
 
a
d
m
i
s
s
i
o
n
 
u
s
i
n
g
 
h
a
d
m
_
i
d

 
 
 
 
 
 
 
 
 
 
 
 
p
r
o
c
e
d
u
r
e
s
_
i
c
d
 
=
 
p
a
t
i
e
n
t
.
g
e
t
_
e
v
e
n
t
s
(

 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
e
v
e
n
t
_
t
y
p
e
=
"
p
r
o
c
e
d
u
r
e
s
_
i
c
d
"
,

 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
f
i
l
t
e
r
s
=
[
(
"
h
a
d
m
_
i
d
"
,
 
"
=
=
"
,
 
a
d
m
i
s
s
i
o
n
.
h
a
d
m
_
i
d
)
]
,

 
 
 
 
 
 
 
 
 
 
 
 
)

 
 
 
 
 
 
 
 
 
 
 
 
v
i
s
i
t
_
p
r
o
c
e
d
u
r
e
s
 
=
 
[

 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
e
v
e
n
t
.
i
c
d
_
c
o
d
e

 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
f
o
r
 
e
v
e
n
t
 
i
n
 
p
r
o
c
e
d
u
r
e
s
_
i
c
d

 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
i
f
 
h
a
s
a
t
t
r
(
e
v
e
n
t
,
 
"
i
c
d
_
c
o
d
e
"
)
 
a
n
d
 
e
v
e
n
t
.
i
c
d
_
c
o
d
e

 
 
 
 
 
 
 
 
 
 
 
 
]


 
 
 
 
 
 
 
 
 
 
 
 
#
 
C
o
m
b
i
n
e
 
d
i
a
g
n
o
s
e
s
 
a
n
d
 
p
r
o
c
e
d
u
r
e
s
 
i
n
t
o
 
s
i
n
g
l
e
 
I
C
D
 
c
o
d
e
 
l
i
s
t

 
 
 
 
 
 
 
 
 
 
 
 
v
i
s
i
t
_
i
c
d
_
c
o
d
e
s
 
=
 
v
i
s
i
t
_
d
i
a
g
n
o
s
e
s
 
+
 
v
i
s
i
t
_
p
r
o
c
e
d
u
r
e
s


 
 
 
 
 
 
 
 
 
 
 
 
i
f
 
v
i
s
i
t
_
i
c
d
_
c
o
d
e
s
:

 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
a
l
l
_
i
c
d
_
c
o
d
e
s
.
a
p
p
e
n
d
(
v
i
s
i
t
_
i
c
d
_
c
o
d
e
s
)

 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
a
l
l
_
i
c
d
_
t
i
m
e
s
.
a
p
p
e
n
d
(
t
i
m
e
_
f
r
o
m
_
p
r
e
v
i
o
u
s
)


 
 
 
 
 
 
 
 
 
 
 
 
#
 
G
e
t
 
l
a
b
 
e
v
e
n
t
s
 
f
o
r
 
t
h
i
s
 
a
d
m
i
s
s
i
o
n

 
 
 
 
 
 
 
 
 
 
 
 
l
a
b
e
v
e
n
t
s
_
d
f
 
=
 
p
a
t
i
e
n
t
.
g
e
t
_
e
v
e
n
t
s
(

 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
e
v
e
n
t
_
t
y
p
e
=
"
l
a
b
e
v
e
n
t
s
"
,

 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
s
t
a
r
t
=
a
d
m
i
s
s
i
o
n
_
t
i
m
e
,

 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
e
n
d
=
a
d
m
i
s
s
i
o
n
_
d
i
s
c
h
t
i
m
e
,

 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
r
e
t
u
r
n
_
d
f
=
T
r
u
e
,

 
 
 
 
 
 
 
 
 
 
 
 
)


 
 
 
 
 
 
 
 
 
 
 
 
#
 
F
i
l
t
e
r
 
t
o
 
r
e
l
e
v
a
n
t
 
l
a
b
 
i
t
e
m
s

 
 
 
 
 
 
 
 
 
 
 
 
l
a
b
e
v
e
n
t
s
_
d
f
 
=
 
l
a
b
e
v
e
n
t
s
_
d
f
.
f
i
l
t
e
r
(

 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
p
l
.
c
o
l
(
"
l
a
b
e
v
e
n
t
s
/
i
t
e
m
i
d
"
)
.
i
s
_
i
n
(
s
e
l
f
.
L
A
B
I
T
E
M
S
)

 
 
 
 
 
 
 
 
 
 
 
 
)


 
 
 
 
 
 
 
 
 
 
 
 
#
 
P
a
r
s
e
 
s
t
o
r
e
t
i
m
e
 
a
n
d
 
f
i
l
t
e
r

 
 
 
 
 
 
 
 
 
 
 
 
i
f
 
l
a
b
e
v
e
n
t
s
_
d
f
.
h
e
i
g
h
t
 
>
 
0
:

 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
l
a
b
e
v
e
n
t
s
_
d
f
 
=
 
l
a
b
e
v
e
n
t
s
_
d
f
.
w
i
t
h
_
c
o
l
u
m
n
s
(

 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
p
l
.
c
o
l
(
"
l
a
b
e
v
e
n
t
s
/
s
t
o
r
e
t
i
m
e
"
)
.
s
t
r
.
s
t
r
p
t
i
m
e
(

 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
p
l
.
D
a
t
e
t
i
m
e
,
 
"
%
Y
-
%
m
-
%
d
 
%
H
:
%
M
:
%
S
"

 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
)

 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
)

 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
l
a
b
e
v
e
n
t
s
_
d
f
 
=
 
l
a
b
e
v
e
n
t
s
_
d
f
.
f
i
l
t
e
r
(

 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
p
l
.
c
o
l
(
"
l
a
b
e
v
e
n
t
s
/
s
t
o
r
e
t
i
m
e
"
)
 
<
=
 
a
d
m
i
s
s
i
o
n
_
d
i
s
c
h
t
i
m
e

 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
)


 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
i
f
 
l
a
b
e
v
e
n
t
s
_
d
f
.
h
e
i
g
h
t
 
>
 
0
:

 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
#
 
S
e
l
e
c
t
 
r
e
l
e
v
a
n
t
 
c
o
l
u
m
n
s

 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
l
a
b
e
v
e
n
t
s
_
d
f
 
=
 
l
a
b
e
v
e
n
t
s
_
d
f
.
s
e
l
e
c
t
(

 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
p
l
.
c
o
l
(
"
t
i
m
e
s
t
a
m
p
"
)
,

 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
p
l
.
c
o
l
(
"
l
a
b
e
v
e
n
t
s
/
i
t
e
m
i
d
"
)
,

 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
p
l
.
c
o
l
(
"
l
a
b
e
v
e
n
t
s
/
v
a
l
u
e
n
u
m
"
)
.
c
a
s
t
(
p
l
.
F
l
o
a
t
6
4
)
,

 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
)


 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
#
 
G
r
o
u
p
 
b
y
 
t
i
m
e
s
t
a
m
p
 
a
n
d
 
a
g
g
r
e
g
a
t
e
 
i
n
t
o
 
1
0
D
 
v
e
c
t
o
r
s

 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
#
 
F
o
r
 
e
a
c
h
 
t
i
m
e
s
t
a
m
p
,
 
c
r
e
a
t
e
 
v
e
c
t
o
r
 
o
f
 
l
a
b
 
c
a
t
e
g
o
r
i
e
s

 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
u
n
i
q
u
e
_
t
i
m
e
s
t
a
m
p
s
 
=
 
s
o
r
t
e
d
(

 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
l
a
b
e
v
e
n
t
s
_
d
f
[
"
t
i
m
e
s
t
a
m
p
"
]
.
u
n
i
q
u
e
(
)
.
t
o
_
l
i
s
t
(
)

 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
)


 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
f
o
r
 
l
a
b
_
t
s
 
i
n
 
u
n
i
q
u
e
_
t
i
m
e
s
t
a
m
p
s
:

 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
#
 
G
e
t
 
a
l
l
 
l
a
b
 
e
v
e
n
t
s
 
a
t
 
t
h
i
s
 
t
i
m
e
s
t
a
m
p

 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
t
s
_
l
a
b
s
 
=
 
l
a
b
e
v
e
n
t
s
_
d
f
.
f
i
l
t
e
r
(
p
l
.
c
o
l
(
"
t
i
m
e
s
t
a
m
p
"
)
 
=
=
 
l
a
b
_
t
s
)


 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
#
 
C
r
e
a
t
e
 
1
0
-
d
i
m
e
n
s
i
o
n
a
l
 
v
e
c
t
o
r
 
(
o
n
e
 
p
e
r
 
c
a
t
e
g
o
r
y
)

 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
l
a
b
_
v
e
c
t
o
r
 
=
 
[
]

 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
f
o
r
 
c
a
t
e
g
o
r
y
_
n
a
m
e
 
i
n
 
s
e
l
f
.
L
A
B
_
C
A
T
E
G
O
R
Y
_
N
A
M
E
S
:

 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
c
a
t
e
g
o
r
y
_
i
t
e
m
i
d
s
 
=
 
s
e
l
f
.
L
A
B
_
C
A
T
E
G
O
R
I
E
S
[
c
a
t
e
g
o
r
y
_
n
a
m
e
]


 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
#
 
F
i
n
d
 
f
i
r
s
t
 
m
a
t
c
h
i
n
g
 
v
a
l
u
e
 
f
o
r
 
t
h
i
s
 
c
a
t
e
g
o
r
y

 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
c
a
t
e
g
o
r
y
_
v
a
l
u
e
 
=
 
N
o
n
e

 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
f
o
r
 
i
t
e
m
i
d
 
i
n
 
c
a
t
e
g
o
r
y
_
i
t
e
m
i
d
s
:

 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
m
a
t
c
h
i
n
g
 
=
 
t
s
_
l
a
b
s
.
f
i
l
t
e
r
(

 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
p
l
.
c
o
l
(
"
l
a
b
e
v
e
n
t
s
/
i
t
e
m
i
d
"
)
 
=
=
 
i
t
e
m
i
d

 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
)

 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
i
f
 
m
a
t
c
h
i
n
g
.
h
e
i
g
h
t
 
>
 
0
:

 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
c
a
t
e
g
o
r
y
_
v
a
l
u
e
 
=
 
m
a
t
c
h
i
n
g
[
"
l
a
b
e
v
e
n
t
s
/
v
a
l
u
e
n
u
m
"
]
[
0
]

 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
b
r
e
a
k


 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
l
a
b
_
v
e
c
t
o
r
.
a
p
p
e
n
d
(
c
a
t
e
g
o
r
y
_
v
a
l
u
e
)


 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
#
 
C
a
l
c
u
l
a
t
e
 
t
i
m
e
 
f
r
o
m
 
a
d
m
i
s
s
i
o
n
 
s
t
a
r
t
 
(
h
o
u
r
s
)

 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
t
i
m
e
_
f
r
o
m
_
a
d
m
i
s
s
i
o
n
 
=
 
(

 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
l
a
b
_
t
s
 
-
 
a
d
m
i
s
s
i
o
n
_
t
i
m
e

 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
)
.
t
o
t
a
l
_
s
e
c
o
n
d
s
(
)
 
/
 
3
6
0
0
.
0


 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
a
l
l
_
l
a
b
_
v
a
l
u
e
s
.
a
p
p
e
n
d
(
l
a
b
_
v
e
c
t
o
r
)

 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
a
l
l
_
l
a
b
_
t
i
m
e
s
.
a
p
p
e
n
d
(
t
i
m
e
_
f
r
o
m
_
a
d
m
i
s
s
i
o
n
)


 
 
 
 
 
 
 
 
#
 
S
k
i
p
 
i
f
 
n
o
 
l
a
b
 
e
v
e
n
t
s
 
(
r
e
q
u
i
r
e
d
 
f
o
r
 
t
h
i
s
 
t
a
s
k
)

 
 
 
 
 
 
 
 
i
f
 
l
e
n
(
a
l
l
_
l
a
b
_
v
a
l
u
e
s
)
 
=
=
 
0
:

 
 
 
 
 
 
 
 
 
 
 
 
r
e
t
u
r
n
 
[
]


 
 
 
 
 
 
 
 
#
 
A
l
s
o
 
s
k
i
p
 
i
f
 
n
o
 
I
C
D
 
c
o
d
e
s
 
a
c
r
o
s
s
 
a
l
l
 
a
d
m
i
s
s
i
o
n
s

 
 
 
 
 
 
 
 
i
f
 
l
e
n
(
a
l
l
_
i
c
d
_
c
o
d
e
s
)
 
=
=
 
0
:

 
 
 
 
 
 
 
 
 
 
 
 
r
e
t
u
r
n
 
[
]


 
 
 
 
 
 
 
 
#
 
F
o
r
m
a
t
 
a
s
 
t
u
p
l
e
s
:
 
(
t
i
m
e
,
 
v
a
l
u
e
s
)

 
 
 
 
 
 
 
 
#
 
I
C
D
 
c
o
d
e
s
:
 
n
e
s
t
e
d
 
l
i
s
t
 
w
i
t
h
 
t
i
m
e
s

 
 
 
 
 
 
 
 
i
c
d
_
c
o
d
e
s
_
d
a
t
a
 
=
 
(
a
l
l
_
i
c
d
_
t
i
m
e
s
,
 
a
l
l
_
i
c
d
_
c
o
d
e
s
)


 
 
 
 
 
 
 
 
#
 
L
a
b
s
:
 
l
i
s
t
 
o
f
 
1
0
D
 
v
e
c
t
o
r
s
 
w
i
t
h
 
t
i
m
e
s

 
 
 
 
 
 
 
 
l
a
b
s
_
d
a
t
a
 
=
 
(
a
l
l
_
l
a
b
_
t
i
m
e
s
,
 
a
l
l
_
l
a
b
_
v
a
l
u
e
s
)


 
 
 
 
 
 
 
 
#
 
C
r
e
a
t
e
 
s
i
n
g
l
e
 
p
a
t
i
e
n
t
-
l
e
v
e
l
 
s
a
m
p
l
e

 
 
 
 
 
 
 
 
s
a
m
p
l
e
 
=
 
{

 
 
 
 
 
 
 
 
 
 
 
 
"
p
a
t
i
e
n
t
_
i
d
"
:
 
p
a
t
i
e
n
t
.
p
a
t
i
e
n
t
_
i
d
,

 
 
 
 
 
 
 
 
 
 
 
 
"
i
c
d
_
c
o
d
e
s
"
:
 
i
c
d
_
c
o
d
e
s
_
d
a
t
a
,

 
 
 
 
 
 
 
 
 
 
 
 
"
l
a
b
s
"
:
 
l
a
b
s
_
d
a
t
a
,

 
 
 
 
 
 
 
 
 
 
 
 
"
m
o
r
t
a
l
i
t
y
"
:
 
f
i
n
a
l
_
m
o
r
t
a
l
i
t
y
,

 
 
 
 
 
 
 
 
}

 
 
 
 
 
 
 
 
r
e
t
u
r
n
 
[
s
a
m
p
l
e
]



#
#
 
S
e
t
t
i
n
g
 
t
h
e
 
t
a
s
k
 
a
n
d
 
c
a
c
h
i
n
g
 
t
h
e
 
d
a
t
a
 
f
o
r
 
q
u
i
c
k
e
r
 
u
s
e
 
d
o
w
n
 
t
h
e
 
r
o
a
d
 
w
i
t
h
 
p
a
d
d
i
n
g

W
e
 
c
a
n
 
f
i
n
a
l
l
y
 
s
e
t
 
o
u
r
 
t
a
s
k
 
a
n
d
 
g
e
t
 
o
u
r
 
t
r
a
i
n
i
n
g
 
s
e
t
 
b
e
l
o
w
.
 
N
o
t
i
c
e
 
t
h
a
t
 
w
e
 
s
a
v
e
 
a
 
p
r
o
c
e
s
s
e
d
 
v
e
r
s
i
o
n
 
o
f
 
o
u
r
 
d
a
t
a
s
e
t
 
i
n
 
.
p
a
r
q
u
e
t
 
f
i
l
e
s
 
i
n
 
o
u
r
 
"
c
a
c
h
e
_
d
i
r
"
 
h
e
r
e
.
 
W
e
 
c
a
n
 
a
l
s
o
 
d
e
f
i
n
e
 
a
 
n
u
m
b
e
r
 
o
f
 
w
o
r
k
s
 
f
o
r
 
f
a
s
t
e
r
 
p
a
r
a
l
l
e
l
 
p
r
o
c
e
s
s
i
n
g
 
(
n
o
t
e
 
t
h
i
s
 
c
a
n
 
b
e
 
u
n
s
t
a
b
l
e
 
i
f
 
t
h
e
 
v
a
l
u
e
 
i
s
 
t
o
o
 
h
i
g
h
)
.


W
e
 
c
a
n
 
a
l
s
o
 
s
a
v
e
 
a
n
d
 
l
o
a
d
 
p
r
o
c
e
s
s
o
r
s
 
s
o
 
w
e
 
d
o
n
'
t
 
n
e
e
d
 
t
o
 
r
e
f
i
t
 
t
h
e
 
p
r
o
c
e
s
s
o
r
 
a
g
a
i
n
 
(
a
n
d
 
w
e
 
c
a
n
 
a
l
s
o
 
t
r
a
n
s
f
e
r
 
p
r
o
c
e
s
s
o
r
s
 
a
c
r
o
s
s
 
d
i
f
f
e
r
e
n
t
 
s
a
m
p
l
e
s
)

In [4]:
f
r
o
m
 
p
y
h
e
a
l
t
h
.
d
a
t
a
s
e
t
s
.
u
t
i
l
s
 
i
m
p
o
r
t
 
s
a
v
e
_
p
r
o
c
e
s
s
o
r
s
,
 
l
o
a
d
_
p
r
o
c
e
s
s
o
r
s

i
m
p
o
r
t
 
o
s
 

p
r
o
c
e
s
s
o
r
_
d
i
r
 
=
 
"
.
.
/
.
.
/
o
u
t
p
u
t
/
p
r
o
c
e
s
s
o
r
s
/
s
t
a
g
e
n
e
t
_
m
o
r
t
a
l
i
t
y
_
m
i
m
i
c
4
"

c
a
c
h
e
_
d
i
r
 
=
 
"
.
.
/
.
.
/
m
i
m
i
c
4
_
s
t
a
g
e
n
e
t
_
c
a
c
h
e
_
v
3
"


i
f
 
o
s
.
p
a
t
h
.
e
x
i
s
t
s
(
o
s
.
p
a
t
h
.
j
o
i
n
(
p
r
o
c
e
s
s
o
r
_
d
i
r
,
 
"
i
n
p
u
t
_
p
r
o
c
e
s
s
o
r
s
.
p
k
l
"
)
)
:

 
 
 
 
p
r
i
n
t
(
"
\
n
=
=
=
 
L
o
a
d
i
n
g
 
P
r
e
-
f
i
t
t
e
d
 
P
r
o
c
e
s
s
o
r
s
 
=
=
=
"
)

 
 
 
 
i
n
p
u
t
_
p
r
o
c
e
s
s
o
r
s
,
 
o
u
t
p
u
t
_
p
r
o
c
e
s
s
o
r
s
 
=
 
l
o
a
d
_
p
r
o
c
e
s
s
o
r
s
(
p
r
o
c
e
s
s
o
r
_
d
i
r
)


 
 
 
 
s
a
m
p
l
e
_
d
a
t
a
s
e
t
 
=
 
b
a
s
e
_
d
a
t
a
s
e
t
.
s
e
t
_
t
a
s
k
(

 
 
 
 
 
 
 
 
M
o
r
t
a
l
i
t
y
P
r
e
d
i
c
t
i
o
n
S
t
a
g
e
N
e
t
M
I
M
I
C
4
(
p
a
d
d
i
n
g
=
2
0
)
,

 
 
 
 
 
 
 
 
n
u
m
_
w
o
r
k
e
r
s
=
1
,

 
 
 
 
 
 
 
 
c
a
c
h
e
_
d
i
r
=
c
a
c
h
e
_
d
i
r
,

 
 
 
 
 
 
 
 
i
n
p
u
t
_
p
r
o
c
e
s
s
o
r
s
=
i
n
p
u
t
_
p
r
o
c
e
s
s
o
r
s
,

 
 
 
 
 
 
 
 
o
u
t
p
u
t
_
p
r
o
c
e
s
s
o
r
s
=
o
u
t
p
u
t
_
p
r
o
c
e
s
s
o
r
s
,

 
 
 
 
)

e
l
s
e
:

 
 
 
 
p
r
i
n
t
(
"
\
n
=
=
=
 
F
i
t
t
i
n
g
 
N
e
w
 
P
r
o
c
e
s
s
o
r
s
 
=
=
=
"
)

 
 
 
 
s
a
m
p
l
e
_
d
a
t
a
s
e
t
 
=
 
b
a
s
e
_
d
a
t
a
s
e
t
.
s
e
t
_
t
a
s
k
(

 
 
 
 
 
 
 
 
M
o
r
t
a
l
i
t
y
P
r
e
d
i
c
t
i
o
n
S
t
a
g
e
N
e
t
M
I
M
I
C
4
(
p
a
d
d
i
n
g
=
2
0
)
,

 
 
 
 
 
 
 
 
n
u
m
_
w
o
r
k
e
r
s
=
1
,

 
 
 
 
 
 
 
 
c
a
c
h
e
_
d
i
r
=
c
a
c
h
e
_
d
i
r
,

 
 
 
 
)


 
 
 
 
#
 
S
a
v
e
 
p
r
o
c
e
s
s
o
r
s
 
f
o
r
 
f
u
t
u
r
e
 
r
u
n
s

 
 
 
 
p
r
i
n
t
(
"
\
n
=
=
=
 
S
a
v
i
n
g
 
P
r
o
c
e
s
s
o
r
s
 
=
=
=
"
)

 
 
 
 
s
a
v
e
_
p
r
o
c
e
s
s
o
r
s
(
s
a
m
p
l
e
_
d
a
t
a
s
e
t
,
 
p
r
o
c
e
s
s
o
r
_
d
i
r
)


p
r
i
n
t
(
f
"
T
o
t
a
l
 
s
a
m
p
l
e
s
:
 
{
l
e
n
(
s
a
m
p
l
e
_
d
a
t
a
s
e
t
)
}
"
)

p
r
i
n
t
(
f
"
I
n
p
u
t
 
s
c
h
e
m
a
:
 
{
s
a
m
p
l
e
_
d
a
t
a
s
e
t
.
i
n
p
u
t
_
s
c
h
e
m
a
}
"
)

p
r
i
n
t
(
f
"
O
u
t
p
u
t
 
s
c
h
e
m
a
:
 
{
s
a
m
p
l
e
_
d
a
t
a
s
e
t
.
o
u
t
p
u
t
_
s
c
h
e
m
a
}
"
)


=
=
=
 
L
o
a
d
i
n
g
 
P
r
e
-
f
i
t
t
e
d
 
P
r
o
c
e
s
s
o
r
s
 
=
=
=

✓
 
L
o
a
d
e
d
 
i
n
p
u
t
 
p
r
o
c
e
s
s
o
r
s
 
f
r
o
m
 
.
.
/
.
.
/
o
u
t
p
u
t
/
p
r
o
c
e
s
s
o
r
s
/
s
t
a
g
e
n
e
t
_
m
o
r
t
a
l
i
t
y
_
m
i
m
i
c
4
/
i
n
p
u
t
_
p
r
o
c
e
s
s
o
r
s
.
p
k
l

✓
 
L
o
a
d
e
d
 
o
u
t
p
u
t
 
p
r
o
c
e
s
s
o
r
s
 
f
r
o
m
 
.
.
/
.
.
/
o
u
t
p
u
t
/
p
r
o
c
e
s
s
o
r
s
/
s
t
a
g
e
n
e
t
_
m
o
r
t
a
l
i
t
y
_
m
i
m
i
c
4
/
o
u
t
p
u
t
_
p
r
o
c
e
s
s
o
r
s
.
p
k
l

S
e
t
t
i
n
g
 
t
a
s
k
 
M
o
r
t
a
l
i
t
y
P
r
e
d
i
c
t
i
o
n
S
t
a
g
e
N
e
t
M
I
M
I
C
4
 
f
o
r
 
m
i
m
i
c
4
 
b
a
s
e
 
d
a
t
a
s
e
t
.
.
.

G
e
n
e
r
a
t
i
n
g
 
s
a
m
p
l
e
s
 
w
i
t
h
 
1
 
w
o
r
k
e
r
(
s
)
.
.
.

C
o
l
l
e
c
t
i
n
g
 
g
l
o
b
a
l
 
e
v
e
n
t
 
d
a
t
a
f
r
a
m
e
.
.
.

D
e
v
 
m
o
d
e
 
e
n
a
b
l
e
d
:
 
l
i
m
i
t
i
n
g
 
t
o
 
1
0
0
0
 
p
a
t
i
e
n
t
s

C
o
l
l
e
c
t
e
d
 
d
a
t
a
f
r
a
m
e
 
w
i
t
h
 
s
h
a
p
e
:
 
(
3
9
8
3
6
0
,
 
3
9
)



G
e
n
e
r
a
t
i
n
g
 
s
a
m
p
l
e
s
 
f
o
r
 
M
o
r
t
a
l
i
t
y
P
r
e
d
i
c
t
i
o
n
S
t
a
g
e
N
e
t
M
I
M
I
C
4
 
w
i
t
h
 
1
 
w
o
r
k
e
r
:
 
1
0
0
%
|
█
█
█
█
█
█
█
█
█
█
|
 
1
0
0
0
/
1
0
0
0
 
[
0
0
:
3
3
<
0
0
:
0
0
,
 
3
0
.
2
4
i
t
/
s
]

C
a
c
h
i
n
g
 
s
a
m
p
l
e
s
 
t
o
 
.
.
/
.
.
/
m
i
m
i
c
4
_
s
t
a
g
e
n
e
t
_
c
a
c
h
e
_
v
3
/
M
o
r
t
a
l
i
t
y
P
r
e
d
i
c
t
i
o
n
S
t
a
g
e
N
e
t
M
I
M
I
C
4
.
p
a
r
q
u
e
t

F
a
i
l
e
d
 
t
o
 
c
a
c
h
e
 
s
a
m
p
l
e
s
:
 
f
a
i
l
e
d
 
t
o
 
d
e
t
e
r
m
i
n
e
 
s
u
p
e
r
t
y
p
e
 
o
f
 
l
i
s
t
[
f
6
4
]
 
a
n
d
 
l
i
s
t
[
l
i
s
t
[
s
t
r
]
]




P
r
o
c
e
s
s
i
n
g
 
s
a
m
p
l
e
s
:
 
1
0
0
%
|
█
█
█
█
█
█
█
█
█
█
|
 
4
4
5
/
4
4
5
 
[
0
0
:
0
0
<
0
0
:
0
0
,
 
4
5
4
2
.
9
8
i
t
/
s
]

G
e
n
e
r
a
t
e
d
 
4
4
5
 
s
a
m
p
l
e
s
 
f
o
r
 
t
a
s
k
 
M
o
r
t
a
l
i
t
y
P
r
e
d
i
c
t
i
o
n
S
t
a
g
e
N
e
t
M
I
M
I
C
4

T
o
t
a
l
 
s
a
m
p
l
e
s
:
 
4
4
5

I
n
p
u
t
 
s
c
h
e
m
a
:
 
{
'
i
c
d
_
c
o
d
e
s
'
:
 
(
'
s
t
a
g
e
n
e
t
'
,
 
{
'
p
a
d
d
i
n
g
'
:
 
2
0
}
)
,
 
'
l
a
b
s
'
:
 
(
'
s
t
a
g
e
n
e
t
_
t
e
n
s
o
r
'
,
 
{
}
)
}

O
u
t
p
u
t
 
s
c
h
e
m
a
:
 
{
'
m
o
r
t
a
l
i
t
y
'
:
 
'
b
i
n
a
r
y
'
}



In [5]:
#
 
I
n
s
p
e
c
t
 
a
 
s
a
m
p
l
e

s
a
m
p
l
e
 
=
 
s
a
m
p
l
e
_
d
a
t
a
s
e
t
.
s
a
m
p
l
e
s
[
0
]

p
r
i
n
t
(
"
\
n
S
a
m
p
l
e
 
s
t
r
u
c
t
u
r
e
:
"
)

p
r
i
n
t
(
f
"
 
 
P
a
t
i
e
n
t
 
I
D
:
 
{
s
a
m
p
l
e
[
'
p
a
t
i
e
n
t
_
i
d
'
]
}
"
)

p
r
i
n
t
(
f
"
I
C
D
 
C
o
d
e
s
:
 
{
s
a
m
p
l
e
[
'
i
c
d
_
c
o
d
e
s
'
]
}
"
)

p
r
i
n
t
(
f
"
 
 
L
a
b
s
 
s
h
a
p
e
:
 
{
l
e
n
(
s
a
m
p
l
e
[
'
l
a
b
s
'
]
[
0
]
)
}
 
t
i
m
e
s
t
e
p
s
"
)

p
r
i
n
t
(
f
"
 
 
M
o
r
t
a
l
i
t
y
:
 
{
s
a
m
p
l
e
[
'
m
o
r
t
a
l
i
t
y
'
]
}
"
)





S
a
m
p
l
e
 
s
t
r
u
c
t
u
r
e
:

 
 
P
a
t
i
e
n
t
 
I
D
:
 
1
1
2
0
4
8
0
1

I
C
D
 
C
o
d
e
s
:
 
(
t
e
n
s
o
r
(
[
 
 
0
.
0
0
0
0
,
 
3
6
6
.
8
1
6
7
,
 
 
1
0
.
1
5
0
0
]
)
,
 
t
e
n
s
o
r
(
[
[
3
6
5
6
,
 
1
3
3
8
,
 
 
3
4
4
,
 
3
6
5
6
,
 
1
5
9
9
,
 
1
0
8
2
,
 
3
6
5
6
,
 
 
4
9
1
,
 
 
1
8
9
,
 
 
 
1
6
,
 
 
9
8
5
,
 
 
3
5
7
,

 
 
 
 
 
 
 
 
 
3
6
5
6
,
 
3
6
5
6
,
 
1
3
3
9
,
 
3
6
5
6
,
 
 
8
1
2
,
 
2
5
2
3
,
 
 
 
 
0
,
 
 
 
 
0
,
 
 
 
 
0
,
 
 
 
 
0
,
 
 
 
 
0
,
 
 
 
 
0
,

 
 
 
 
 
 
 
 
 
 
 
 
0
,
 
 
 
 
0
,
 
 
 
 
0
,
 
 
 
 
0
,
 
 
 
 
0
,
 
 
 
 
0
,
 
 
 
 
0
,
 
 
 
 
0
,
 
 
 
 
0
,
 
 
 
 
0
,
 
 
 
 
0
,
 
 
 
 
0
,

 
 
 
 
 
 
 
 
 
 
 
 
0
,
 
 
 
 
0
,
 
 
 
 
0
,
 
 
 
 
0
,
 
 
 
 
0
,
 
 
 
 
0
,
 
 
 
 
0
,
 
 
 
 
0
,
 
 
 
 
0
,
 
 
 
 
0
,
 
 
 
 
0
,
 
 
 
 
0
,

 
 
 
 
 
 
 
 
 
 
 
 
0
,
 
 
 
 
0
,
 
 
 
 
0
,
 
 
 
 
0
,
 
 
 
 
0
,
 
 
 
 
0
,
 
 
 
 
0
,
 
 
 
 
0
,
 
 
 
 
0
,
 
 
 
 
0
,
 
 
 
 
0
,
 
 
 
 
0
,

 
 
 
 
 


#
#
 
T
r
a
i
n
,
 
V
a
l
i
d
a
t
i
o
n
,
 
T
e
s
t
 
S
p
l
i
t
s
 
a
n
d
 
T
r
a
i
n
i
n
g


T
h
i
s
 
s
e
c
t
i
o
n
 
f
u
n
d
a
m
e
n
t
a
l
l
y
 
f
o
l
l
o
w
s
 
a
n
y
 
t
y
p
i
c
a
l
 
t
r
a
i
n
i
n
g
 
p
i
p
e
l
i
n
e
.
 
W
e
 
d
o
n
'
t
 
r
e
c
o
m
m
e
n
d
 
t
h
e
 
P
y
H
e
a
l
t
h
 
t
r
a
i
n
e
r
 
b
e
y
o
n
d
 
j
u
s
t
 
t
e
s
t
i
n
g
 
o
u
t
 
b
a
s
e
l
i
n
e
s
,
 
b
u
t
 
a
n
y
 
c
o
d
e
 
y
o
u
 
w
r
i
t
e
 
h
e
r
e
 
s
h
o
u
l
d
 
f
l
e
x
i
b
l
y
 
t
r
a
n
s
l
a
t
e
 
t
o
 
m
o
r
e
 
a
d
v
a
n
c
e
d
 
d
e
e
p
 
l
e
a
r
n
i
n
g
 
t
r
a
i
n
i
n
g
 
p
a
c
k
a
g
e
s
 
l
i
k
e
 
P
y
T
o
r
c
h
 
l
i
g
h
t
n
i
n
g
 
a
n
d
 
m
a
n
y
 
o
t
h
e
r
s
.

In [6]:
s
a
m
p
l
e
_
d
a
t
a
s
e
t
.
i
n
p
u
t
_
s
c
h
e
m
a

{'icd_codes': ('stagenet', {'padding': 20}), 'labs': ('stagenet_tensor', {})}

In [7]:
#
 
S
T
E
P
 
3
:
 
S
p
l
i
t
 
d
a
t
a
s
e
t

t
r
a
i
n
_
d
a
t
a
s
e
t
,
 
v
a
l
_
d
a
t
a
s
e
t
,
 
t
e
s
t
_
d
a
t
a
s
e
t
 
=
 
s
p
l
i
t
_
b
y
_
p
a
t
i
e
n
t
(

 
 
 
 
s
a
m
p
l
e
_
d
a
t
a
s
e
t
,
 
[
0
.
8
,
 
0
.
1
,
 
0
.
1
]

)


#
 
C
r
e
a
t
e
 
d
a
t
a
l
o
a
d
e
r
s

t
r
a
i
n
_
l
o
a
d
e
r
 
=
 
g
e
t
_
d
a
t
a
l
o
a
d
e
r
(
t
r
a
i
n
_
d
a
t
a
s
e
t
,
 
b
a
t
c
h
_
s
i
z
e
=
2
5
6
,
 
s
h
u
f
f
l
e
=
T
r
u
e
)

v
a
l
_
l
o
a
d
e
r
 
=
 
g
e
t
_
d
a
t
a
l
o
a
d
e
r
(
v
a
l
_
d
a
t
a
s
e
t
,
 
b
a
t
c
h
_
s
i
z
e
=
2
5
6
,
 
s
h
u
f
f
l
e
=
F
a
l
s
e
)

t
e
s
t
_
l
o
a
d
e
r
 
=
 
g
e
t
_
d
a
t
a
l
o
a
d
e
r
(
t
e
s
t
_
d
a
t
a
s
e
t
,
 
b
a
t
c
h
_
s
i
z
e
=
2
5
6
,
 
s
h
u
f
f
l
e
=
F
a
l
s
e
)


#
 
S
T
E
P
 
4
:
 
I
n
i
t
i
a
l
i
z
e
 
S
t
a
g
e
N
e
t
 
m
o
d
e
l

m
o
d
e
l
 
=
 
S
t
a
g
e
N
e
t
(

 
 
 
 
d
a
t
a
s
e
t
=
s
a
m
p
l
e
_
d
a
t
a
s
e
t
,

 
 
 
 
e
m
b
e
d
d
i
n
g
_
d
i
m
=
1
2
8
,

 
 
 
 
c
h
u
n
k
_
s
i
z
e
=
1
2
8
,

 
 
 
 
l
e
v
e
l
s
=
3
,

 
 
 
 
d
r
o
p
o
u
t
=
0
.
3
,

)


n
u
m
_
p
a
r
a
m
s
 
=
 
s
u
m
(
p
.
n
u
m
e
l
(
)
 
f
o
r
 
p
 
i
n
 
m
o
d
e
l
.
p
a
r
a
m
e
t
e
r
s
(
)
)

p
r
i
n
t
(
f
"
\
n
M
o
d
e
l
 
i
n
i
t
i
a
l
i
z
e
d
 
w
i
t
h
 
{
n
u
m
_
p
a
r
a
m
s
}
 
p
a
r
a
m
e
t
e
r
s
"
)


#
 
S
T
E
P
 
5
:
 
T
r
a
i
n
 
t
h
e
 
m
o
d
e
l

t
r
a
i
n
e
r
 
=
 
T
r
a
i
n
e
r
(

 
 
 
 
m
o
d
e
l
=
m
o
d
e
l
,

 
 
 
 
d
e
v
i
c
e
=
"
c
u
d
a
:
4
"
,
 
 
#
 
o
r
 
"
c
p
u
"

 
 
 
 
m
e
t
r
i
c
s
=
[
"
p
r
_
a
u
c
"
,
 
"
r
o
c
_
a
u
c
"
,
 
"
a
c
c
u
r
a
c
y
"
,
 
"
f
1
"
]
,

)


#
 
1
 
e
p
o
c
h
 
f
o
r
 
d
e
m
o
n
s
t
r
a
t
i
o
n
;
 
i
n
c
r
e
a
s
e
 
f
o
r
 
r
e
a
l
 
t
r
a
i
n
i
n
g
,
 
i
t
 
s
h
o
u
l
d
 
w
o
r
k
 
p
r
e
t
t
y
 
w
e
l
l
 
c
l
o
s
e
r
 
t
o
 
5
0

t
r
a
i
n
e
r
.
t
r
a
i
n
(

 
 
 
 
t
r
a
i
n
_
d
a
t
a
l
o
a
d
e
r
=
t
r
a
i
n
_
l
o
a
d
e
r
,

 
 
 
 
v
a
l
_
d
a
t
a
l
o
a
d
e
r
=
v
a
l
_
l
o
a
d
e
r
,

 
 
 
 
e
p
o
c
h
s
=
1
,

 
 
 
 
m
o
n
i
t
o
r
=
"
r
o
c
_
a
u
c
"
,

 
 
 
 
o
p
t
i
m
i
z
e
r
_
p
a
r
a
m
s
=
{
"
l
r
"
:
 
1
e
-
5
}
,

)


#
 
S
T
E
P
 
6
:
 
E
v
a
l
u
a
t
e
 
o
n
 
t
e
s
t
 
s
e
t

r
e
s
u
l
t
s
 
=
 
t
r
a
i
n
e
r
.
e
v
a
l
u
a
t
e
(
t
e
s
t
_
l
o
a
d
e
r
)

p
r
i
n
t
(
"
\
n
T
e
s
t
 
R
e
s
u
l
t
s
:
"
)

f
o
r
 
m
e
t
r
i
c
,
 
v
a
l
u
e
 
i
n
 
r
e
s
u
l
t
s
.
i
t
e
m
s
(
)
:

 
 
 
 
p
r
i
n
t
(
f
"
 
 
{
m
e
t
r
i
c
}
:
 
{
v
a
l
u
e
:
.
4
f
}
"
)


#
 
S
T
E
P
 
7
:
 
I
n
s
p
e
c
t
 
m
o
d
e
l
 
p
r
e
d
i
c
t
i
o
n
s

s
a
m
p
l
e
_
b
a
t
c
h
 
=
 
n
e
x
t
(
i
t
e
r
(
t
e
s
t
_
l
o
a
d
e
r
)
)

w
i
t
h
 
t
o
r
c
h
.
n
o
_
g
r
a
d
(
)
:

 
 
 
 
o
u
t
p
u
t
 
=
 
m
o
d
e
l
(
*
*
s
a
m
p
l
e
_
b
a
t
c
h
)


p
r
i
n
t
(
"
\
n
S
a
m
p
l
e
 
p
r
e
d
i
c
t
i
o
n
s
:
"
)

p
r
i
n
t
(
f
"
 
 
P
r
e
d
i
c
t
e
d
 
p
r
o
b
a
b
i
l
i
t
i
e
s
:
 
{
o
u
t
p
u
t
[
'
y
_
p
r
o
b
'
]
[
:
5
]
}
"
)

p
r
i
n
t
(
f
"
 
 
T
r
u
e
 
l
a
b
e
l
s
:
 
{
o
u
t
p
u
t
[
'
y
_
t
r
u
e
'
]
[
:
5
]
}
"
)


M
o
d
e
l
 
i
n
i
t
i
a
l
i
z
e
d
 
w
i
t
h
 
5
1
1
0
7
0
5
 
p
a
r
a
m
e
t
e
r
s

S
t
a
g
e
N
e
t
(

 
 
(
e
m
b
e
d
d
i
n
g
_
m
o
d
e
l
)
:
 
E
m
b
e
d
d
i
n
g
M
o
d
e
l
(
e
m
b
e
d
d
i
n
g
_
l
a
y
e
r
s
=
M
o
d
u
l
e
D
i
c
t
(

 
 
 
 
(
i
c
d
_
c
o
d
e
s
)
:
 
E
m
b
e
d
d
i
n
g
(
3
6
5
7
,
 
1
2
8
,
 
p
a
d
d
i
n
g
_
i
d
x
=
0
)

 
 
 
 
(
l
a
b
s
)
:
 
L
i
n
e
a
r
(
i
n
_
f
e
a
t
u
r
e
s
=
1
0
,
 
o
u
t
_
f
e
a
t
u
r
e
s
=
1
2
8
,
 
b
i
a
s
=
T
r
u
e
)

 
 
)
)

 
 
(
s
t
a
g
e
n
e
t
)
:
 
M
o
d
u
l
e
D
i
c
t
(

 
 
 
 
(
i
c
d
_
c
o
d
e
s
)
:
 
S
t
a
g
e
N
e
t
L
a
y
e
r
(

 
 
 
 
 
 
(
k
e
r
n
e
l
)
:
 
L
i
n
e
a
r
(
i
n
_
f
e
a
t
u
r
e
s
=
1
2
9
,
 
o
u
t
_
f
e
a
t
u
r
e
s
=
1
5
4
2
,
 
b
i
a
s
=
T
r
u
e
)

 
 
 
 
 
 
(
r
e
c
u
r
r
e
n
t
_
k
e
r
n
e
l
)
:
 
L
i
n
e
a
r
(
i
n
_
f
e
a
t
u
r
e
s
=
3
8
5
,
 
o
u
t
_
f
e
a
t
u
r
e
s
=
1
5
4
2
,
 
b
i
a
s
=
T
r
u
e
)

 
 
 
 
 
 
(
n
n
_
s
c
a
l
e
)
:
 
L
i
n
e
a
r
(
i
n
_
f
e
a
t
u
r
e
s
=
3
8
4
,
 
o
u
t
_
f
e
a
t
u
r
e
s
=
6
4
,

E
p
o
c
h
 
0
 
/
 
1
:
 
1
0
0
%
|
█
█
█
█
█
█
█
█
█
█
|
 
2
/
2
 
[
0
0
:
0
1
<
0
0
:
0
0
,
 
 
1
.
2
9
i
t
/
s
]

-
-
-
 
T
r
a
i
n
 
e
p
o
c
h
-
0
,
 
s
t
e
p
-
2
 
-
-
-

l
o
s
s
:
 
0
.
6
8
1
4




E
v
a
l
u
a
t
i
o
n
:
 
1
0
0
%
|
█
█
█
█
█
█
█
█
█
█
|
 
1
/
1
 
[
0
0
:
0
0
<
0
0
:
0
0
,
 
 
4
.
4
4
i
t
/
s
]

-
-
-
 
E
v
a
l
 
e
p
o
c
h
-
0
,
 
s
t
e
p
-
2
 
-
-
-

p
r
_
a
u
c
:
 
0
.
1
8
1
2

r
o
c
_
a
u
c
:
 
0
.
7
2
5
0

a
c
c
u
r
a
c
y
:
 
0
.
8
6
3
6

f
1
:
 
0
.
0
0
0
0

l
o
s
s
:
 
0
.
6
4
9
4

N
e
w
 
b
e
s
t
 
r
o
c
_
a
u
c
 
s
c
o
r
e
 
(
0
.
7
2
5
0
)
 
a
t
 
e
p
o
c
h
-
0
,
 
s
t
e
p
-
2



L
o
a
d
e
d
 
b
e
s
t
 
m
o
d
e
l



E
v
a
l
u
a
t
i
o
n
:
 
1
0
0
%
|
█
█
█
█
█
█
█
█
█
█
|
 
1
/
1
 
[
0
0
:
0
0
<
0
0
:
0
0
,
 
 
4
.
6
6
i
t
/
s
]




T
e
s
t
 
R
e
s
u
l
t
s
:

 
 
p
r
_
a
u
c
:
 
0
.
2
7
3
5

 
 
r
o
c
_
a
u
c
:
 
0
.
4
2
1
1

 
 
a
c
c
u
r
a
c
y
:
 
0
.
7
1
1
1

 
 
f
1
:
 
0
.
1
3
3
3

 
 
l
o
s
s
:
 
0
.
6
7
0
1


S
a
m
p
l
e
 
p
r
e
d
i
c
t
i
o
n
s
:

 
 
P
r
e
d
i
c
t
e
d
 
p
r
o
b
a
b
i
l
i
t
i
e
s
:
 
t
e
n
s
o
r
(
[
[
0
.
5
1
6
3
]
,

 
 
 
 
 
 
 
 
[
0
.
4
7
9
9
]
,

 
 
 
 
 
 
 
 
[
0
.
4
7
9
6
]
,

 
 
 
 
 
 
 
 
[
0
.
4
5
9
1
]
,

 
 
 
 
 
 
 
 
[
0
.
5
0
4
6
]
]
,
 
d
e
v
i
c
e
=
'
c
u
d
a
:
4
'
)

 
 
T
r
u
e
 
l
a
b
e
l
s
:
 
t
e
n
s
o
r
(
[
[
0
.
]
,

 
 
 
 
 
 
 
 
[
0
.
]
,

 
 
 
 
 
 
 
 
[
0
.
]
,

 
 
 
 
 
 
 
 
[
0
.
]
,

 
 
 
 
 
 
 
 
[
0
.
]
]
,
 
d
e
v
i
c
e
=
'
c
u
d
a
:
4
'
)



#
#
 
I
n
f
e
r
e
n
c
e
 
O
n
 
a
 
H
o
l
d
o
u
t
 
S
e
t
 
E
x
a
m
p
l
e

B
e
l
o
w
,
 
w
e
'
l
l
 
g
e
n
e
r
a
t
e
 
s
o
m
e
 
p
s
e
u
d
o
 
s
a
m
p
l
e
s
 
w
i
t
h
 
a
 
b
u
n
c
h
 
o
f
 
u
n
k
n
o
w
n
 
t
o
k
e
n
s
 
a
n
d
 
v
i
s
i
t
 
l
e
n
g
t
h
s
 
b
e
y
o
n
d
 
w
h
a
t
'
s
 
o
b
s
e
r
v
e
d
 
i
n
 
t
h
e
 
t
r
a
i
n
i
n
g
 
s
e
t
.

In [8]:
f
r
o
m
 
p
y
h
e
a
l
t
h
.
d
a
t
a
s
e
t
s
.
b
a
s
e
_
d
a
t
a
s
e
t
 
i
m
p
o
r
t
 
S
a
m
p
l
e
D
a
t
a
s
e
t

i
m
p
o
r
t
 
r
a
n
d
o
m

i
m
p
o
r
t
 
n
u
m
p
y
 
a
s
 
n
p


d
e
f
 
g
e
n
e
r
a
t
e
_
h
o
l
d
o
u
t
_
s
e
t
(

 
 
 
 
s
a
m
p
l
e
_
d
a
t
a
s
e
t
:
 
S
a
m
p
l
e
D
a
t
a
s
e
t
,
 
n
u
m
_
s
a
m
p
l
e
s
:
 
i
n
t
 
=
 
1
0
,
 
s
e
e
d
:
 
i
n
t
 
=
 
4
2

)
 
-
>
 
S
a
m
p
l
e
D
a
t
a
s
e
t
:

 
 
 
 
"
"
"
G
e
n
e
r
a
t
e
 
s
y
n
t
h
e
t
i
c
 
h
o
l
d
-
o
u
t
 
s
e
t
 
w
i
t
h
 
u
n
s
e
e
n
 
c
o
d
e
s
 
a
n
d
 
v
a
r
y
i
n
g
 
l
e
n
g
t
h
s
.


 
 
 
 
T
h
i
s
 
f
u
n
c
t
i
o
n
 
c
r
e
a
t
e
s
 
s
y
n
t
h
e
t
i
c
 
s
a
m
p
l
e
s
 
t
o
 
t
e
s
t
 
t
h
e
 
p
r
o
c
e
s
s
o
r
'
s
 
a
b
i
l
i
t
y
 
t
o
:

 
 
 
 
1
.
 
H
a
n
d
l
e
 
c
o
m
p
l
e
t
e
l
y
 
u
n
s
e
e
n
 
t
o
k
e
n
s
 
(
m
a
p
p
e
d
 
t
o
 
<
u
n
k
>
)

 
 
 
 
2
.
 
H
a
n
d
l
e
 
s
e
q
u
e
n
c
e
 
l
e
n
g
t
h
s
 
l
a
r
g
e
r
 
t
h
a
n
 
t
r
a
i
n
i
n
g
 
b
u
t
 
w
i
t
h
i
n
 
p
a
d
d
i
n
g


 
 
 
 
A
r
g
s
:

 
 
 
 
 
 
 
 
s
a
m
p
l
e
_
d
a
t
a
s
e
t
:
 
O
r
i
g
i
n
a
l
 
S
a
m
p
l
e
D
a
t
a
s
e
t
 
w
i
t
h
 
f
i
t
t
e
d
 
p
r
o
c
e
s
s
o
r
s

 
 
 
 
 
 
 
 
n
u
m
_
s
a
m
p
l
e
s
:
 
N
u
m
b
e
r
 
o
f
 
s
y
n
t
h
e
t
i
c
 
s
a
m
p
l
e
s
 
t
o
 
g
e
n
e
r
a
t
e

 
 
 
 
 
 
 
 
s
e
e
d
:
 
R
a
n
d
o
m
 
s
e
e
d
 
f
o
r
 
r
e
p
r
o
d
u
c
i
b
i
l
i
t
y


 
 
 
 
R
e
t
u
r
n
s
:

 
 
 
 
 
 
 
 
S
a
m
p
l
e
D
a
t
a
s
e
t
 
w
i
t
h
 
s
y
n
t
h
e
t
i
c
 
s
a
m
p
l
e
s
 
u
s
i
n
g
 
f
i
t
t
e
d
 
p
r
o
c
e
s
s
o
r
s

 
 
 
 
"
"
"

 
 
 
 
r
a
n
d
o
m
.
s
e
e
d
(
s
e
e
d
)

 
 
 
 
n
p
.
r
a
n
d
o
m
.
s
e
e
d
(
s
e
e
d
)


 
 
 
 
#
 
G
e
t
 
t
h
e
 
f
i
t
t
e
d
 
p
r
o
c
e
s
s
o
r
s

 
 
 
 
i
c
d
_
p
r
o
c
e
s
s
o
r
 
=
 
s
a
m
p
l
e
_
d
a
t
a
s
e
t
.
i
n
p
u
t
_
p
r
o
c
e
s
s
o
r
s
[
"
i
c
d
_
c
o
d
e
s
"
]


 
 
 
 
#
 
G
e
t
 
m
a
x
 
n
e
s
t
e
d
 
l
e
n
g
t
h
 
f
r
o
m
 
I
C
D
 
p
r
o
c
e
s
s
o
r

 
 
 
 
m
a
x
_
i
c
d
_
l
e
n
 
=
 
i
c
d
_
p
r
o
c
e
s
s
o
r
.
_
m
a
x
_
n
e
s
t
e
d
_
l
e
n

 
 
 
 
#
 
H
a
n
d
l
e
 
b
o
t
h
 
o
l
d
 
a
n
d
 
n
e
w
 
p
r
o
c
e
s
s
o
r
 
v
e
r
s
i
o
n
s

 
 
 
 
p
a
d
d
i
n
g
 
=
 
g
e
t
a
t
t
r
(
i
c
d
_
p
r
o
c
e
s
s
o
r
,
 
"
_
p
a
d
d
i
n
g
"
,
 
0
)


 
 
 
 
p
r
i
n
t
(
"
\
n
=
=
=
 
H
o
l
d
-
o
u
t
 
S
e
t
 
G
e
n
e
r
a
t
i
o
n
 
=
=
=
"
)

 
 
 
 
p
r
i
n
t
(
f
"
P
r
o
c
e
s
s
o
r
 
a
t
t
r
i
b
u
t
e
s
:
 
{
d
i
r
(
i
c
d
_
p
r
o
c
e
s
s
o
r
)
}
"
)

 
 
 
 
p
r
i
n
t
(
f
"
H
a
s
 
_
p
a
d
d
i
n
g
 
a
t
t
r
i
b
u
t
e
:
 
{
h
a
s
a
t
t
r
(
i
c
d
_
p
r
o
c
e
s
s
o
r
,
 
'
_
p
a
d
d
i
n
g
'
)
}
"
)

 
 
 
 
p
r
i
n
t
(
f
"
I
C
D
 
m
a
x
 
n
e
s
t
e
d
 
l
e
n
g
t
h
:
 
{
m
a
x
_
i
c
d
_
l
e
n
}
"
)

 
 
 
 
p
r
i
n
t
(
f
"
P
a
d
d
i
n
g
 
(
v
i
a
 
g
e
t
a
t
t
r
)
:
 
{
p
a
d
d
i
n
g
}
"
)

 
 
 
 
i
f
 
h
a
s
a
t
t
r
(
i
c
d
_
p
r
o
c
e
s
s
o
r
,
 
"
_
p
a
d
d
i
n
g
"
)
:

 
 
 
 
 
 
 
 
p
r
i
n
t
(
f
"
P
a
d
d
i
n
g
 
(
d
i
r
e
c
t
 
a
c
c
e
s
s
)
:
 
{
i
c
d
_
p
r
o
c
e
s
s
o
r
.
_
p
a
d
d
i
n
g
}
"
)

 
 
 
 
p
r
i
n
t
(
f
"
O
b
s
e
r
v
e
d
 
m
a
x
 
(
w
i
t
h
o
u
t
 
p
a
d
d
i
n
g
)
:
 
{
m
a
x
_
i
c
d
_
l
e
n
 
-
 
p
a
d
d
i
n
g
}
"
)


 
 
 
 
s
y
n
t
h
e
t
i
c
_
s
a
m
p
l
e
s
 
=
 
[
]


 
 
 
 
f
o
r
 
i
 
i
n
 
r
a
n
g
e
(
n
u
m
_
s
a
m
p
l
e
s
)
:

 
 
 
 
 
 
 
 
#
 
G
e
n
e
r
a
t
e
 
r
a
n
d
o
m
 
n
u
m
b
e
r
 
o
f
 
v
i
s
i
t
s
 
(
1
-
5
)

 
 
 
 
 
 
 
 
n
u
m
_
v
i
s
i
t
s
 
=
 
r
a
n
d
o
m
.
r
a
n
d
i
n
t
(
1
,
 
5
)


 
 
 
 
 
 
 
 
#
 
G
e
n
e
r
a
t
e
 
I
C
D
 
c
o
d
e
s
 
w
i
t
h
 
u
n
s
e
e
n
 
t
o
k
e
n
s

 
 
 
 
 
 
 
 
i
c
d
_
c
o
d
e
s
_
l
i
s
t
 
=
 
[
]

 
 
 
 
 
 
 
 
i
c
d
_
t
i
m
e
s
_
l
i
s
t
 
=
 
[
]


 
 
 
 
 
 
 
 
f
o
r
 
v
i
s
i
t
_
i
d
x
 
i
n
 
r
a
n
g
e
(
n
u
m
_
v
i
s
i
t
s
)
:

 
 
 
 
 
 
 
 
 
 
 
 
#
 
G
e
n
e
r
a
t
e
 
s
e
q
u
e
n
c
e
 
l
e
n
g
t
h
 
b
e
t
w
e
e
n
 
o
b
s
e
r
v
e
d
_
m
a
x
 
a
n
d
 
m
a
x
_
i
c
d
_
l
e
n

 
 
 
 
 
 
 
 
 
 
 
 
#
 
T
h
i
s
 
t
e
s
t
s
 
t
h
e
 
p
a
d
d
i
n
g
 
c
a
p
a
c
i
t
y

 
 
 
 
 
 
 
 
 
 
 
 
o
b
s
e
r
v
e
d
_
m
a
x
 
=
 
m
a
x
_
i
c
d
_
l
e
n
 
-
 
p
a
d
d
i
n
g

 
 
 
 
 
 
 
 
 
 
 
 
s
e
q
_
l
e
n
 
=
 
r
a
n
d
o
m
.
r
a
n
d
i
n
t
(
m
a
x
(
1
,
 
o
b
s
e
r
v
e
d
_
m
a
x
 
-
 
2
)
,
 
m
a
x
_
i
c
d
_
l
e
n
 
-
 
1
)


 
 
 
 
 
 
 
 
 
 
 
 
#
 
G
e
n
e
r
a
t
e
 
u
n
s
e
e
n
 
c
o
d
e
s

 
 
 
 
 
 
 
 
 
 
 
 
v
i
s
i
t
_
c
o
d
e
s
 
=
 
[
f
"
N
E
W
C
O
D
E
_
{
i
}
_
{
v
i
s
i
t
_
i
d
x
}
_
{
j
}
"
 
f
o
r
 
j
 
i
n
 
r
a
n
g
e
(
s
e
q
_
l
e
n
)
]

 
 
 
 
 
 
 
 
 
 
 
 
i
c
d
_
c
o
d
e
s
_
l
i
s
t
.
a
p
p
e
n
d
(
v
i
s
i
t
_
c
o
d
e
s
)


 
 
 
 
 
 
 
 
 
 
 
 
#
 
G
e
n
e
r
a
t
e
 
t
i
m
e
 
i
n
t
e
r
v
a
l
s
 
(
h
o
u
r
s
 
f
r
o
m
 
p
r
e
v
i
o
u
s
 
v
i
s
i
t
)

 
 
 
 
 
 
 
 
 
 
 
 
i
f
 
v
i
s
i
t
_
i
d
x
 
=
=
 
0
:

 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
i
c
d
_
t
i
m
e
s
_
l
i
s
t
.
a
p
p
e
n
d
(
0
.
0
)

 
 
 
 
 
 
 
 
 
 
 
 
e
l
s
e
:

 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
i
c
d
_
t
i
m
e
s
_
l
i
s
t
.
a
p
p
e
n
d
(
r
a
n
d
o
m
.
u
n
i
f
o
r
m
(
2
4
.
0
,
 
7
2
0
.
0
)
)


 
 
 
 
 
 
 
 
#
 
G
e
n
e
r
a
t
e
 
l
a
b
 
d
a
t
a
 
(
1
0
-
d
i
m
e
n
s
i
o
n
a
l
 
v
e
c
t
o
r
s
)

 
 
 
 
 
 
 
 
n
u
m
_
l
a
b
_
t
i
m
e
s
t
a
m
p
s
 
=
 
r
a
n
d
o
m
.
r
a
n
d
i
n
t
(
5
,
 
1
5
)

 
 
 
 
 
 
 
 
l
a
b
_
v
a
l
u
e
s
_
l
i
s
t
 
=
 
[
]

 
 
 
 
 
 
 
 
l
a
b
_
t
i
m
e
s
_
l
i
s
t
 
=
 
[
]


 
 
 
 
 
 
 
 
f
o
r
 
t
s
_
i
d
x
 
i
n
 
r
a
n
g
e
(
n
u
m
_
l
a
b
_
t
i
m
e
s
t
a
m
p
s
)
:

 
 
 
 
 
 
 
 
 
 
 
 
#
 
G
e
n
e
r
a
t
e
 
1
0
D
 
v
e
c
t
o
r
 
w
i
t
h
 
s
o
m
e
 
r
a
n
d
o
m
 
v
a
l
u
e
s
 
a
n
d
 
s
o
m
e
 
N
o
n
e

 
 
 
 
 
 
 
 
 
 
 
 
l
a
b
_
v
e
c
t
o
r
 
=
 
[
]

 
 
 
 
 
 
 
 
 
 
 
 
f
o
r
 
d
i
m
 
i
n
 
r
a
n
g
e
(
1
0
)
:

 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
i
f
 
r
a
n
d
o
m
.
r
a
n
d
o
m
(
)
 
<
 
0
.
8
:
 
 
#
 
8
0
%
 
c
h
a
n
c
e
 
o
f
 
v
a
l
u
e

 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
l
a
b
_
v
e
c
t
o
r
.
a
p
p
e
n
d
(
r
a
n
d
o
m
.
u
n
i
f
o
r
m
(
5
0
.
0
,
 
1
5
0
.
0
)
)

 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
e
l
s
e
:

 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
l
a
b
_
v
e
c
t
o
r
.
a
p
p
e
n
d
(
N
o
n
e
)


 
 
 
 
 
 
 
 
 
 
 
 
l
a
b
_
v
a
l
u
e
s
_
l
i
s
t
.
a
p
p
e
n
d
(
l
a
b
_
v
e
c
t
o
r
)

 
 
 
 
 
 
 
 
 
 
 
 
l
a
b
_
t
i
m
e
s
_
l
i
s
t
.
a
p
p
e
n
d
(
r
a
n
d
o
m
.
u
n
i
f
o
r
m
(
0
.
0
,
 
4
8
.
0
)
)


 
 
 
 
 
 
 
 
#
 
C
r
e
a
t
e
 
s
a
m
p
l
e
 
i
n
 
t
h
e
 
e
x
p
e
c
t
e
d
 
f
o
r
m
a
t
 
(
b
e
f
o
r
e
 
p
r
o
c
e
s
s
i
n
g
)

 
 
 
 
 
 
 
 
s
y
n
t
h
e
t
i
c
_
s
a
m
p
l
e
 
=
 
{

 
 
 
 
 
 
 
 
 
 
 
 
"
p
a
t
i
e
n
t
_
i
d
"
:
 
f
"
H
O
L
D
O
U
T
_
P
A
T
I
E
N
T
_
{
i
}
"
,

 
 
 
 
 
 
 
 
 
 
 
 
"
i
c
d
_
c
o
d
e
s
"
:
 
(
i
c
d
_
t
i
m
e
s
_
l
i
s
t
,
 
i
c
d
_
c
o
d
e
s
_
l
i
s
t
)
,

 
 
 
 
 
 
 
 
 
 
 
 
"
l
a
b
s
"
:
 
(
l
a
b
_
t
i
m
e
s
_
l
i
s
t
,
 
l
a
b
_
v
a
l
u
e
s
_
l
i
s
t
)
,

 
 
 
 
 
 
 
 
 
 
 
 
"
m
o
r
t
a
l
i
t
y
"
:
 
r
a
n
d
o
m
.
r
a
n
d
i
n
t
(
0
,
 
1
)
,

 
 
 
 
 
 
 
 
}


 
 
 
 
 
 
 
 
s
y
n
t
h
e
t
i
c
_
s
a
m
p
l
e
s
.
a
p
p
e
n
d
(
s
y
n
t
h
e
t
i
c
_
s
a
m
p
l
e
)


 
 
 
 
#
 
C
r
e
a
t
e
 
a
 
n
e
w
 
S
a
m
p
l
e
D
a
t
a
s
e
t
 
w
i
t
h
 
t
h
e
 
F
I
T
T
E
D
 
p
r
o
c
e
s
s
o
r
s

 
 
 
 
h
o
l
d
o
u
t
_
d
a
t
a
s
e
t
 
=
 
S
a
m
p
l
e
D
a
t
a
s
e
t
(

 
 
 
 
 
 
 
 
s
a
m
p
l
e
s
=
s
y
n
t
h
e
t
i
c
_
s
a
m
p
l
e
s
,

 
 
 
 
 
 
 
 
i
n
p
u
t
_
s
c
h
e
m
a
=
s
a
m
p
l
e
_
d
a
t
a
s
e
t
.
i
n
p
u
t
_
s
c
h
e
m
a
,

 
 
 
 
 
 
 
 
o
u
t
p
u
t
_
s
c
h
e
m
a
=
s
a
m
p
l
e
_
d
a
t
a
s
e
t
.
o
u
t
p
u
t
_
s
c
h
e
m
a
,

 
 
 
 
 
 
 
 
d
a
t
a
s
e
t
_
n
a
m
e
=
f
"
{
s
a
m
p
l
e
_
d
a
t
a
s
e
t
.
d
a
t
a
s
e
t
_
n
a
m
e
}
_
h
o
l
d
o
u
t
"
,

 
 
 
 
 
 
 
 
t
a
s
k
_
n
a
m
e
=
s
a
m
p
l
e
_
d
a
t
a
s
e
t
.
t
a
s
k
_
n
a
m
e
,

 
 
 
 
 
 
 
 
i
n
p
u
t
_
p
r
o
c
e
s
s
o
r
s
=
s
a
m
p
l
e
_
d
a
t
a
s
e
t
.
i
n
p
u
t
_
p
r
o
c
e
s
s
o
r
s
,

 
 
 
 
 
 
 
 
o
u
t
p
u
t
_
p
r
o
c
e
s
s
o
r
s
=
s
a
m
p
l
e
_
d
a
t
a
s
e
t
.
o
u
t
p
u
t
_
p
r
o
c
e
s
s
o
r
s
,

 
 
 
 
)


 
 
 
 
p
r
i
n
t
(
f
"
G
e
n
e
r
a
t
e
d
 
{
l
e
n
(
h
o
l
d
o
u
t
_
d
a
t
a
s
e
t
)
}
 
s
y
n
t
h
e
t
i
c
 
s
a
m
p
l
e
s
"
)

 
 
 
 
s
a
m
p
l
e
_
s
e
q
_
l
e
n
s
 
=
 
[
l
e
n
(
s
[
"
i
c
d
_
c
o
d
e
s
"
]
[
1
]
)
 
f
o
r
 
s
 
i
n
 
s
y
n
t
h
e
t
i
c
_
s
a
m
p
l
e
s
[
:
3
]
]

 
 
 
 
p
r
i
n
t
(
f
"
S
a
m
p
l
e
 
I
C
D
 
s
e
q
u
e
n
c
e
 
l
e
n
g
t
h
s
:
 
{
s
a
m
p
l
e
_
s
e
q
_
l
e
n
s
}
"
)

 
 
 
 
s
a
m
p
l
e
_
c
o
d
e
s
_
p
e
r
_
v
i
s
i
t
 
=
 
[

 
 
 
 
 
 
 
 
[
l
e
n
(
v
i
s
i
t
)
 
f
o
r
 
v
i
s
i
t
 
i
n
 
s
[
"
i
c
d
_
c
o
d
e
s
"
]
[
1
]
]
 
f
o
r
 
s
 
i
n
 
s
y
n
t
h
e
t
i
c
_
s
a
m
p
l
e
s
[
:
3
]

 
 
 
 
]

 
 
 
 
p
r
i
n
t
(
f
"
S
a
m
p
l
e
 
c
o
d
e
s
 
p
e
r
 
v
i
s
i
t
:
 
{
s
a
m
p
l
e
_
c
o
d
e
s
_
p
e
r
_
v
i
s
i
t
}
"
)


 
 
 
 
r
e
t
u
r
n
 
h
o
l
d
o
u
t
_
d
a
t
a
s
e
t




In [9]:
h
o
l
d
o
u
t
_
d
a
t
a
s
e
t
 
=
 
g
e
n
e
r
a
t
e
_
h
o
l
d
o
u
t
_
s
e
t
(
s
a
m
p
l
e
_
d
a
t
a
s
e
t
,
 
n
u
m
_
s
a
m
p
l
e
s
=
1
0
,
 
s
e
e
d
=
4
2
)

#
 
C
r
e
a
t
e
 
d
a
t
a
l
o
a
d
e
r
 
f
o
r
 
h
o
l
d
-
o
u
t
 
s
e
t

h
o
l
d
o
u
t
_
l
o
a
d
e
r
 
=
 
g
e
t
_
d
a
t
a
l
o
a
d
e
r
(
h
o
l
d
o
u
t
_
d
a
t
a
s
e
t
,
 
b
a
t
c
h
_
s
i
z
e
=
1
6
,
 
s
h
u
f
f
l
e
=
F
a
l
s
e
)

#
 
I
n
s
p
e
c
t
 
p
r
o
c
e
s
s
e
d
 
s
a
m
p
l
e
s

p
r
i
n
t
(
"
\
n
=
=
=
 
I
n
s
p
e
c
t
i
n
g
 
P
r
o
c
e
s
s
e
d
 
H
o
l
d
-
o
u
t
 
S
a
m
p
l
e
s
 
=
=
=
"
)

h
o
l
d
o
u
t
_
b
a
t
c
h
 
=
 
n
e
x
t
(
i
t
e
r
(
h
o
l
d
o
u
t
_
l
o
a
d
e
r
)
)

p
r
i
n
t
(
"
F
e
a
t
u
r
e
 
T
e
n
s
o
r
 
D
i
m
e
n
s
i
o
n
s
"
)

p
r
i
n
t
(
h
o
l
d
o
u
t
_
b
a
t
c
h
[
"
i
c
d
_
c
o
d
e
s
"
]
[
1
]
.
s
h
a
p
e
)

w
i
t
h
 
t
o
r
c
h
.
n
o
_
g
r
a
d
(
)
:

 
 
 
 
h
o
l
d
o
u
t
_
o
u
t
p
u
t
 
=
 
m
o
d
e
l
(
*
*
h
o
l
d
o
u
t
_
b
a
t
c
h
)




=
=
=
 
H
o
l
d
-
o
u
t
 
S
e
t
 
G
e
n
e
r
a
t
i
o
n
 
=
=
=

P
r
o
c
e
s
s
o
r
 
a
t
t
r
i
b
u
t
e
s
:
 
[
'
_
_
a
b
s
t
r
a
c
t
m
e
t
h
o
d
s
_
_
'
,
 
'
_
_
c
l
a
s
s
_
_
'
,
 
'
_
_
d
e
l
a
t
t
r
_
_
'
,
 
'
_
_
d
i
c
t
_
_
'
,
 
'
_
_
d
i
r
_
_
'
,
 
'
_
_
d
o
c
_
_
'
,
 
'
_
_
e
q
_
_
'
,
 
'
_
_
f
o
r
m
a
t
_
_
'
,
 
'
_
_
g
e
_
_
'
,
 
'
_
_
g
e
t
a
t
t
r
i
b
u
t
e
_
_
'
,
 
'
_
_
g
t
_
_
'
,
 
'
_
_
h
a
s
h
_
_
'
,
 
'
_
_
i
n
i
t
_
_
'
,
 
'
_
_
i
n
i
t
_
s
u
b
c
l
a
s
s
_
_
'
,
 
'
_
_
l
e
_
_
'
,
 
'
_
_
l
t
_
_
'
,
 
'
_
_
m
o
d
u
l
e
_
_
'
,
 
'
_
_
n
e
_
_
'
,
 
'
_
_
n
e
w
_
_
'
,
 
'
_
_
r
e
d
u
c
e
_
_
'
,
 
'
_
_
r
e
d
u
c
e
_
e
x
_
_
'
,
 
'
_
_
r
e
p
r
_
_
'
,
 
'
_
_
s
e
t
a
t
t
r
_
_
'
,
 
'
_
_
s
i
z
e
o
f
_
_
'
,
 
'
_
_
s
l
o
t
s
_
_
'
,
 
'
_
_
s
t
r
_
_
'
,
 
'
_
_
s
u
b
c
l
a
s
s
h
o
o
k
_
_
'
,
 
'
_
_
w
e
a
k
r
e
f
_
_
'
,
 
'
_
a
b
c
_
i
m
p
l
'
,
 
'
_
e
n
c
o
d
e
_
c
o
d
e
s
'
,
 
'
_
e
n
c
o
d
e
_
n
e
s
t
e
d
_
c
o
d
e
s
'
,
 
'
_
i
s
_
n
e
s
t
e
d


P
r
o
c
e
s
s
i
n
g
 
s
a
m
p
l
e
s
:
 
1
0
0
%
|
█
█
█
█
█
█
█
█
█
█
|
 
1
0
/
1
0
 
[
0
0
:
0
0
<
0
0
:
0
0
,
 
4
4
2
0
.
1
7
i
t
/
s
]

G
e
n
e
r
a
t
e
d
 
1
0
 
s
y
n
t
h
e
t
i
c
 
s
a
m
p
l
e
s

S
a
m
p
l
e
 
I
C
D
 
s
e
q
u
e
n
c
e
 
l
e
n
g
t
h
s
:
 
[
1
,
 
2
,
 
2
]

S
a
m
p
l
e
 
c
o
d
e
s
 
p
e
r
 
v
i
s
i
t
:
 
[
[
7
0
]
,
 
[
7
0
,
 
7
0
]
,
 
[
7
0
,
 
7
0
]
]


=
=
=
 
I
n
s
p
e
c
t
i
n
g
 
P
r
o
c
e
s
s
e
d
 
H
o
l
d
-
o
u
t
 
S
a
m
p
l
e
s
 
=
=
=

F
e
a
t
u
r
e
 
T
e
n
s
o
r
 
D
i
m
e
n
s
i
o
n
s

t
o
r
c
h
.
S
i
z
e
(
[
1
0
,
 
5
,
 
7
0
]
)



#
#
 
P
o
s
t
-
h
o
c
 
M
L
 
p
r
o
c
e
s
s
i
n
g
 
(
I
n
t
e
r
p
r
e
t
a
b
i
l
i
t
y
)

W
e
 
n
o
t
e
 
t
h
a
t
 
o
n
c
e
 
t
h
e
 
m
o
d
e
l
'
s
 
t
r
a
i
n
e
d
 
a
n
d
 
e
v
a
l
u
a
t
i
o
n
 
m
e
t
r
i
c
s
 
a
r
e
 
d
e
r
i
v
e
d
.
 
P
e
o
p
l
e
 
m
a
y
 
b
e
 
i
n
t
e
r
e
s
t
e
d
 
i
n
 
t
h
i
n
g
s
 
l
i
k
e
 
p
o
s
t
-
h
o
c
 
i
n
t
e
r
p
r
e
t
a
b
i
l
i
t
y
 
o
r
 
u
n
c
e
r
t
a
i
n
t
y
 
q
u
a
n
t
i
f
i
c
a
t
i
o
n
.


W
e
 
n
o
t
e
 
t
h
a
t
 
t
h
i
s
 
i
s
 
q
u
i
t
e
 
a
 
w
o
r
k
-
i
n
-
p
r
o
g
r
e
s
s
 
f
o
r
 
P
y
H
e
a
l
t
h
 
2
.
0
,
 
b
u
t
 
t
h
e
 
r
o
a
d
m
a
p
 
i
n
c
l
u
d
e
s
 
t
h
e
 
f
o
l
l
o
w
i
n
g
:


-
 
I
n
t
e
g
r
a
t
e
d
 
G
r
a
d
i
e
n
t
s
 
(
d
e
e
p
 
N
N
-
b
a
s
e
d
 
i
n
t
e
r
p
r
e
t
a
b
i
l
i
t
y
)

-
 
C
o
n
f
o
r
m
a
l
 
P
r
e
d
i
c
t
i
o
n
:
 
W
e
 
d
o
 
h
a
v
e
 
m
a
n
y
 
o
t
h
e
r
 
U
Q
 
t
e
c
h
n
i
q
u
e
s
 
[
h
e
r
e
]
(
h
t
t
p
s
:
/
/
p
y
h
e
a
l
t
h
.
r
e
a
d
t
h
e
d
o
c
s
.
i
o
/
e
n
/
l
a
t
e
s
t
/
a
p
i
/
c
a
l
i
b
.
h
t
m
l
)



In [12]:
f
r
o
m
 
p
y
h
e
a
l
t
h
.
m
e
d
c
o
d
e
 
i
m
p
o
r
t
 
C
r
o
s
s
M
a
p
,
 
I
n
n
e
r
M
a
p


L
A
B
_
C
A
T
E
G
O
R
Y
_
N
A
M
E
S
 
=
 
M
o
r
t
a
l
i
t
y
P
r
e
d
i
c
t
i
o
n
S
t
a
g
e
N
e
t
M
I
M
I
C
4
.
L
A
B
_
C
A
T
E
G
O
R
Y
_
N
A
M
E
S


d
e
f
 
u
n
r
a
v
e
l
(
f
l
a
t
_
i
n
d
e
x
:
 
i
n
t
,
 
s
h
a
p
e
:
 
t
o
r
c
h
.
S
i
z
e
)
:

 
 
 
 
c
o
o
r
d
s
 
=
 
[
]

 
 
 
 
r
e
m
a
i
n
i
n
g
 
=
 
f
l
a
t
_
i
n
d
e
x

 
 
 
 
f
o
r
 
d
i
m
 
i
n
 
r
e
v
e
r
s
e
d
(
s
h
a
p
e
)
:

 
 
 
 
 
 
 
 
c
o
o
r
d
s
.
a
p
p
e
n
d
(
r
e
m
a
i
n
i
n
g
 
%
 
d
i
m
)

 
 
 
 
 
 
 
 
r
e
m
a
i
n
i
n
g
 
/
/
=
 
d
i
m

 
 
 
 
r
e
t
u
r
n
 
l
i
s
t
(
r
e
v
e
r
s
e
d
(
c
o
o
r
d
s
)
)


d
e
f
 
d
e
c
o
d
e
_
t
o
k
e
n
(
i
d
x
:
 
i
n
t
,
 
p
r
o
c
e
s
s
o
r
,
 
f
e
a
t
u
r
e
_
k
e
y
:
 
s
t
r
)
:

 
 
 
 
i
c
d
1
0
c
m
 
=
 
I
n
n
e
r
M
a
p
.
l
o
a
d
(
"
I
C
D
1
0
C
M
"
)

 
 
 
 
i
c
d
1
0
p
c
 
=
 
I
n
n
e
r
M
a
p
.
l
o
a
d
(
"
I
C
D
1
0
P
R
O
C
"
)

 
 
 
 
i
f
 
p
r
o
c
e
s
s
o
r
 
i
s
 
N
o
n
e
 
o
r
 
n
o
t
 
h
a
s
a
t
t
r
(
p
r
o
c
e
s
s
o
r
,
 
"
c
o
d
e
_
v
o
c
a
b
"
)
:

 
 
 
 
 
 
 
 
r
e
t
u
r
n
 
s
t
r
(
i
d
x
)

 
 
 
 
r
e
v
e
r
s
e
_
v
o
c
a
b
 
=
 
{
i
n
d
e
x
:
 
t
o
k
e
n
 
f
o
r
 
t
o
k
e
n
,
 
i
n
d
e
x
 
i
n
 
p
r
o
c
e
s
s
o
r
.
c
o
d
e
_
v
o
c
a
b
.
i
t
e
m
s
(
)
}

 
 
 
 
t
o
k
e
n
 
=
 
r
e
v
e
r
s
e
_
v
o
c
a
b
.
g
e
t
(
i
d
x
,
 
f
"
<
U
N
K
:
{
i
d
x
}
>
"
)


 
 
 
 
i
f
 
f
e
a
t
u
r
e
_
k
e
y
 
=
=
 
"
i
c
d
_
c
o
d
e
s
"
 
a
n
d
 
t
o
k
e
n
 
n
o
t
 
i
n
 
{
"
<
u
n
k
>
"
,
 
"
<
p
a
d
>
"
}
:

 
 
 
 
 
 
 
 
d
e
s
c
 
=
 
N
o
n
e

 
 
 
 
 
 
 
 
i
f
 
t
o
k
e
n
 
i
n
 
i
c
d
1
0
c
m
:

 
 
 
 
 
 
 
 
 
 
 
 
d
e
s
c
 
=
 
i
c
d
1
0
c
m
.
l
o
o
k
u
p
(
t
o
k
e
n
)

 
 
 
 
 
 
 
 
e
l
i
f
 
t
o
k
e
n
 
i
n
 
i
c
d
1
0
p
c
:

 
 
 
 
 
 
 
 
 
 
 
 
d
e
s
c
 
=
 
i
c
d
1
0
p
c
.
l
o
o
k
u
p
(
t
o
k
e
n
)


 
 
 
 
 
 
 
 
i
f
 
d
e
s
c
:

 
 
 
 
 
 
 
 
 
 
 
 
r
e
t
u
r
n
 
f
"
{
t
o
k
e
n
}
:
 
{
d
e
s
c
}
"


 
 
 
 
r
e
t
u
r
n
 
t
o
k
e
n



d
e
f
 
p
r
i
n
t
_
t
o
p
_
a
t
t
r
i
b
u
t
i
o
n
s
(

 
 
 
 
a
t
t
r
i
b
u
t
i
o
n
s
,

 
 
 
 
b
a
t
c
h
,

 
 
 
 
p
r
o
c
e
s
s
o
r
s
,

 
 
 
 
t
o
p
_
k
:
 
i
n
t
 
=
 
1
0
,

)
:

 
 
 
 
f
o
r
 
f
e
a
t
u
r
e
_
k
e
y
,
 
a
t
t
r
 
i
n
 
a
t
t
r
i
b
u
t
i
o
n
s
.
i
t
e
m
s
(
)
:

 
 
 
 
 
 
 
 
a
t
t
r
_
c
p
u
 
=
 
a
t
t
r
.
d
e
t
a
c
h
(
)
.
c
p
u
(
)

 
 
 
 
 
 
 
 
i
f
 
a
t
t
r
_
c
p
u
.
d
i
m
(
)
 
=
=
 
0
 
o
r
 
a
t
t
r
_
c
p
u
.
s
i
z
e
(
0
)
 
=
=
 
0
:

 
 
 
 
 
 
 
 
 
 
 
 
c
o
n
t
i
n
u
e


 
 
 
 
 
 
 
 
f
e
a
t
u
r
e
_
i
n
p
u
t
 
=
 
b
a
t
c
h
[
f
e
a
t
u
r
e
_
k
e
y
]

 
 
 
 
 
 
 
 
i
f
 
i
s
i
n
s
t
a
n
c
e
(
f
e
a
t
u
r
e
_
i
n
p
u
t
,
 
t
u
p
l
e
)
:

 
 
 
 
 
 
 
 
 
 
 
 
f
e
a
t
u
r
e
_
i
n
p
u
t
 
=
 
f
e
a
t
u
r
e
_
i
n
p
u
t
[
1
]

 
 
 
 
 
 
 
 
f
e
a
t
u
r
e
_
i
n
p
u
t
 
=
 
f
e
a
t
u
r
e
_
i
n
p
u
t
.
d
e
t
a
c
h
(
)
.
c
p
u
(
)


 
 
 
 
 
 
 
 
f
l
a
t
t
e
n
e
d
 
=
 
a
t
t
r
_
c
p
u
[
0
]
.
f
l
a
t
t
e
n
(
)

 
 
 
 
 
 
 
 
i
f
 
f
l
a
t
t
e
n
e
d
.
n
u
m
e
l
(
)
 
=
=
 
0
:

 
 
 
 
 
 
 
 
 
 
 
 
c
o
n
t
i
n
u
e


 
 
 
 
 
 
 
 
p
r
i
n
t
(
f
"
\
n
F
e
a
t
u
r
e
:
 
{
f
e
a
t
u
r
e
_
k
e
y
}
"
)

 
 
 
 
 
 
 
 
k
 
=
 
m
i
n
(
t
o
p
_
k
,
 
f
l
a
t
t
e
n
e
d
.
n
u
m
e
l
(
)
)

 
 
 
 
 
 
 
 
t
o
p
_
v
a
l
u
e
s
,
 
t
o
p
_
i
n
d
i
c
e
s
 
=
 
t
o
r
c
h
.
t
o
p
k
(
f
l
a
t
t
e
n
e
d
.
a
b
s
(
)
,
 
k
=
k
)

 
 
 
 
 
 
 
 
p
r
o
c
e
s
s
o
r
 
=
 
p
r
o
c
e
s
s
o
r
s
.
g
e
t
(
f
e
a
t
u
r
e
_
k
e
y
)
 
i
f
 
p
r
o
c
e
s
s
o
r
s
 
e
l
s
e
 
N
o
n
e

 
 
 
 
 
 
 
 
i
s
_
c
o
n
t
i
n
u
o
u
s
 
=
 
t
o
r
c
h
.
i
s
_
f
l
o
a
t
i
n
g
_
p
o
i
n
t
(
f
e
a
t
u
r
e
_
i
n
p
u
t
)


 
 
 
 
 
 
 
 
f
o
r
 
r
a
n
k
,
 
(
_
,
 
f
l
a
t
_
i
d
x
)
 
i
n
 
e
n
u
m
e
r
a
t
e
(
z
i
p
(
t
o
p
_
v
a
l
u
e
s
,
 
t
o
p
_
i
n
d
i
c
e
s
)
,
 
1
)
:

 
 
 
 
 
 
 
 
 
 
 
 
a
t
t
r
i
b
u
t
i
o
n
_
v
a
l
u
e
 
=
 
f
l
a
t
t
e
n
e
d
[
f
l
a
t
_
i
d
x
]
.
i
t
e
m
(
)

 
 
 
 
 
 
 
 
 
 
 
 
c
o
o
r
d
s
 
=
 
u
n
r
a
v
e
l
(
f
l
a
t
_
i
d
x
.
i
t
e
m
(
)
,
 
a
t
t
r
_
c
p
u
[
0
]
.
s
h
a
p
e
)


 
 
 
 
 
 
 
 
 
 
 
 
i
f
 
i
s
_
c
o
n
t
i
n
u
o
u
s
:

 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
a
c
t
u
a
l
_
v
a
l
u
e
 
=
 
f
e
a
t
u
r
e
_
i
n
p
u
t
[
0
]
[
t
u
p
l
e
(
c
o
o
r
d
s
)
]
.
i
t
e
m
(
)

 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
l
a
b
e
l
 
=
 
"
"

 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
i
f
 
f
e
a
t
u
r
e
_
k
e
y
 
=
=
 
"
l
a
b
s
"
 
a
n
d
 
l
e
n
(
c
o
o
r
d
s
)
 
>
=
 
1
:

 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
l
a
b
_
i
d
x
 
=
 
c
o
o
r
d
s
[
-
1
]

 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
i
f
 
l
a
b
_
i
d
x
 
<
 
l
e
n
(
L
A
B
_
C
A
T
E
G
O
R
Y
_
N
A
M
E
S
)
:

 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
l
a
b
e
l
 
=
 
f
"
{
L
A
B
_
C
A
T
E
G
O
R
Y
_
N
A
M
E
S
[
l
a
b
_
i
d
x
]
}
 
"

 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
p
r
i
n
t
(

 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
f
"
 
 
{
r
a
n
k
:
2
d
}
.
 
i
d
x
=
{
c
o
o
r
d
s
}
 
{
l
a
b
e
l
}
v
a
l
u
e
=
{
a
c
t
u
a
l
_
v
a
l
u
e
:
.
4
f
}
 
"

 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
f
"
a
t
t
r
=
{
a
t
t
r
i
b
u
t
i
o
n
_
v
a
l
u
e
:
+
.
6
f
}
"

 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
)

 
 
 
 
 
 
 
 
 
 
 
 
e
l
s
e
:

 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
t
o
k
e
n
_
i
d
x
 
=
 
i
n
t
(
f
e
a
t
u
r
e
_
i
n
p
u
t
[
0
]
[
t
u
p
l
e
(
c
o
o
r
d
s
)
]
.
i
t
e
m
(
)
)

 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
t
o
k
e
n
 
=
 
d
e
c
o
d
e
_
t
o
k
e
n
(
t
o
k
e
n
_
i
d
x
,
 
p
r
o
c
e
s
s
o
r
,
 
f
e
a
t
u
r
e
_
k
e
y
)

 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
p
r
i
n
t
(

 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
f
"
 
 
{
r
a
n
k
:
2
d
}
.
 
i
d
x
=
{
c
o
o
r
d
s
}
 
t
o
k
e
n
=
'
{
t
o
k
e
n
}
'
 
"

 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
f
"
a
t
t
r
=
{
a
t
t
r
i
b
u
t
i
o
n
_
v
a
l
u
e
:
+
.
6
f
}
"

 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
)



In [13]:
f
r
o
m
 
p
y
h
e
a
l
t
h
.
i
n
t
e
r
p
r
e
t
.
m
e
t
h
o
d
s
 
i
m
p
o
r
t
 
D
e
e
p
L
i
f
t
,
 
I
n
t
e
g
r
a
t
e
d
G
r
a
d
i
e
n
t
s

d
e
f
 
m
o
v
e
_
b
a
t
c
h
_
t
o
_
d
e
v
i
c
e
(
b
a
t
c
h
,
 
t
a
r
g
e
t
_
d
e
v
i
c
e
)
:

 
 
 
 
m
o
v
e
d
 
=
 
{
}

 
 
 
 
f
o
r
 
k
e
y
,
 
v
a
l
u
e
 
i
n
 
b
a
t
c
h
.
i
t
e
m
s
(
)
:

 
 
 
 
 
 
 
 
i
f
 
i
s
i
n
s
t
a
n
c
e
(
v
a
l
u
e
,
 
t
o
r
c
h
.
T
e
n
s
o
r
)
:

 
 
 
 
 
 
 
 
 
 
 
 
m
o
v
e
d
[
k
e
y
]
 
=
 
v
a
l
u
e
.
t
o
(
t
a
r
g
e
t
_
d
e
v
i
c
e
)

 
 
 
 
 
 
 
 
e
l
i
f
 
i
s
i
n
s
t
a
n
c
e
(
v
a
l
u
e
,
 
t
u
p
l
e
)
:

 
 
 
 
 
 
 
 
 
 
 
 
m
o
v
e
d
[
k
e
y
]
 
=
 
t
u
p
l
e
(
v
.
t
o
(
t
a
r
g
e
t
_
d
e
v
i
c
e
)
 
f
o
r
 
v
 
i
n
 
v
a
l
u
e
)

 
 
 
 
 
 
 
 
e
l
s
e
:

 
 
 
 
 
 
 
 
 
 
 
 
m
o
v
e
d
[
k
e
y
]
 
=
 
v
a
l
u
e

 
 
 
 
r
e
t
u
r
n
 
m
o
v
e
d


d
e
v
i
c
e
 
=
 
t
o
r
c
h
.
d
e
v
i
c
e
(
"
c
p
u
"
)

m
o
d
e
l
.
t
o
(
d
e
v
i
c
e
)

i
g
 
=
 
I
n
t
e
g
r
a
t
e
d
G
r
a
d
i
e
n
t
s
(
m
o
d
e
l
)



s
a
m
p
l
e
_
b
a
t
c
h
 
=
 
n
e
x
t
(
i
t
e
r
(
t
e
s
t
_
l
o
a
d
e
r
)
)

s
a
m
p
l
e
_
b
a
t
c
h
_
d
e
v
i
c
e
 
=
 
m
o
v
e
_
b
a
t
c
h
_
t
o
_
d
e
v
i
c
e
(
s
a
m
p
l
e
_
b
a
t
c
h
,
 
d
e
v
i
c
e
)


w
i
t
h
 
t
o
r
c
h
.
n
o
_
g
r
a
d
(
)
:

 
 
 
 
o
u
t
p
u
t
 
=
 
m
o
d
e
l
(
*
*
s
a
m
p
l
e
_
b
a
t
c
h
_
d
e
v
i
c
e
)

 
 
 
 
p
r
o
b
s
 
=
 
o
u
t
p
u
t
[
"
y
_
p
r
o
b
"
]

 
 
 
 
p
r
e
d
s
 
=
 
t
o
r
c
h
.
a
r
g
m
a
x
(
p
r
o
b
s
,
 
d
i
m
=
-
1
)

 
 
 
 
l
a
b
e
l
_
k
e
y
 
=
 
m
o
d
e
l
.
l
a
b
e
l
_
k
e
y

 
 
 
 
t
r
u
e
_
l
a
b
e
l
 
=
 
s
a
m
p
l
e
_
b
a
t
c
h
_
d
e
v
i
c
e
[
l
a
b
e
l
_
k
e
y
]


 
 
 
 
p
r
i
n
t
(
"
\
n
M
o
d
e
l
 
p
r
e
d
i
c
t
i
o
n
 
f
o
r
 
t
h
e
 
s
a
m
p
l
e
d
 
p
a
t
i
e
n
t
:
"
)

 
 
 
 
p
r
i
n
t
(
f
"
 
 
T
r
u
e
 
l
a
b
e
l
:
 
{
i
n
t
(
t
r
u
e
_
l
a
b
e
l
.
c
p
u
(
)
[
0
]
.
i
t
e
m
(
)
)
}
"
)

 
 
 
 
p
r
i
n
t
(
f
"
 
 
P
r
e
d
i
c
t
e
d
 
c
l
a
s
s
:
 
{
i
n
t
(
p
r
e
d
s
.
c
p
u
(
)
[
0
]
.
i
t
e
m
(
)
)
}
"
)

 
 
 
 
p
r
i
n
t
(
f
"
 
 
P
r
o
b
a
b
i
l
i
t
i
e
s
:
 
{
p
r
o
b
s
[
0
]
.
c
p
u
(
)
.
n
u
m
p
y
(
)
}
"
)



a
t
t
r
i
b
u
t
i
o
n
s
 
=
 
i
g
.
a
t
t
r
i
b
u
t
e
(
*
*
s
a
m
p
l
e
_
b
a
t
c
h
_
d
e
v
i
c
e
)

p
r
i
n
t
_
t
o
p
_
a
t
t
r
i
b
u
t
i
o
n
s
(
a
t
t
r
i
b
u
t
i
o
n
s
,
 
s
a
m
p
l
e
_
b
a
t
c
h
_
d
e
v
i
c
e
,
 
i
n
p
u
t
_
p
r
o
c
e
s
s
o
r
s
,
 
t
o
p
_
k
=
1
0
)




M
o
d
e
l
 
p
r
e
d
i
c
t
i
o
n
 
f
o
r
 
t
h
e
 
s
a
m
p
l
e
d
 
p
a
t
i
e
n
t
:

 
 
T
r
u
e
 
l
a
b
e
l
:
 
0

 
 
P
r
e
d
i
c
t
e
d
 
c
l
a
s
s
:
 
0

 
 
P
r
o
b
a
b
i
l
i
t
i
e
s
:
 
[
0
.
5
1
6
2
8
5
4
]


F
e
a
t
u
r
e
:
 
i
c
d
_
c
o
d
e
s

 
 
 
1
.
 
i
d
x
=
[
0
,
 
7
]
 
t
o
k
e
n
=
'
E
6
6
9
:
 
O
b
e
s
i
t
y
,
 
u
n
s
p
e
c
i
f
i
e
d
'
 
a
t
t
r
=
-
0
.
0
0
2
0
2
2

 
 
 
2
.
 
i
d
x
=
[
0
,
 
9
]
 
t
o
k
e
n
=
'
Z
8
7
8
9
1
:
 
P
e
r
s
o
n
a
l
 
h
i
s
t
o
r
y
 
o
f
 
n
i
c
o
t
i
n
e
 
d
e
p
e
n
d
e
n
c
e
'
 
a
t
t
r
=
+
0
.
0
0
1
8
8
4

 
 
 
3
.
 
i
d
x
=
[
0
,
 
2
]
 
t
o
k
e
n
=
'
E
1
1
9
:
 
T
y
p
e
 
2
 
d
i
a
b
e
t
e
s
 
m
e
l
l
i
t
u
s
 
w
i
t
h
o
u
t
 
c
o
m
p
l
i
c
a
t
i
o
n
s
'
 
a
t
t
r
=
-
0
.
0
0
1
4
0
7

 
 
 
4
.
 
i
d
x
=
[
0
,
 
6
]
 
t
o
k
e
n
=
'
K
2
1
9
:
 
G
a
s
t
r
o
-
e
s
o
p
h
a
g
e
a
l
 
r
e
f
l
u
x
 
d
i
s
e
a
s
e
 
w
i
t
h
o
u
t
 
e
s
o
p
h
a
g
i
t
i
s
'
 
a
t
t
r
=
+
0
.
0
0
1
0
8
5

 
 
 
5
.
 
i
d
x
=
[
0
,
 
3
]
 
t
o
k
e
n
=
'
J
4
4
9
:
 

In [ ]:
d
e
f
 
b
u
i
l
d
_
r
a
n
d
o
m
_
e
m
b
e
d
d
i
n
g
_
b
a
s
e
l
i
n
e
(

 
 
 
 
m
o
d
e
l
:
 
S
t
a
g
e
N
e
t
,

 
 
 
 
b
a
t
c
h
:
 
d
i
c
t
,

 
 
 
 
s
c
a
l
e
:
 
f
l
o
a
t
 
=
 
0
.
0
1
,

 
 
 
 
s
e
e
d
:
 
i
n
t
 
=
 
4
2
,

)
 
-
>
 
d
i
c
t
:

 
 
 
 
"
"
"
C
o
n
s
t
r
u
c
t
 
a
 
n
o
n
-
e
m
p
t
y
 
b
a
s
e
l
i
n
e
 
d
i
r
e
c
t
l
y
 
i
n
 
e
m
b
e
d
d
i
n
g
 
s
p
a
c
e
.


 
 
 
 
D
e
e
p
L
I
F
T
 
s
u
b
t
r
a
c
t
s
 
t
h
e
 
b
a
s
e
l
i
n
e
 
e
m
b
e
d
d
i
n
g
 
f
r
o
m
 
t
h
e
 
a
c
t
u
a
l
 
e
m
b
e
d
d
i
n
g
.

 
 
 
 
U
s
i
n
g
 
p
u
r
e
 
z
e
r
o
s
 
c
o
l
l
a
p
s
e
s
 
S
t
a
g
e
N
e
t
 
m
a
s
k
s
 
(
a
l
l
 
v
i
s
i
t
s
 
b
e
c
o
m
e
 
p
a
d
d
i
n
g
)
,

 
 
 
 
s
o
 
w
e
 
a
d
d
 
s
m
a
l
l
 
r
a
n
d
o
m
 
n
o
i
s
e
 
t
o
 
k
e
e
p
 
a
t
 
l
e
a
s
t
 
o
n
e
 
t
i
m
e
s
t
e
p
 
a
c
t
i
v
e
.

 
 
 
 
"
"
"


 
 
 
 
t
o
r
c
h
.
m
a
n
u
a
l
_
s
e
e
d
(
s
e
e
d
)

 
 
 
 
f
e
a
t
u
r
e
_
i
n
p
u
t
s
 
=
 
{
}

 
 
 
 
f
o
r
 
k
e
y
 
i
n
 
m
o
d
e
l
.
f
e
a
t
u
r
e
_
k
e
y
s
:

 
 
 
 
 
 
 
 
v
a
l
u
e
 
=
 
b
a
t
c
h
[
k
e
y
]

 
 
 
 
 
 
 
 
i
f
 
i
s
i
n
s
t
a
n
c
e
(
v
a
l
u
e
,
 
t
u
p
l
e
)
:

 
 
 
 
 
 
 
 
 
 
 
 
v
a
l
u
e
 
=
 
v
a
l
u
e
[
1
]

 
 
 
 
 
 
 
 
f
e
a
t
u
r
e
_
i
n
p
u
t
s
[
k
e
y
]
 
=
 
v
a
l
u
e
.
t
o
(
m
o
d
e
l
.
d
e
v
i
c
e
)


 
 
 
 
e
m
b
e
d
d
e
d
 
=
 
m
o
d
e
l
.
e
m
b
e
d
d
i
n
g
_
m
o
d
e
l
(
f
e
a
t
u
r
e
_
i
n
p
u
t
s
)

 
 
 
 
b
a
s
e
l
i
n
e
 
=
 
{
}

 
 
 
 
f
o
r
 
k
e
y
,
 
e
m
b
 
i
n
 
e
m
b
e
d
d
e
d
.
i
t
e
m
s
(
)
:

 
 
 
 
 
 
 
 
b
a
s
e
l
i
n
e
[
k
e
y
]
 
=
 
t
o
r
c
h
.
r
a
n
d
n
_
l
i
k
e
(
e
m
b
)
 
*
 
s
c
a
l
e

 
 
 
 
r
e
t
u
r
n
 
b
a
s
e
l
i
n
e



In [ ]:
d
e
e
p
l
i
f
t
 
=
 
D
e
e
p
L
i
f
t
(
m
o
d
e
l
)


r
a
n
d
o
m
_
b
a
s
e
l
i
n
e
 
=
 
b
u
i
l
d
_
r
a
n
d
o
m
_
e
m
b
e
d
d
i
n
g
_
b
a
s
e
l
i
n
e
(
m
o
d
e
l
,
 
s
a
m
p
l
e
_
b
a
t
c
h
_
d
e
v
i
c
e
)

a
t
t
r
i
b
u
t
i
o
n
s
 
=
 
d
e
e
p
l
i
f
t
.
a
t
t
r
i
b
u
t
e
(

 
 
 
 
b
a
s
e
l
i
n
e
=
r
a
n
d
o
m
_
b
a
s
e
l
i
n
e
,

 
 
 
 
*
*
s
a
m
p
l
e
_
b
a
t
c
h
_
d
e
v
i
c
e
,

)

p
r
i
n
t
_
t
o
p
_
a
t
t
r
i
b
u
t
i
o
n
s
(
a
t
t
r
i
b
u
t
i
o
n
s
,
 
s
a
m
p
l
e
_
b
a
t
c
h
_
d
e
v
i
c
e
,
 
i
n
p
u
t
_
p
r
o
c
e
s
s
o
r
s
,
 
t
o
p
_
k
=
1
0
)




F
e
a
t
u
r
e
:
 
i
c
d
_
c
o
d
e
s

 
 
 
1
.
 
i
d
x
=
[
0
,
 
1
]
 
t
o
k
e
n
=
'
4
2
8
3
2
:
 
C
h
r
o
n
i
c
 
d
i
a
s
t
o
l
i
c
 
h
e
a
r
t
 
f
a
i
l
u
r
e
'
 
a
t
t
r
=
+
0
.
0
7
9
8
2
5

 
 
 
2
.
 
i
d
x
=
[
0
,
 
6
]
 
t
o
k
e
n
=
'
V
5
8
6
1
:
 
L
o
n
g
-
t
e
r
m
 
(
c
u
r
r
e
n
t
)
 
u
s
e
 
o
f
 
a
n
t
i
c
o
a
g
u
l
a
n
t
s
'
 
a
t
t
r
=
-
0
.
0
7
0
6
6
7

 
 
 
3
.
 
i
d
x
=
[
0
,
 
5
]
 
t
o
k
e
n
=
'
V
4
5
0
1
:
 
C
a
r
d
i
a
c
 
p
a
c
e
m
a
k
e
r
 
i
n
 
s
i
t
u
'
 
a
t
t
r
=
-
0
.
0
5
8
0
4
3

 
 
 
4
.
 
i
d
x
=
[
0
,
 
1
0
]
 
t
o
k
e
n
=
'
3
7
0
:
 
K
e
r
a
t
i
t
i
s
'
 
a
t
t
r
=
+
0
.
0
5
6
9
1
4

 
 
 
5
.
 
i
d
x
=
[
2
,
 
1
0
]
 
t
o
k
e
n
=
'
V
4
5
0
1
:
 
C
a
r
d
i
a
c
 
p
a
c
e
m
a
k
e
r
 
i
n
 
s
i
t
u
'
 
a
t
t
r
=
-
0
.
0
5
0
8
8
8

 
 
 
6
.
 
i
d
x
=
[
0
,
 
7
]
 
t
o
k
e
n
=
'
4
0
1
9
:
 
U
n
s
p
e
c
i
f
i
e
d
 
e
s
s
e
n
t
i
a
l
 
h
y
p
e
r
t
e
n
s
i
o
n
'
 
a
t
t
r
=
-
0
.
0
4
8
5
0
2

 
 
 
7
.
 
i
d
x
=
[
0
,
 
3
]
 
t
o
k
e
n
=
'
4
2
8
0
:
 
C
o
